In [1]:
from datetime import datetime
import os 
import pandas as pd
import numpy as np

In [2]:
bucket = os.getenv("WORKSPACE_BUCKET")

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')

In [3]:
%env USER_NAME={USER_NAME}

env: USER_NAME=williamsjacr


In [4]:
!pip3 install --upgrade dsub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.5/187.5 kB 1.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
%%writefile ~/aou_dsub.bash
#!/bin/bash
function aou_dsub () {

  # Get a shorter username to leave more characters for the job name.
  local DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

  # For AoU RWB projects network name is "network".
  local AOU_NETWORK=network
  local AOU_SUBNETWORK=subnetwork

  dsub \
      --provider google-cls-v2 \
      --user-project "${GOOGLE_PROJECT}"\
      --project "${GOOGLE_PROJECT}"\
      --image 'marketplace.gcr.io/google/ubuntu1804:latest' \
      --network "${AOU_NETWORK}" \
      --subnetwork "${AOU_SUBNETWORK}" \
      --service-account "$(gcloud config get-value account)" \
      --user "${DSUB_USER_NAME}" \
      --regions us-central1 \
      --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
      "$@"
}

Writing /home/jupyter/aou_dsub.bash


In [6]:
!echo source ~/aou_dsub.bash >> ~/.bashrc

In [7]:
%%bash 
dstat \
    --provider google-cls-v2 \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --users "${USER_NAME}" \
    --jobs '*' \
    --status '*' 

Job Name           Task  Status                                      Last Update
---------------  ------  ------------------------------------------  --------------
single-rarev...       6  Started running "user-command"              08-06 18:47:34
single-rarev...       5  Stopped running "user-command": exit st...  08-06 19:13:14
single-rarev...       4  Stopped running "user-command": exit st...  08-06 19:17:42
single-rarev...       3  Stopped running "user-command": exit st...  08-06 19:11:39
single-rarev...       2  Stopped running "user-command": exit st...  08-06 19:10:33
single-rarev...       1  Stopped running "user-command": exit st...  08-06 19:28:51
single-rarev...       6  The operation was cancelled                 08-06 18:29:01
single-rarev...       5  Stopped running "user-command": exit st...  08-06 18:09:51
single-rarev...       4  Stopped running "user-command": exit st...  08-06 18:14:04
single-rarev...       3  Stopped running "user-command": exit st...  08-06 18:0

coding-analysis     171  Success                                     08-04 21:39:33
coding-analysis     778  Success                                     08-04 21:35:34
coding-analysis     736  Success                                     08-04 21:31:55
coding-analysis     233  Success                                     08-04 21:37:29
coding-analysis      49  Success                                     08-04 21:38:26
coding-analysis     229  Success                                     08-04 21:30:54
coding-analysis     290  Success                                     08-04 22:07:05
coding-analysis    2274  Success                                     08-04 21:49:28
coding-analysis    2273  Success                                     08-04 21:58:39
coding-analysis    2272  Success                                     08-04 21:57:38
coding-analysis    2271  Success                                     08-04 21:42:18
coding-analysis    2270  Success                                     08-04 2

coding-analysis    2183  Success                                     08-04 21:42:57
coding-analysis    2182  Success                                     08-04 22:03:43
coding-analysis    2181  Success                                     08-04 21:42:58
coding-analysis    2180  Success                                     08-04 22:00:17
coding-analysis    2179  Success                                     08-04 21:57:49
coding-analysis    2178  Success                                     08-04 21:54:38
coding-analysis    2177  Success                                     08-04 21:51:00
coding-analysis    2176  Success                                     08-04 21:44:57
coding-analysis    2175  Success                                     08-04 21:47:30
coding-analysis    2174  Success                                     08-04 21:47:32
coding-analysis    2173  Success                                     08-04 21:48:17
coding-analysis    2172  Success                                     08-04 2

coding-analysis    2085  Success                                     08-04 21:55:26
coding-analysis    2084  Success                                     08-04 21:57:54
coding-analysis    2083  Success                                     08-04 21:43:28
coding-analysis    2082  Success                                     08-04 21:44:06
coding-analysis    2081  Success                                     08-04 21:58:55
coding-analysis    2080  Success                                     08-04 21:45:00
coding-analysis    2079  Success                                     08-04 21:58:58
coding-analysis    2078  Success                                     08-04 21:47:24
coding-analysis    2077  Success                                     08-04 21:41:58
coding-analysis    2076  Success                                     08-04 21:44:24
coding-analysis    2075  Success                                     08-04 21:52:11
coding-analysis    2074  Success                                     08-04 2

coding-analysis    1987  Success                                     08-04 21:52:51
coding-analysis    1986  Success                                     08-04 21:55:28
coding-analysis    1985  Success                                     08-04 21:57:55
coding-analysis    1984  Success                                     08-04 21:39:55
coding-analysis    1983  Success                                     08-04 21:47:35
coding-analysis    1982  Success                                     08-04 22:01:21
coding-analysis    1981  Success                                     08-04 21:43:43
coding-analysis    1980  Success                                     08-04 21:46:52
coding-analysis    1979  Success                                     08-04 22:02:15
coding-analysis    1978  Success                                     08-04 21:45:03
coding-analysis    1977  Success                                     08-04 21:58:07
coding-analysis    1976  Success                                     08-04 2

...

tune-validat...       1  Success                                     06-29 12:49:20
prosper-lass...       6  Stopped running "localization": exit st...  06-29 01:18:39
prosper-lass...       5  Stopped running "localization": exit st...  06-29 01:22:42
prosper-lass...       4  Stopped running "localization": exit st...  06-29 01:34:45
prosper-lass...       3  Stopped running "localization": exit st...  06-29 01:25:30
prosper-lass...       2  Stopped running "localization": exit st...  06-29 01:18:10
prosper-lass...       1  Stopped running "localization": exit st...  06-29 01:25:39
prosper-lass...       6  Stopped running "localization": exit st...  06-29 01:00:29
prosper-lass...       5  Stopped running "localization": exit st...  06-29 01:00:30
prosper-lass...       4  Stopped running "localization": exit st...  06-29 01:00:34
prosper-lass...       3  Stopped running "localization": exit st...  06-29 01:00:35
prosper-lass...       2  Stopped running "localization": exit st...  06-29 0

add-rsids-su...      17  Stopped running "localization": exit st...  06-28 13:30:08
add-rsids-su...      16  Stopped running "localization": exit st...  06-28 13:31:31
add-rsids-su...      15  Stopped running "localization": exit st...  06-28 13:31:35
add-rsids-su...      14  Stopped running "localization": exit st...  06-28 13:30:01
add-rsids-su...      13  Stopped running "localization": exit st...  06-28 13:29:58
add-rsids-su...      12  Stopped running "localization": exit st...  06-28 13:29:44
add-rsids-su...      11  Stopped running "localization": exit st...  06-28 13:30:05
add-rsids-su...      10  Stopped running "localization": exit st...  06-28 13:29:48
add-rsids-su...       9  Stopped running "localization": exit st...  06-28 13:29:51
add-rsids-su...       8  Stopped running "localization": exit st...  06-28 13:31:33
add-rsids-su...       7  Stopped running "localization": exit st...  06-28 13:29:52
add-rsids-su...       6  Stopped running "localization": exit st...  06-28 1

train-tune-v...       1  Success                                     06-27 15:37:44
train-tune-v...       1  Stopped running "user-command": exit st...  06-27 15:30:31
all-phenotypes        1  Success                                     06-27 15:26:30
all-phenotypes        1  Success                                     06-27 15:20:38
all-phenotypes        1  Success                                     06-27 15:14:34
all-phenotypes        1  Stopped running "user-command": exit st...  06-27 15:04:32
ct-sleb-dimct        12  The operation was cancelled                 06-27 14:37:42
ct-sleb-dimct        11  The operation was cancelled                 06-27 14:37:42
ct-sleb-dimct        10  The operation was cancelled                 06-27 14:37:42
ct-sleb-dimct         9  The operation was cancelled                 06-27 14:37:42
ct-sleb-dimct         8  The operation was cancelled                 06-27 14:37:42
ct-sleb-dimct         7  The operation was cancelled                 06-27 1

prosper-sums...       2  Success                                     06-26 12:23:17
prosper-sums...       1  Success                                     06-26 12:22:42
prosper-sums...      18  Stopped running "user-command": exit st...  06-25 21:24:18
prosper-sums...      17  Stopped running "user-command": exit st...  06-25 21:24:44
prosper-sums...      16  Stopped running "user-command": exit st...  06-25 21:24:21
prosper-sums...      15  Stopped running "user-command": exit st...  06-25 21:23:55
prosper-sums...      14  Stopped running "user-command": exit st...  06-25 21:24:18
prosper-sums...      13  Stopped running "user-command": exit st...  06-25 21:24:38
prosper-sums...      12  Stopped running "user-command": exit st...  06-25 21:24:04
prosper-sums...      11  Stopped running "user-command": exit st...  06-25 21:24:08
prosper-sums...      10  Stopped running "user-command": exit st...  06-25 21:24:26
prosper-sums...       9  Stopped running "user-command": exit st...  06-25 2

prosper-sums...      12  pulling image: docker pull: retry budge...  06-25 15:23:04
prosper-sums...      11  pulling image: docker pull: retry budge...  06-25 15:21:33
prosper-sums...      10  pulling image: docker pull: retry budge...  06-25 15:20:38
prosper-sums...       9  pulling image: docker pull: retry budge...  06-25 15:20:27
prosper-sums...       8  pulling image: docker pull: retry budge...  06-25 15:20:21
prosper-sums...       7  pulling image: docker pull: retry budge...  06-25 15:21:06
prosper-sums...       6  pulling image: docker pull: retry budge...  06-25 15:21:59
prosper-sums...       5  pulling image: docker pull: retry budge...  06-25 15:21:16
prosper-sums...       4  pulling image: docker pull: retry budge...  06-25 15:20:07
prosper-sums...       3  pulling image: docker pull: retry budge...  06-25 15:20:04
prosper-sums...       2  pulling image: docker pull: retry budge...  06-25 15:21:33
prosper-sums...       1  pulling image: docker pull: retry budge...  06-25 1

In [7]:
%%bash
ddel \
    --provider google-cls-v2 \
    --project "${GOOGLE_PROJECT}" \
    --location us-central1 \
    --jobs '*'\
    --users 'williamsjacr'

Delete running jobs:
  user:
    {'williamsjacr'}

  job-id:
    ['*']

Found 0 tasks to delete.


0 jobs deleted


In [ ]:
%%writefile Common_Variants.R
rm(list = ls())

chr <- as.numeric(commandArgs(TRUE)[1])
print(chr)

BED_file <- commandArgs(TRUE)[2]
BED_file <- gsub(".bed","",BED_file)
print(BED_file)

Ancestry_File <- commandArgs(TRUE)[5]
print(Ancestry_File)

OUTPUT_PATH <- commandArgs(TRUE)[6]
print(OUTPUT_PATH)


ancestries <- read.delim(Ancestry_File)
ancestries <- data.frame(IID = as.numeric(ancestries[,c("research_id")]),ancestry = toupper(ancestries[,c("ancestry_pred")]))


AFR_Keep <- data.frame(FID = 0,IID = ancestries$IID[ancestries$ancestry == "AFR"])
AMR_Keep <- data.frame(FID = 0,IID = ancestries$IID[ancestries$ancestry == "AMR"])
EUR_Keep <- data.frame(FID = 0,IID = ancestries$IID[ancestries$ancestry == "EUR"])


write.table(AFR_Keep,"AFR_Keep.txt",row.names = FALSE,col.names = FALSE)
write.table(AMR_Keep,"AMR_Keep.txt",row.names = FALSE,col.names = FALSE)
write.table(EUR_Keep,"EUR_Keep.txt",row.names = FALSE,col.names = FALSE)

system(paste0("plink --bfile ",BED_file," --keep AFR_Keep.txt --freq --out chr",chr,"_freq_AFR"),intern = TRUE)
system(paste0("plink --bfile ",BED_file," --keep AMR_Keep.txt --freq --out chr",chr,"_freq_AMR"),intern = TRUE)
system(paste0("plink --bfile ",BED_file," --keep EUR_Keep.txt --freq --out chr",chr,"_freq_EUR"),intern = TRUE)

a <- read.csv(paste0("chr",chr,"_freq_AFR.frq"),sep = "")
b <- read.csv(paste0("chr",chr,"_freq_AMR.frq"),sep = "")
c <- read.csv(paste0("chr",chr,"_freq_EUR.frq"),sep = "")

exclude_list <- a$SNP[is.na(a$MAF) | is.na(b$MAF) | is.na(c$MAF) | a$MAF <= 0.01 | b$MAF <= 0.01 | c$MAF <= 0.01]
write.table(exclude_list,"exclude_list.txt",row.names = F,col.names = F,quote=F)

system(paste0("plink --bfile ",BED_file," --keep AFR_Keep.txt --exclude exclude_list.txt --make-bed --out ",OUTPUT_PATH,"/AFR_chr",chr,"_corrected"),intern = TRUE)
system(paste0("plink --bfile ",BED_file," --keep AMR_Keep.txt --exclude exclude_list.txt --make-bed --out ",OUTPUT_PATH,"/AMR_chr",chr,"_corrected"),intern = TRUE)
system(paste0("plink --bfile ",BED_file," --keep EUR_Keep.txt --exclude exclude_list.txt --make-bed --out ",OUTPUT_PATH,"/EUR_chr",chr,"_corrected"),intern = TRUE)

In [ ]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for (chrom in 1:22) {
  tasks <- rbind(tasks, data.frame(
    '--env CHR'=chrom,
    '--input BED_File'=paste0("gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr",chrom,".bed"),
    '--input BIM_File'=paste0("gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr",chrom,".bim"),
    '--input FAM_File'=paste0("gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr",chrom,".fam"),
    '--input Ancestry_File'="gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/Common_Variants.R",
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants",
    # '--output out_file'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/test",chrom,".Rdata"),
    check.names = FALSE
  ))
}

write.table(tasks,
            file="score_task.txt",
            row.names=F, col.names=T,
            sep='\t', quote=F)

In [ ]:
%%writefile Common_Variants.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${CHR} ${BED_File} ${BIM_File} ${FAM_File} ${Ancestry_File} ${OUTPUT_PATH}

In [36]:
%%writefile Common_Variants_BED_Sum.R
rm(list = ls())

chr <- as.numeric(commandArgs(TRUE)[1])
print(chr)

BED_file <- commandArgs(TRUE)[2]
BED_file <- gsub(".bed","",BED_file,fixed = TRUE)
print(BED_file)

CT_BIM_FILE <- commandArgs(TRUE)[3]
print(CT_BIM_FILE)

PROSPER_BIM_FILE <- commandArgs(TRUE)[4]
print(PROSPER_BIM_FILE)

Ancestry_File <- commandArgs(TRUE)[5]
print(Ancestry_File)

OUTPUT_PATH <- commandArgs(TRUE)[6]
print(OUTPUT_PATH)


ancestries <- read.delim(Ancestry_File)
ancestries <- data.frame(IID = as.numeric(ancestries[,c("research_id")]),ancestry = toupper(ancestries[,c("ancestry_pred")]))


AFR_Keep <- data.frame(FID = 0,IID = ancestries$IID[ancestries$ancestry == "AFR"])
AMR_Keep <- data.frame(FID = 0,IID = ancestries$IID[ancestries$ancestry == "AMR"])
EUR_Keep <- data.frame(FID = 0,IID = ancestries$IID[ancestries$ancestry == "EUR"])


write.table(AFR_Keep,"AFR_Keep.txt",row.names = FALSE,col.names = FALSE)
write.table(AMR_Keep,"AMR_Keep.txt",row.names = FALSE,col.names = FALSE)
write.table(EUR_Keep,"EUR_Keep.txt",row.names = FALSE,col.names = FALSE)

system(paste0("plink --bfile ",BED_file," --keep AFR_Keep.txt --freq --out chr",chr,"_freq_AFR"),intern = TRUE)
system(paste0("plink --bfile ",BED_file," --keep AMR_Keep.txt --freq --out chr",chr,"_freq_AMR"),intern = TRUE)
system(paste0("plink --bfile ",BED_file," --keep EUR_Keep.txt --freq --out chr",chr,"_freq_EUR"),intern = TRUE)

a <- read.csv(paste0("chr",chr,"_freq_AFR.frq"),sep = "")
b <- read.csv(paste0("chr",chr,"_freq_AMR.frq"),sep = "")
c <- read.csv(paste0("chr",chr,"_freq_EUR.frq"),sep = "")

exclude_list <- a$SNP[is.na(a$MAF) | is.na(b$MAF) | is.na(c$MAF) | a$MAF <= 0.01 | b$MAF <= 0.01 | c$MAF <= 0.01]
rarevariant_list <- a$SNP[is.na(a$MAF) | is.na(b$MAF) | is.na(c$MAF)]

Number_Common_Variants <- nrow(a) - length(exclude_list)
Number_Rare_Variants <- nrow(a) - length(rarevariant_list) - Number_Common_Variants

CT_BIM <- read.delim(CT_BIM_FILE,header = FALSE)
CT_SNPs <- nrow(CT_BIM)

PROSPER_BIM <- read.delim(PROSPER_BIM_FILE,header = FALSE)
PROSPER_SNPs <- nrow(PROSPER_BIM)

tmp <- data.frame(Chr = chr,
                  Number_Common_Variants = Number_Common_Variants,
                  Number_Rare_Variants = Number_Rare_Variants,
                 CT_SNPs = CT_SNPs,
                 PROSPER_SNPs = PROSPER_SNPs)
write.csv(tmp,row.names = FALSE,file = paste0(OUTPUT_PATH,"/",chr,"_NumSNPs.csv"))


Overwriting Common_Variants_BED_Sum.R


In [37]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for (chrom in 1:22) {
  tasks <- rbind(tasks, data.frame(
    '--env CHR'=chrom,
    '--input BED_File'=paste0("gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/exome_v7.1/plink_bed/exome.chr",chrom,".bed"),
    '--input BIM_File'=paste0("gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/exome_v7.1/plink_bed/exome.chr",chrom,".bim"),
    '--input FAM_File'=paste0("gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/exome_v7.1/plink_bed/exome.chr",chrom,".fam"),
    '--input CT_BIM_FILE'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.bim",
    '--input PROSPER_BIM_FILE'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_JointPRS_MUSSEL_PROSPER.bim",  
    '--input Ancestry_File'="gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/Common_Variants_BED_Sum.R",
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Number_SNPs",
    check.names = FALSE
  ))
}

write.table(tasks,
            file="score_task.txt",
            row.names=F, col.names=T,
            sep='\t', quote=F)

Overwriting score_task.R


In [38]:
%%writefile Common_Variants_BED_Sum.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${CHR} ${BED_File} ${CT_BIM_FILE} ${PROSPER_BIM_FILE} ${Ancestry_File} ${OUTPUT_PATH}

Overwriting Common_Variants_BED_Sum.sh


In [39]:
!Rscript score_task.R

In [40]:
!gsutil -m cp Common_Variants_BED_Sum.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://Common_Variants_BED_Sum.R [Content-Type=application/octet-stream]...
/ [1/1 files][  2.3 KiB/  2.3 KiB] 100% Done                                    
Operation completed over 1 objects/2.3 KiB.                                      


In [41]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 400 \
  --boot-disk-size 25 \
  --min-ram 32 \
  --timeout "48h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Common_Variants_BED_Sum.sh \
  --tasks score_task.txt

Job properties:
  job-id: common-var--williamsjacr--240901-131943-66
  job-name: common-variants-bed-sum
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11196432129391680803
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/7824575572013587726
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/9758527684499917372
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/15865405136015907616
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/13738081057261705017
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/1215447884564362733
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/12706738842639517649
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/9128748637006943225
Pro

In [77]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'common-var--williamsjacr--240901-131943-66' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
common-varia...      22  Success   09-01 14:09:08
common-varia...      21  Success   09-01 13:33:01
common-varia...      20  Success   09-01 14:19:11
common-varia...      19  Success   09-01 16:18:57
common-varia...      18  Success   09-01 14:00:15
common-varia...      17  Success   09-01 15:48:39
common-varia...      16  Success   09-01 15:15:55
common-varia...      15  Success   09-01 14:50:13
common-varia...      14  Success   09-01 14:45:19
common-varia...      13  Success   09-01 14:04:41
common-varia...      12  Success   09-01 15:38:07
common-varia...      11  Success   09-01 15:49:05
common-varia...      10  Success   09-01 14:59:51
common-varia...       9  Success   09-01 15:02:14
common-varia...       8  Success   09-01 14:53:29
common-varia...       7  Success   09-01 15:14:49
common-varia...       6  Success   09-01 15:30:24
common-varia...       5  Success   09-01 15:21:54
com

In [ ]:
%%writefile merge_bed.R

rm(list = ls())

INPUT_PATH <- commandArgs(TRUE)[1]
print(INPUT_PATH)

OUTPUT_PATH <- commandArgs(TRUE)[2]
print(OUTPUT_PATH)

merge_list_train <- paste0(INPUT_PATH,"/chr",2:22,"_corrected")
write.table(merge_list_train,file = "merge_list.txt",col.names = F,row.names = F,quote=F)
system(paste0("plink --bfile ",INPUT_PATH,"/chr",1,"_corrected --merge-list merge_list.txt --make-bed --out ",OUTPUT_PATH,"/all_chr"))

In [ ]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

tasks <- rbind(tasks, data.frame(
  '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/merge_bed.R",
  '--input-recursive INPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants",
  '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants",
  # '--output out_file'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/test",chrom,".Rdata"),
  check.names = FALSE
))

write.table(tasks,
            file="score_task.txt",
            row.names=F, col.names=T,
            sep='\t', quote=F)

In [ ]:
%%writefile merge_bed.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${INPUT_PATH} ${OUTPUT_PATH}

In [58]:
%%writefile all_phenotypes.R

rm(list = ls())

Related_File <- commandArgs(TRUE)[1]
print(Related_File)

Ancestry_File <- commandArgs(TRUE)[2]
print(Ancestry_File)

INPUT_PATH <- commandArgs(TRUE)[3]
print(INPUT_PATH)

OUTPUT_PATH <- commandArgs(TRUE)[4]
print(OUTPUT_PATH)

flagged_relatedness <- read.delim(Related_File)
colnames(flagged_relatedness) <- "IID"

ancestries <- read.delim(Ancestry_File)

ancestries <- data.frame(IID = as.numeric(ancestries[,c("research_id")]),ancestry = toupper(ancestries[,c("ancestry_pred")]))

aou_BMI_pheno <- read.delim(paste0(INPUT_PATH,"/aou_BMI_pheno.tsv"))
aou_LDL_pheno <- read.delim(paste0(INPUT_PATH,"/aou_LDL_pheno.tsv"))
aou_HDL_pheno <- read.delim(paste0(INPUT_PATH,"/aou_HDL_pheno.tsv"))
aou_TG_pheno <- read.delim(paste0(INPUT_PATH,"/aou_TG_pheno.tsv"))
colnames(aou_TG_pheno)[3] <- "logTG"
aou_TC_pheno <- read.delim(paste0(INPUT_PATH,"/aou_TC_pheno.tsv"))
aou_Height_pheno <- read.csv(paste0(INPUT_PATH,"/aou_Height_pheno.tsv"),sep ="")

aou_BMI_cov <- read.delim(paste0(INPUT_PATH,"/aou_BMI_cov.tsv"))
aou_LDL_cov <- read.csv(paste0(INPUT_PATH,"/aou_LDL_cov.tsv"),sep ="")
aou_HDL_cov <- read.csv(paste0(INPUT_PATH,"/aou_HDL_cov.tsv"),sep ="")
aou_TG_cov <- read.csv(paste0(INPUT_PATH,"/aou_TG_cov.tsv"),sep ="")
aou_TC_cov <- read.csv(paste0(INPUT_PATH,"/aou_TC_cov.tsv"),sep ="")
aou_Height_cov <- read.csv(paste0(INPUT_PATH,"/aou_Height_cov.tsv"),sep ="")

library(dplyr)
unique_ids_pheno <- unique(c(aou_BMI_pheno$IID,aou_LDL_pheno$IID,aou_HDL_pheno$IID,aou_TG_pheno$IID,aou_TC_pheno$IID,aou_Height_pheno$IID))
unique_ids_cov <- unique(c(aou_BMI_cov$IID,aou_LDL_cov$IID,aou_HDL_cov$IID,aou_TG_cov$IID,aou_TC_cov$IID,aou_Height_cov$IID))


all_phenotypes <- data.frame(IID = unique(c(unique_ids_pheno,unique_ids_cov)))
all_phenotypes <- all_phenotypes[!(all_phenotypes$IID %in% flagged_relatedness$IID),,drop = FALSE]

all_phenotypes <- left_join(all_phenotypes,aou_BMI_pheno)
all_phenotypes <- left_join(all_phenotypes,aou_LDL_pheno)
all_phenotypes <- left_join(all_phenotypes,aou_HDL_pheno)
all_phenotypes <- left_join(all_phenotypes,aou_TG_pheno)
all_phenotypes <- left_join(all_phenotypes,aou_TC_pheno)
all_phenotypes <- left_join(all_phenotypes,aou_Height_pheno)

all_phenotypes <- left_join(all_phenotypes,aou_BMI_cov)
all_phenotypes <- left_join(all_phenotypes,aou_LDL_cov)
all_phenotypes <- left_join(all_phenotypes,aou_HDL_cov)
all_phenotypes <- left_join(all_phenotypes,aou_TG_cov)
all_phenotypes <- left_join(all_phenotypes,aou_TC_cov)
all_phenotypes <- left_join(all_phenotypes,aou_Height_cov)

all_phenotypes <- left_join(all_phenotypes,ancestries)

all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$age),]
all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$female),]
all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$PC1),]
all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$PC2),]
all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$PC3),]
all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$PC4),]
all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$PC5),]
all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$PC6),]
all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$PC7),]
all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$PC8),]
all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$PC9),]
all_phenotypes <- all_phenotypes[!is.na(all_phenotypes$PC10),]

write.csv(all_phenotypes,file = paste0(OUTPUT_PATH,"/all_phenotypes.csv"),row.names = FALSE)

Overwriting all_phenotypes.R


In [59]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

tasks <- rbind(tasks, data.frame(
  '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/all_phenotypes.R",
  '--input Related_File'="gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/relatedness/relatedness_flagged_samples.tsv",
  '--input Ancestry_File'="gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv",
  '--input-recursive INPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/Tony_Files",
  '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/",
  check.names = FALSE
))

write.table(tasks,
            file="score_task.txt",
            row.names=F, col.names=T,
            sep='\t', quote=F)

Overwriting score_task.R


In [60]:
%%writefile all_phenotypes.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${Related_File} ${Ancestry_File} ${INPUT_PATH} ${OUTPUT_PATH}

Overwriting all_phenotypes.sh


In [61]:
!Rscript score_task.R

In [62]:
!gsutil -m cp all_phenotypes.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://all_phenotypes.R [Content-Type=application/octet-stream]...
/ [1/1 files][  3.4 KiB/  3.4 KiB] 100% Done                                    
Operation completed over 1 objects/3.4 KiB.                                      


In [63]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "48h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script all_phenotypes.sh \
  --tasks score_task.txt

Job properties:
  job-id: all-phenot--williamsjacr--240627-152247-11
  job-name: all-phenotypes
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3165015638967736981
Launched job-id: all-phenot--williamsjacr--240627-152247-11
1 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'all-phenot--williamsjacr--240627-152247-11' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'all-phenot--williamsjacr--240627-152247-11' --users 'williamsjacr'


In [57]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'all-phenot--williamsjacr--240627-151710-43' --users 'williamsjacr' --status '*'

Job Name          Task  Status    Last Update
--------------  ------  --------  --------------
all-phenotypes       1  Success   06-27 15:20:38



In [73]:
%%writefile Train_Tune_Validation.R

rm(list = ls())

all_phenotype_File <- commandArgs(TRUE)[1]
print(all_phenotype_File)

OUTPUT_PATH <- commandArgs(TRUE)[2]
print(OUTPUT_PATH)

all_phenotypes <- read.csv(all_phenotype_File)

i <- (1:length(all_phenotypes$IID))[all_phenotypes$ancestry %in% c("EUR","AFR","AMR")]
train_number <- round(length(all_phenotypes$IID)*0.7) + 1
train <- sample(i, train_number)

i <- (1:length(all_phenotypes$IID))[!((1:length(all_phenotypes$IID)) %in% train)]
i_EUR <- i[all_phenotypes$ancestry[i] == "EUR"]
i_AFR <- i[all_phenotypes$ancestry[i] == "AFR"]
i_SAS <- i[all_phenotypes$ancestry[i] == "SAS"]
i_EAS <- i[all_phenotypes$ancestry[i] == "EAS"]
i_MID <- i[all_phenotypes$ancestry[i] == "MID"]
i_AMR <- i[all_phenotypes$ancestry[i] == "AMR"]


tune <- c(sample(i_EUR,round(length(i_EUR)/2)),
          sample(i_AFR,round(length(i_AFR)/2)),
          sample(i_SAS,round(length(i_SAS)/2)),
          sample(i_EAS,round(length(i_EAS)/2)),
          sample(i_MID,round(length(i_MID)/2)),
          sample(i_AMR,round(length(i_AMR)/2)))

validation <- i[!(i %in% tune)]


train <- all_phenotypes$IID[train]
tune <- all_phenotypes$IID[tune]
validation <- all_phenotypes$IID[validation]


write.table(train,paste0(OUTPUT_PATH,"/train.txt"),row.names = FALSE,col.names = FALSE)
write.table(tune,paste0(OUTPUT_PATH,"/tune.txt"),row.names = FALSE,col.names = FALSE)
write.table(validation,paste0(OUTPUT_PATH,"/validation.txt"),row.names = FALSE,col.names = FALSE)

reference <- train[sample(1:length(train),3000,replace = FALSE)]

write.table(reference,paste0(OUTPUT_PATH,"/reference.txt"),row.names = FALSE,col.names = FALSE)

colnames(all_phenotypes)[10] <- "sex"

phenotype_train <- all_phenotypes[all_phenotypes$IID %in% train,]
phenotype_tune <- all_phenotypes[all_phenotypes$IID %in% tune,]
phenotype_validation <- all_phenotypes[all_phenotypes$IID %in% validation,]

phenotype_train$FID <- 0
phenotype_tune$FID <- 0
phenotype_validation$FID <- 0

phenotype_train <- phenotype_train[,c("IID","FID","BMI","TC","HDL","LDL","logTG","Height","age","sex","PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10")]
phenotype_train$age2 <- phenotype_train$age^2
phenotype_tune <- phenotype_tune[,c("IID","FID","BMI","TC","HDL","LDL","logTG","Height","age","sex","PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10")]
phenotype_tune$age2 <- phenotype_tune$age^2
phenotype_validation <- phenotype_validation[,c("IID","FID","BMI","TC","HDL","LDL","logTG","Height","age","sex","PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10")]
phenotype_validation$age2 <- phenotype_validation$age^2

colnames(phenotype_train) <- c("IID",colnames(phenotype_train)[-1])
colnames(phenotype_tune) <- c("IID",colnames(phenotype_tune)[-1])
colnames(phenotype_validation) <- c("IID",colnames(phenotype_validation)[-1])

phenotype_train[c(9,11:21)] <- lapply(phenotype_train[c(9,11:21)], function(x){c(scale(x))})
phenotype_tune[c(9,11:21)] <- lapply(phenotype_tune[c(9,11:21)], function(x){c(scale(x))})
phenotype_validation[c(9,11:21)] <- lapply(phenotype_validation[c(9,11:21)], function(x){c(scale(x))})

write.table(phenotype_train,file = paste0(OUTPUT_PATH,"/All_Train.txt"),sep = '\t',row.names = FALSE,quote = FALSE)
write.table(phenotype_tune,file = paste0(OUTPUT_PATH,"/All_Tune.txt"),sep = '\t',row.names = FALSE,quote = FALSE)
write.table(phenotype_validation,file = paste0(OUTPUT_PATH,"/All_Validation.txt"),sep = '\t',row.names = FALSE,quote = FALSE)


Overwriting Train_Tune_Validation.R


In [74]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

tasks <- rbind(tasks, data.frame(
  '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/Train_Tune_Validation.R",
  '--input all_phenotype_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
  '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/",
  check.names = FALSE
))

write.table(tasks,
            file="score_task.txt",
            row.names=F, col.names=T,
            sep='\t', quote=F)

Overwriting score_task.R


In [75]:
%%writefile Train_Tune_Validation.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${all_phenotype_File} ${OUTPUT_PATH}

Overwriting Train_Tune_Validation.sh


In [76]:
!Rscript score_task.R

In [77]:
!gsutil -m cp Train_Tune_Validation.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://Train_Tune_Validation.R [Content-Type=application/octet-stream]...
/ [1/1 files][  3.4 KiB/  3.4 KiB] 100% Done                                    
Operation completed over 1 objects/3.4 KiB.                                      


In [78]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 800 \
  --boot-disk-size 25 \
  --min-ram 60 \
  --timeout "48h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Train_Tune_Validation.sh \
  --tasks score_task.txt

Job properties:
  job-id: train-tune--williamsjacr--240627-153410-54
  job-name: train-tune-validation
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/9958687970653510792
Launched job-id: train-tune--williamsjacr--240627-153410-54
1 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'train-tune--williamsjacr--240627-153410-54' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'train-tune--williamsjacr--240627-153410-54' --users 'williamsjacr'


In [81]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'train-tune--williamsjacr--240627-153410-54' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
train-tune-v...       1  Success   06-27 15:37:44



In [82]:
%%writefile GWAS_SumStats.R
rm(list = ls())

BED_file <- commandArgs(TRUE)[1]
BED_file <- gsub(".bed","",BED_file)
print(BED_file)

all_phenotypes_file <- commandArgs(TRUE)[2]
print(all_phenotypes_file)

all_train_file <- commandArgs(TRUE)[3]
print(all_phenotypes_file)

anc <- commandArgs(TRUE)[4]
print(anc)

trait <- commandArgs(TRUE)[5]
print(trait)

OUTPUT_PATH <- commandArgs(TRUE)[6]
print(OUTPUT_PATH)

all_phenotypes <- read.csv(all_phenotypes_file)

tmp <- read.delim(all_train_file)
write.table(tmp[tmp$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == anc],],file = paste0("All_Train_",anc,".txt"),sep = '\t',row.names = FALSE,quote = FALSE)

system(paste0("plink2 --bfile ",BED_file," --pheno All_Train_",anc,".txt --pheno-name ",trait," --linear --covar All_Train_",anc,".txt --covar-name age, age2, sex, PC1, PC2, PC3, PC4, PC5, PC6, PC7, PC8, PC9, PC10 --vif 999 --out ",OUTPUT_PATH,"/",trait,"_sumstats_",anc))


Writing GWAS_SumStats.R


In [83]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(anc in c("AFR","AMR","EUR")){
  for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
      '--input BED_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.bed",
      '--input BIM_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.bim",
      '--input FAM_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.fam",
      '--input all_phenotypes_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
      '--input all_train_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Train.txt",
      '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/GWAS_SumStats.R",
      '--env anc'=anc,
      '--env trait'=trait,
      '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats",
      check.names = FALSE
    ))
  }
}


write.table(tasks,
            file="score_task.txt",
            row.names=F, col.names=T,
            sep='\t', quote=F)


Overwriting score_task.R


In [84]:
%%writefile GWAS_SumStats.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${BED_File} ${all_phenotypes_file} ${all_train_file} ${anc} ${trait} ${OUTPUT_PATH}

Writing GWAS_SumStats.sh


In [85]:
!Rscript score_task.R

In [86]:
!gsutil -m cp GWAS_SumStats.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://GWAS_SumStats.R [Content-Type=application/octet-stream]...
/ [1/1 files][  905.0 B/  905.0 B] 100% Done                                    
Operation completed over 1 objects/905.0 B.                                      


In [87]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 800 \
  --boot-disk-size 25 \
  --min-ram 60 \
  --timeout "48h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script GWAS_SumStats.sh \
  --tasks score_task.txt

Job properties:
  job-id: gwas-sumst--williamsjacr--240627-154651-71
  job-name: gwas-sumstats
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/8204918055333401999
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/17136902588676638775
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/8833417735648720266
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/13338584417530022384
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11984449143327305597
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/14282491897173240579
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/8331165620674023108
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/2532056940772854370
Provider inte

In [101]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'gwas-sumst--williamsjacr--240627-154651-71' --users 'williamsjacr' --status '*'

Job Name         Task  Status                          Last Update
-------------  ------  ------------------------------  --------------
gwas-sumstats      18  Started running "localization"  06-27 15:49:27
gwas-sumstats      17  Started running "localization"  06-27 15:49:24
gwas-sumstats      16  Started running "localization"  06-27 15:49:39
gwas-sumstats      15  Started running "localization"  06-27 15:49:25
gwas-sumstats      14  Started running "localization"  06-27 15:49:57
gwas-sumstats      13  Started running "localization"  06-27 15:49:43
gwas-sumstats      12  Started running "localization"  06-27 15:49:27
gwas-sumstats      11  Started running "localization"  06-27 15:49:41
gwas-sumstats      10  Started running "localization"  06-27 15:49:40
gwas-sumstats       9  Started running "localization"  06-27 15:49:33
gwas-sumstats       8  Started running "localization"  06-27 15:49:26
gwas-sumstats       7  Started running "localization"  06-27 15:49:43
gwas-sumstats       6  

In [89]:
%%writefile tune_validation_anc_bed.R

rm(list = ls())

Tune_File <- commandArgs(TRUE)[1]
print(Tune_File)

Valid_File <- commandArgs(TRUE)[2]
print(Valid_File)

all_File <- commandArgs(TRUE)[3]
print(all_File)

BED_Full_File <- commandArgs(TRUE)[4]
BED_Full_File <- gsub(".bed","",BED_Full_File)
print(BED_Full_File)

anc <- commandArgs(TRUE)[5]
print(anc)

OUTPUT_PATH <- commandArgs(TRUE)[6]
print(OUTPUT_PATH)

Tune <- read.delim(Tune_File)
All <- read.csv(all_File)

Keep <- data.frame(FID = 0,IID = Tune$IID[Tune$IID %in% All$IID[All$ancestry == anc]])

write.table(Keep,"Keep.txt",row.names = FALSE,col.names = FALSE)

system(paste0("plink --bfile ",BED_Full_File," --keep Keep.txt --make-bed --out ",OUTPUT_PATH,"/all_tune_",anc),intern = TRUE)


Valid <- read.delim(Valid_File)
All <- read.csv(all_File)

Keep <- data.frame(FID = 0,IID = Valid$IID[Valid$IID %in% All$IID[All$ancestry == anc]])

write.table(Keep,"Keep.txt",row.names = FALSE,col.names = FALSE)

system(paste0("plink --bfile ",BED_Full_File," --keep Keep.txt --make-bed --out ",OUTPUT_PATH,"/all_valid_",anc),intern = TRUE)


Overwriting tune_validation_anc_bed.R


In [90]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(anc in c("AFR","EUR","AMR")){
    tasks <- rbind(tasks, data.frame(
    '--env anc'=anc,
    '--input Tune_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt",
    '--input Valid_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt",
    '--input all_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
    '--input BED_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.bed",
    '--input BIM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.bim",
    '--input FAM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.fam",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/tune_validation_anc_bed.R",
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_BedFiles",
    # '--output out_file'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/test",chrom,".Rdata"),
    check.names = FALSE
  ))
}


write.table(tasks,
            file="score_task.txt",
            row.names=F, col.names=T,
            sep='\t', quote=F)

Overwriting score_task.R


In [91]:
%%writefile tune_validation_anc_bed.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${Tune_File} ${Valid_File} ${all_File} ${BED_Full_File} ${anc} ${OUTPUT_PATH}

Overwriting tune_validation_anc_bed.sh


In [92]:
!Rscript score_task.R

In [93]:
!gsutil -m cp tune_validation_anc_bed.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://tune_validation_anc_bed.R [Content-Type=application/octet-stream]...
/ [1/1 files][  1.0 KiB/  1.0 KiB] 100% Done                                    
Operation completed over 1 objects/1.0 KiB.                                      


In [94]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 800 \
  --boot-disk-size 25 \
  --min-ram 80 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script tune_validation_anc_bed.sh \
  --tasks score_task.txt

Job properties:
  job-id: tune-valid--williamsjacr--240627-160659-03
  job-name: tune-validation-anc-bed
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11205462176011724146
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/13842708501326363877
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3863052655684454429
Launched job-id: tune-valid--williamsjacr--240627-160659-03
3 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'tune-valid--williamsjacr--240627-160659-03' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'tune-valid--williamsjacr--240627-160659-03' --users 'williamsjacr'


In [102]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'tune-valid--williamsjacr--240627-160659-03' --users 'williamsjacr' --status '*'

Job Name           Task  Status                                      Last Update
---------------  ------  ------------------------------------------  --------------
tune-validat...       3  Pulling "gcr.io/google.com/cloudsdktool...  06-27 16:07:39
tune-validat...       2  Pulling "gcr.io/google.com/cloudsdktool...  06-27 16:07:40
tune-validat...       1  Pulling "gcr.io/google.com/cloudsdktool...  06-27 16:07:37



In [10]:
%%writefile JointPRS_MUSSEL_PROSPER_BED_Files.R

rm(list = ls())

library(dplyr)

Tune_File <- commandArgs(TRUE)[1]
print(Tune_File)

Valid_File <- commandArgs(TRUE)[2]
print(Valid_File)

all_File <- commandArgs(TRUE)[3]
print(all_File)

snp_info_rsids <- commandArgs(TRUE)[4]
print(snp_info_rsids)

snp_info_extractlist <- commandArgs(TRUE)[5]
print(snp_info_extractlist)

BED_Full_File <- commandArgs(TRUE)[6]
BED_Full_File <- gsub(".bed","",BED_Full_File)
print(BED_Full_File)

OUTPUT_PATH <- commandArgs(TRUE)[7]
print(OUTPUT_PATH)

all_chr_bim <- read.delim(paste0(BED_Full_File,".bim"), header=FALSE)
all_chr_bim <- as.data.frame(all_chr_bim)
colnames(all_chr_bim) <- c("CHR","SNP","IDK","BP","A1","A2")

all_chr_bim$snp <- all_chr_bim$SNP
all_chr_bim$rs_ID <- all_chr_bim$SNP

all_chr_bim$SNP <- toupper(all_chr_bim$SNP)
all_chr_bim$SNP <- gsub(" ","",all_chr_bim$SNP)

SNP_GRCh37_38_match_update <- readRDS(snp_info_rsids)

system(paste0("rm ",snp_info_rsids))

SNP_GRCh37_38_match_update$unique_id1 <- paste0("chr",SNP_GRCh37_38_match_update$chr,":",SNP_GRCh37_38_match_update$pos38,":",
                                                SNP_GRCh37_38_match_update$allele1_38,":",SNP_GRCh37_38_match_update$allele2_38)
SNP_GRCh37_38_match_update$unique_id2 <- paste0("chr",SNP_GRCh37_38_match_update$chr,":",SNP_GRCh37_38_match_update$pos38,":",
                                                SNP_GRCh37_38_match_update$allele2_38,":",SNP_GRCh37_38_match_update$allele1_38)

SNP_GRCh37_38_match_update$unique_id1 <- toupper(SNP_GRCh37_38_match_update$unique_id1)
SNP_GRCh37_38_match_update$unique_id1 <- gsub(" ","",SNP_GRCh37_38_match_update$unique_id1)

SNP_GRCh37_38_match_update$unique_id2 <- toupper(SNP_GRCh37_38_match_update$unique_id2)
SNP_GRCh37_38_match_update$unique_id2 <- gsub(" ","",SNP_GRCh37_38_match_update$unique_id2)

all_chr_bim <- left_join(all_chr_bim,SNP_GRCh37_38_match_update[,c("unique_id1","rsid")],by = c("SNP"="unique_id1"))
all_chr_bim$rs_ID[!is.na(all_chr_bim$rsid)] <- all_chr_bim$rsid[!is.na(all_chr_bim$rsid)]
    
all_chr_bim <- subset(all_chr_bim,select = -c(rsid))
    
all_chr_bim <- left_join(all_chr_bim,SNP_GRCh37_38_match_update[,c("unique_id2","rsid")],by = c("SNP"="unique_id2"))
all_chr_bim$rs_ID[!is.na(all_chr_bim$rsid)] <- all_chr_bim$rsid[!is.na(all_chr_bim$rsid)]

snp_info_extractlist <- read.csv(snp_info_extractlist)
extract_list <- all_chr_bim$snp[all_chr_bim$rs_ID %in% snp_info_extractlist$SNP]
write.table(extract_list,"extract_list.txt",row.names = F,col.names = F,quote=F)

# Extract Needed SNPs in Full File
system(paste0("plink --bfile ",BED_Full_File," --extract extract_list.txt --make-bed --out ",OUTPUT_PATH,"/all_JointPRS_MUSSEL_PROSPER"),intern = TRUE)

system(paste0("rm ",BED_Full_File,".bed"))
system(paste0("rm ",BED_Full_File,".bim"))
system(paste0("rm ",BED_Full_File,".fam"))

write.table(all_chr_bim[,c("snp","rs_ID")],file = "snp_rename.txt",row.names = F,col.names = F,quote=F)

# Rename SNPs to rsids
system(paste0("plink --bfile ",OUTPUT_PATH,"/all_JointPRS_MUSSEL_PROSPER --update-name snp_rename.txt 2 1 --make-bed --out ",OUTPUT_PATH,"/all_JointPRS_MUSSEL_PROSPER"))

Tune <- read.delim(Tune_File)

Keep <- data.frame(FID = 0,IID = Tune$IID)
write.table(Keep,"Keep.txt",row.names = FALSE,col.names = FALSE)

## Full Tune
system(paste0("plink --bfile ",OUTPUT_PATH,"/all_JointPRS_MUSSEL_PROSPER --keep Keep.txt --make-bed --out ",OUTPUT_PATH,"/all_tune_JointPRS_MUSSEL_PROSPER"),intern = TRUE)

All <- read.csv(all_File)

# Anc Specific Tune
for(anc in c("AFR","EUR","AMR")){
    Keep <- data.frame(FID = 0,IID = Tune$IID[Tune$IID %in% All$IID[All$ancestry == anc]])
    write.table(Keep,"Keep.txt",row.names = FALSE,col.names = FALSE)

    system(paste0("plink --bfile ",OUTPUT_PATH,"/all_JointPRS_MUSSEL_PROSPER --keep Keep.txt --make-bed --out ",OUTPUT_PATH,"/all_tune_JointPRS_MUSSEL_PROSPER_",anc),intern = TRUE)
}


Valid <- read.delim(Valid_File)

Keep <- data.frame(FID = 0,IID = Valid$IID)
write.table(Keep,"Keep.txt",row.names = FALSE,col.names = FALSE)

## Full Valid
system(paste0("plink --bfile ",OUTPUT_PATH,"/all_JointPRS_MUSSEL_PROSPER --keep Keep.txt --make-bed --out ",OUTPUT_PATH,"/all_valid_JointPRS_MUSSEL_PROSPER"),intern = TRUE)

All <- read.csv(all_File)

# Anc Specific Tune
for(anc in c("AFR","EUR","AMR")){
    Keep <- data.frame(FID = 0,IID = Valid$IID[Valid$IID %in% All$IID[All$ancestry == anc]])
    write.table(Keep,"Keep.txt",row.names = FALSE,col.names = FALSE)

    system(paste0("plink --bfile ",OUTPUT_PATH,"/all_JointPRS_MUSSEL_PROSPER --keep Keep.txt --make-bed --out ",OUTPUT_PATH,"/all_Valid_JointPRS_MUSSEL_PROSPER_",anc),intern = TRUE)
}

Overwriting JointPRS_MUSSEL_PROSPER_BED_Files.R


In [11]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)


tasks <- rbind(tasks, data.frame(
    '--input Tune_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt",
    '--input Valid_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt",
    '--input all_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
    '--input snp_info_rsids'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/SNP_GRCh37_38_match_update.rds",
    '--input snp_info_extractlist'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/SNP_Info_JointPRS_PROSPER_MUSSEL.csv",
    '--input BED_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.bed",
    '--input BIM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.bim",
    '--input FAM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.fam",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/JointPRS_MUSSEL_PROSPER_BED_Files.R",
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED",
    check.names = FALSE
  ))



write.table(tasks,
            file="score_task.txt",
            row.names=F, col.names=T,
            sep='\t', quote=F)

Overwriting score_task.R


In [12]:
%%writefile JointPRS_MUSSEL_PROSPER_BED_Files.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${Tune_File} ${Valid_File} ${all_File} ${snp_info_rsids} ${snp_info_extractlist} ${BED_Full_File} ${OUTPUT_PATH}

Overwriting JointPRS_MUSSEL_PROSPER_BED_Files.sh


In [13]:
!Rscript score_task.R

In [14]:
!gsutil -m cp JointPRS_MUSSEL_PROSPER_BED_Files.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://JointPRS_MUSSEL_PROSPER_BED_Files.R [Content-Type=application/octet-stream]...
/ [1/1 files][  4.5 KiB/  4.5 KiB] 100% Done                                    
Operation completed over 1 objects/4.5 KiB.                                      


In [15]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 800 \
  --boot-disk-size 25 \
  --min-ram 64 \
  --timeout "168h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script JointPRS_MUSSEL_PROSPER_BED_Files.sh \
  --tasks score_task.txt

Job properties:
  job-id: jointprs-m--williamsjacr--240720-133736-74
  job-name: jointprs-mussel-prosper-bed-files
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/6898823270404457997
Launched job-id: jointprs-m--williamsjacr--240720-133736-74
1 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'jointprs-m--williamsjacr--240720-133736-74' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'jointprs-m--williamsjacr--240720-133736-74' --users 'williamsjacr'


In [8]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'jointprs-m--williamsjacr--240720-133736-74' --users 'williamsjacr' --status '*'

Job Name           Task  Status                          Last Update
---------------  ------  ------------------------------  --------------
jointprs-mus...       1  Started running "user-command"  07-20 16:15:24



In [9]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/jointprs-m--williamsjacr--240720-133736-74.1-stdout.log

[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/SNP_GRCh37_38_match_update.rds"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/SNP_Info_JointPRS_PROSPER_MUSSEL.csv"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr"
[1] "/mnt/data/output/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED"
 [1] "PLINK v1.90b7.2 64-bit (11 Dec 2023)           www.cog-genomics.org/plink/1.9/"                                                                      

In [9]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/jointprs-m--williamsjacr--240715-165205-70.1-stdout.log

[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/SNP_GRCh37_38_match_update.rds"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/SNP_Info_JointPRS_PROSPER_MUSSEL.csv"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr"
[1] "/mnt/data/output/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED"
 [1] "PLINK v1.90b7.2 64-bit (11 Dec 2023)           www.cog-genomics.org/plink/1.9/"                                                                      

In [59]:
%%writefile tune_validation_bed.R

rm(list = ls())

Tune_File <- commandArgs(TRUE)[1]
print(Tune_File)

Valid_File <- commandArgs(TRUE)[2]
print(Valid_File)

BED_Full_File <- commandArgs(TRUE)[3]
BED_Full_File <- gsub(".bed","",BED_Full_File)
print(BED_Full_File)

OUTPUT_PATH <- commandArgs(TRUE)[4]
print(OUTPUT_PATH)

Tune <- read.delim(Tune_File)

Keep <- data.frame(FID = 0,IID = Tune$IID)

write.table(Keep,"Keep.txt",row.names = FALSE,col.names = FALSE)

system(paste0("plink --bfile ",BED_Full_File," --keep Keep.txt --make-bed --out ",OUTPUT_PATH,"/all_tune"),intern = TRUE)






Valid <- read.delim(Valid_File)

Keep <- data.frame(FID = 0,IID = Valid$IID)

write.table(Keep,"Keep.txt",row.names = FALSE,col.names = FALSE)

system(paste0("plink --bfile ",BED_Full_File," --keep Keep.txt --make-bed --out ",OUTPUT_PATH,"/all_valid"),intern = TRUE)


Writing tune_validation_bed.R


In [60]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

tasks <- rbind(tasks, data.frame(
    '--input Tune_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt",
    '--input Valid_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt",
    '--input BED_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.bed",
    '--input BIM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.bim",
    '--input FAM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.fam",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/tune_validation_bed.R",
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_BedFiles",
    # '--output out_file'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/test",chrom,".Rdata"),
    check.names = FALSE
  ))


write.table(tasks,
            file="score_task.txt",
            row.names=F, col.names=T,
            sep='\t', quote=F)

Overwriting score_task.R


In [61]:
%%writefile tune_validation_bed.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${Tune_File} ${Valid_File} ${BED_Full_File} ${OUTPUT_PATH}

Writing tune_validation_bed.sh


In [62]:
!Rscript score_task.R

In [63]:
!gsutil -m cp tune_validation_bed.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://tune_validation_bed.R [Content-Type=application/octet-stream]...
/ [1/1 files][  823.0 B/  823.0 B] 100% Done                                    
Operation completed over 1 objects/823.0 B.                                      


In [64]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 800 \
  --boot-disk-size 40 \
  --min-ram 80 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script tune_validation_bed.sh \
  --tasks score_task.txt

Job properties:
  job-id: tune-valid--williamsjacr--240629-011433-61
  job-name: tune-validation-bed
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/15401220709288402463
Launched job-id: tune-valid--williamsjacr--240629-011433-61
1 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'tune-valid--williamsjacr--240629-011433-61' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'tune-valid--williamsjacr--240629-011433-61' --users 'williamsjacr'


In [65]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'tune-valid--williamsjacr--240629-011433-61' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
tune-validat...       1  Pending   06-29 01:14:33



In [21]:
%%writefile tune_validation_phenotypes.R

rm(list = ls())

Tune_File <- commandArgs(TRUE)[1]
print(Tune_File)

Valid_File <- commandArgs(TRUE)[2]
print(Valid_File)

all_File <- commandArgs(TRUE)[3]
print(all_File)

OUTPUT_PATH <- commandArgs(TRUE)[4]
print(OUTPUT_PATH)

Tune <- read.delim(Tune_File)
All <- read.csv(all_File)

covar_tmp <- data.frame(FID = Tune$FID,IID = Tune$IID,age = Tune$age,age2 = Tune$age2,sex = Tune$sex,
                        PC1 = Tune$PC1,PC2 = Tune$PC2,PC3 = Tune$PC3,PC4 = Tune$PC4,PC5 = Tune$PC5,
                        PC6 = Tune$PC6,PC7 = Tune$PC7,PC8 = Tune$PC8,PC9 = Tune$PC9,PC10 = Tune$PC10)

write.table(covar_tmp,file = paste0(OUTPUT_PATH,"/full_tune_covar.txt"),sep = '\t',row.names = FALSE,quote = FALSE,col.names = FALSE)
    
for(anc in c("AFR","AMR","EUR")){
    covar_tmp_anc <- covar_tmp[covar_tmp$IID %in% All$IID[All$ancestry == anc],]
    write.table(covar_tmp_anc,file = paste0(OUTPUT_PATH,"/full_tune_covar_",anc,".txt"),sep = '\t',row.names = FALSE,quote = FALSE,col.names = FALSE)
}

for(trait in c("BMI","LDL","HDL","logTG","TC","Height")){
    fam_tmp <- data.frame(FID = Tune$FID,IID = Tune$IID,Y = Tune[,trait])
    write.table(fam_tmp,file = paste0(OUTPUT_PATH,"/",trait,"_full_tune_y.txt"),sep = '\t',row.names = FALSE,quote = FALSE,col.names = FALSE)
    
    for(anc in c("AFR","AMR","EUR")){
        fam_tmp_anc <- fam_tmp[fam_tmp$IID %in% All$IID[All$ancestry == anc],]
        write.table(fam_tmp_anc,file = paste0(OUTPUT_PATH,"/",trait,"_full_tune_y_",anc,".txt"),sep = '\t',row.names = FALSE,quote = FALSE,col.names = FALSE)
    }
}




Valid <- read.delim(Valid_File)
All <- read.csv(all_File)

covar_tmp <- data.frame(FID = Valid$FID,IID = Valid$IID,age = Valid$age,age2 = Valid$age2,sex = Valid$sex,
                        PC1 = Valid$PC1,PC2 = Valid$PC2,PC3 = Valid$PC3,PC4 = Valid$PC4,PC5 = Valid$PC5,
                        PC6 = Valid$PC6,PC7 = Valid$PC7,PC8 = Valid$PC8,PC9 = Valid$PC9,PC10 = Valid$PC10)

write.table(covar_tmp,file = paste0(OUTPUT_PATH,"/full_valid_covar.txt"),sep = '\t',row.names = FALSE,quote = FALSE,col.names = FALSE)
    
for(anc in c("AFR","AMR","EUR")){
    covar_tmp_anc <- covar_tmp[covar_tmp$IID %in% All$IID[All$ancestry == anc],]
    write.table(covar_tmp_anc,file = paste0(OUTPUT_PATH,"/full_valid_covar_",anc,".txt"),sep = '\t',row.names = FALSE,quote = FALSE,col.names = FALSE)
}

for(trait in c("BMI","LDL","HDL","logTG","TC","Height")){
    fam_tmp <- data.frame(FID = Valid$FID,IID = Valid$IID,Y = Valid[,trait])
    write.table(fam_tmp,file = paste0(OUTPUT_PATH,"/",trait,"_full_valid_y.txt"),sep = '\t',row.names = FALSE,quote = FALSE,col.names = FALSE)
    
    for(anc in c("AFR","AMR","EUR")){
        fam_tmp_anc <- fam_tmp[fam_tmp$IID %in% All$IID[All$ancestry == anc],]
        write.table(fam_tmp_anc,file = paste0(OUTPUT_PATH,"/",trait,"_full_valid_y_",anc,".txt"),sep = '\t',row.names = FALSE,quote = FALSE,col.names = FALSE)
    }
}



Writing tune_validation_phenotypes.R


In [22]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

tasks <- rbind(tasks, data.frame(
    '--input Tune_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt",
    '--input Valid_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt",
    '--input all_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/tune_validation_phenotypes.R",
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes",
    check.names = FALSE
  ))


write.table(tasks,
            file="score_task.txt",
            row.names=F, col.names=T,
            sep='\t', quote=F)

Overwriting score_task.R


In [23]:
%%writefile tune_validation_phenotypes.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${Tune_File} ${Valid_File} ${all_File} ${OUTPUT_PATH}

Writing tune_validation_phenotypes.sh


In [24]:
!Rscript score_task.R

In [25]:
!gsutil -m cp tune_validation_phenotypes.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://tune_validation_phenotypes.R [Content-Type=application/octet-stream]...
/ [1/1 files][  2.8 KiB/  2.8 KiB] 100% Done                                    
Operation completed over 1 objects/2.8 KiB.                                      


In [26]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 40 \
  --boot-disk-size 25 \
  --min-ram 8 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script tune_validation_phenotypes.sh \
  --tasks score_task.txt

Job properties:
  job-id: tune-valid--williamsjacr--240722-175637-83
  job-name: tune-validation-phenotypes
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/16970955615058126312
Launched job-id: tune-valid--williamsjacr--240722-175637-83
1 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'tune-valid--williamsjacr--240722-175637-83' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'tune-valid--williamsjacr--240722-175637-83' --users 'williamsjacr'


In [27]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'tune-valid--williamsjacr--240722-175637-83' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
tune-validat...       1  Pending   07-22 17:56:37



In [15]:
%%writefile GWAS_SumStats_Clean.R

sumstats <- commandArgs(TRUE)[1]

anc <- commandArgs(TRUE)[2]
print(anc)

trait <- commandArgs(TRUE)[3]
print(trait)

OUTPUT_PATH <- commandArgs(TRUE)[4]
print(OUTPUT_PATH)

sumstats <- read.delim(sumstats, header=FALSE, comment.char="#")
colnames(sumstats) <- c("CHROM","POS","ID","REF","ALT","PROVISIONAL_REF","A1","OMITTED","A1_FREQ","TEST","OBS_CT","BETA","SE","T_STAT","P","ERRCODE")
sumstats <- sumstats[sumstats$TEST == "ADD",]
sumstats <- sumstats[,c("CHROM","ID","POS","REF","ALT","A1","OBS_CT","BETA","SE","P")]
colnames(sumstats) <- c("CHR","SNP","BP","REF","ALT","A1","OBS_CT","BETA","SE","P")
write.table(sumstats, file = paste0(OUTPUT_PATH,"/",anc,"_",trait,"_GWAS_SumStats_Cleaned"), sep = "\t", row.names = FALSE, quote = FALSE)


Overwriting GWAS_SumStats_Clean.R


In [16]:
%%writefile GWAS_SumStats_Clean.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${sumstats} ${anc} ${trait} ${OUTPUT_PATH}

Writing GWAS_SumStats_Clean.sh


In [17]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

anc <- "EUR"
trait <- "HDL"

for(anc in c("EUR","AMR","AFR")){
    for(trait in c("BMI","LDL","HDL","Height","TC","logTG")){
     tasks <- rbind(tasks, data.frame(
    '--input sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats/",trait,"_sumstats_",anc,".",trait,".glm.linear"),
    '--env anc'=anc,
    '--env trait'=trait,
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/GWAS_SumStats_Clean.R",
    check.names = FALSE
))   
    }
}

write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Writing score_task.R


In [18]:
!Rscript score_task.R

In [19]:
!gsutil -m cp GWAS_SumStats_Clean.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://GWAS_SumStats_Clean.R [Content-Type=application/octet-stream]...
/ [1/1 files][  746.0 B/  746.0 B] 100% Done                                    
Operation completed over 1 objects/746.0 B.                                      


In [20]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 800 \
  --boot-disk-size 25 \
  --min-ram 60 \
  --timeout "48h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script GWAS_SumStats_Clean.sh \
  --tasks score_task.txt

Job properties:
  job-id: gwas-sumst--williamsjacr--240628-124107-15
  job-name: gwas-sumstats-clean
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11902671578390765630
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/14232277539497694331
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/9176029849610361544
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11783176353215920730
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/9411797687353540377
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/17709795272143767001
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/12861438726488854482
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11276180165803765558
Provi

In [28]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'gwas-sumst--williamsjacr--240628-124107-15' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
gwas-sumstat...      18  Success   06-28 13:18:49
gwas-sumstat...      17  Success   06-28 13:19:27
gwas-sumstat...      16  Success   06-28 13:22:32
gwas-sumstat...      15  Success   06-28 13:19:52
gwas-sumstat...      14  Success   06-28 13:20:00
gwas-sumstat...      13  Success   06-28 13:22:37
gwas-sumstat...      12  Success   06-28 13:19:44
gwas-sumstat...      11  Success   06-28 13:17:56
gwas-sumstat...      10  Success   06-28 13:15:51
gwas-sumstat...       9  Success   06-28 13:20:56
gwas-sumstat...       8  Success   06-28 13:19:23
gwas-sumstat...       7  Success   06-28 13:23:57
gwas-sumstat...       6  Success   06-28 13:20:20
gwas-sumstat...       5  Success   06-28 13:22:08
gwas-sumstat...       4  Success   06-28 13:17:51
gwas-sumstat...       3  Success   06-28 13:25:17
gwas-sumstat...       2  Success   06-28 13:19:29
gwas-sumstat...       1  Success   06-28 13:19:24



In [22]:
%%writefile GWAS_SumStats_Clean_QQPlots.R

sumstats <- commandArgs(TRUE)[1]

anc <- commandArgs(TRUE)[2]
print(anc)

trait <- commandArgs(TRUE)[3]
print(trait)

OUTPUT_PATH <- commandArgs(TRUE)[4]
print(OUTPUT_PATH)

sumstats <- read.delim(sumstats, header=FALSE, comment.char="#")
colnames(sumstats) <- c("CHROM","POS","ID","REF","ALT","PROVISIONAL_REF","A1","OMITTED","A1_FREQ","TEST","OBS_CT","BETA","SE","T_STAT","P","ERRCODE")
sumstats <- sumstats[sumstats$TEST == "ADD",]
sumstats <- sumstats[,c("CHROM","ID","POS","A1_FREQ","P")]
colnames(sumstats) <- c("CHR","SNP","BP","A1_FREQ","P")
write.table(sumstats, file = paste0(OUTPUT_PATH,"/",anc,"_",trait,"_GWAS_SumStats_Cleaned_QQPlots"), sep = "\t", row.names = FALSE, quote = FALSE)


Overwriting GWAS_SumStats_Clean_QQPlots.R


In [23]:
%%writefile GWAS_SumStats_Clean_QQPlots.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${sumstats} ${anc} ${trait} ${OUTPUT_PATH}

Overwriting GWAS_SumStats_Clean_QQPlots.sh


In [24]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

anc <- "EUR"
trait <- "HDL"

for(anc in c("EUR","AMR","AFR")){
    for(trait in c("BMI","LDL","HDL","Height","TC","logTG")){
     tasks <- rbind(tasks, data.frame(
    '--input sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats/",trait,"_sumstats_",anc,".",trait,".glm.linear"),
    '--env anc'=anc,
    '--env trait'=trait,
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/GWAS_SumStats_Clean_QQPlots.R",
    check.names = FALSE
))   
    }
}

write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [25]:
!Rscript score_task.R

In [26]:
!gsutil -m cp GWAS_SumStats_Clean_QQPlots.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://GWAS_SumStats_Clean_QQPlots.R [Content-Type=application/octet-stream]...
/ [1/1 files][  698.0 B/  698.0 B] 100% Done                                    
Operation completed over 1 objects/698.0 B.                                      


In [27]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 200 \
  --boot-disk-size 25 \
  --min-ram 60 \
  --timeout "48h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script GWAS_SumStats_Clean_QQPlots.sh \
  --tasks score_task.txt

Job properties:
  job-id: gwas-sumst--williamsjacr--240813-154726-79
  job-name: gwas-sumstats-clean-qqplots
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/5519588274004611166
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/7476960220488265516
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/6735816343148071838
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/13156461183141985096
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/5264466287709118151
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11897381666247388351
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/17490078748917350948
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/12347047526151723993

In [29]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'gwas-sumst--williamsjacr--240813-154726-79' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
gwas-sumstat...      18  Success   08-13 16:26:08
gwas-sumstat...      17  Success   08-13 16:25:11
gwas-sumstat...      16  Success   08-13 16:25:27
gwas-sumstat...      15  Success   08-13 16:23:44
gwas-sumstat...      14  Success   08-13 16:25:01
gwas-sumstat...      13  Success   08-13 16:22:28
gwas-sumstat...      12  Success   08-13 16:22:16
gwas-sumstat...      11  Success   08-13 16:21:35
gwas-sumstat...      10  Success   08-13 16:25:12
gwas-sumstat...       9  Success   08-13 16:29:40
gwas-sumstat...       8  Success   08-13 16:36:59
gwas-sumstat...       7  Success   08-13 16:22:48
gwas-sumstat...       6  Success   08-13 16:25:23
gwas-sumstat...       5  Success   08-13 16:24:32
gwas-sumstat...       4  Success   08-13 16:22:15
gwas-sumstat...       3  Success   08-13 16:23:24
gwas-sumstat...       2  Success   08-13 16:23:29
gwas-sumstat...       1  Success   08-13 16:25:01



In [21]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/gwas-sumst--williamsjacr--240813-144353-97.1-stderr.log

/mnt/data/script/GWAS_SumStats_Clean_QQPlots.sh: line 6:     7 Killed                  Rscript ${R_Script} ${sumstats} ${anc} ${trait} ${OUTPUT_PATH}


In [37]:
%%writefile Add_RSIDs_Sumstats.R
rm(list = ls())

sumstats <- commandArgs(TRUE)[1]
print(sumstats)

snp_info <- commandArgs(TRUE)[2]
print(snp_info)

anc <- commandArgs(TRUE)[3]
print(anc)

trait <- commandArgs(TRUE)[4]
print(trait)

OUTPUT_PATH <- commandArgs(TRUE)[5]
print(OUTPUT_PATH)

library(data.table)
library(dplyr)

SNP_GRCh37_38_match_update <- readRDS(snp_info)

SNP_GRCh37_38_match_update$unique_id1 <- paste0("chr",SNP_GRCh37_38_match_update$chr,":",SNP_GRCh37_38_match_update$pos38,":",
                                                SNP_GRCh37_38_match_update$allele1_38,":",SNP_GRCh37_38_match_update$allele2_38)
SNP_GRCh37_38_match_update$unique_id2 <- paste0("chr",SNP_GRCh37_38_match_update$chr,":",SNP_GRCh37_38_match_update$pos38,":",
                                                SNP_GRCh37_38_match_update$allele2_38,":",SNP_GRCh37_38_match_update$allele1_38)

SNP_GRCh37_38_match_update$unique_id1 <- toupper(SNP_GRCh37_38_match_update$unique_id1)
SNP_GRCh37_38_match_update$unique_id1 <- gsub(" ","",SNP_GRCh37_38_match_update$unique_id1)

SNP_GRCh37_38_match_update$unique_id2 <- toupper(SNP_GRCh37_38_match_update$unique_id2)
SNP_GRCh37_38_match_update$unique_id2 <- gsub(" ","",SNP_GRCh37_38_match_update$unique_id2)


sumstats <- fread(sumstats)
    
sumstats <- as.data.frame(sumstats)
sumstats <- sumstats[,c("CHR","SNP","BP","REF","ALT","A1","OBS_CT","BETA","SE","P","SNP")]
colnames(sumstats) <- c("CHR","SNP","BP","REF","ALT","A1","OBS_CT","BETA","SE","P","rs_ID")
    
sumstats$SNP <- toupper(sumstats$SNP)
sumstats$SNP <- gsub(" ","",sumstats$SNP)
    
sumstats <- left_join(sumstats,SNP_GRCh37_38_match_update[,c("unique_id1","rsid")],by = c("SNP"="unique_id1"))
sumstats$rs_ID[!is.na(sumstats$rsid)] <- sumstats$rsid[!is.na(sumstats$rsid)]
    
sumstats <- subset(sumstats,select = -c(rsid))
    
sumstats <- left_join(sumstats,SNP_GRCh37_38_match_update[,c("unique_id2","rsid")],by = c("SNP"="unique_id2"))
sumstats$rs_ID[!is.na(sumstats$rsid)] <- sumstats$rsid[!is.na(sumstats$rsid)]
    
sumstats <- sumstats[,c("CHR","SNP","BP","REF","ALT","A1","OBS_CT","BETA","SE","P","rs_ID")]
    
sumstats$SNP <- sumstats$rs_ID
    
doubles <- names(which(table(sumstats$rs_ID) > 1))
    
sumstats <- sumstats[!(sumstats$SNP %in% doubles),]
    
write.csv(sumstats,file = paste0(OUTPUT_PATH,"/",anc,"_",trait,"_GWAS_SumStats_RSIDs.csv"),row.names = FALSE)

Overwriting Add_RSIDs_Sumstats.R


In [38]:
%%writefile Add_RSIDs_Sumstats.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${sumstats} ${snp_info} ${anc} ${trait} ${OUTPUT_PATH}

Overwriting Add_RSIDs_Sumstats.sh


In [39]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

anc <- "AFR"
trait <- "LDL"

for(anc in c("EUR","AFR","AMR")){
   for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
    '--input sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats/",anc,"_",trait,"_GWAS_SumStats_Cleaned"),
    '--input snp_info'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/SNP_GRCh37_38_match_update.rds",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/Add_RSIDs_Sumstats.R",
    '--env anc'=anc,
    '--env trait'=trait,
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats",
    check.names = FALSE
))   
   }
}



write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [40]:
!Rscript score_task.R

In [41]:
!gsutil -m cp Add_RSIDs_Sumstats.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://Add_RSIDs_Sumstats.R [Content-Type=application/octet-stream]...
/ [1/1 files][  2.3 KiB/  2.3 KiB] 100% Done                                    
Operation completed over 1 objects/2.3 KiB.                                      


In [42]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 200 \
  --boot-disk-size 25 \
  --min-ram 80 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Add_RSIDs_Sumstats.sh \
  --tasks score_task.txt

Job properties:
  job-id: add-rsids---williamsjacr--240628-133112-38
  job-name: add-rsids-sumstats
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/596445960202423148
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10680619813942882207
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11523552278899343275
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/14699910471220127595
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/14751023867850149884
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10842220169385773957
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/2424441718384203511
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/4459686452747780496
Provider

In [45]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'add-rsids---williamsjacr--240628-133112-38' --users 'williamsjacr' --status '*'

Job Name           Task  Status                          Last Update
---------------  ------  ------------------------------  --------------
add-rsids-su...      18  Started running "user-command"  06-28 13:34:13
add-rsids-su...      17  Started running "user-command"  06-28 13:34:04
add-rsids-su...      16  Started running "user-command"  06-28 13:34:16
add-rsids-su...      15  Started running "user-command"  06-28 13:34:12
add-rsids-su...      14  Started running "user-command"  06-28 13:34:25
add-rsids-su...      13  Started running "user-command"  06-28 13:34:22
add-rsids-su...      12  Started running "user-command"  06-28 13:34:05
add-rsids-su...      11  Started running "user-command"  06-28 13:34:05
add-rsids-su...      10  Started running "user-command"  06-28 13:34:09
add-rsids-su...       9  Started running "user-command"  06-28 13:34:22
add-rsids-su...       8  Started running "user-command"  06-28 13:34:09
add-rsids-su...       7  Started running "user-command"  06-28 13:3

In [9]:
%%writefile PROSPER_SumStats_Format.R
rm(list = ls())

sumstats <- commandArgs(TRUE)[1]
print(sumstats)

ref_bim <- commandArgs(TRUE)[2]
print(ref_bim)

anc <- commandArgs(TRUE)[3]
print(anc)

trait <- commandArgs(TRUE)[4]
print(trait)

OUTPUT_PATH <- commandArgs(TRUE)[5]
print(OUTPUT_PATH)

library(data.table)
library(dplyr)

sumstats <- fread(sumstats)
    
sumstats <- as.data.frame(sumstats)
colnames(sumstats) <- c("CHR","SNP","BP","REF","ALT","A1","OBS_CT","BETA","SE","P","rs_ID")

sumstats <- sumstats[,c("rs_ID","CHR","ALT","REF","BETA","SE","OBS_CT")]
colnames(sumstats) <- c("rsid","chr","a1","a0","beta","beta_se","n_eff")

sumstats$n_eff <- as.numeric(sumstats$n_eff)

ref_bim <- fread(ref_bim)
ref_bim <- as.data.frame(ref_bim)

sumstats <- sumstats[sumstats$rsid %in% ref_bim[,2],]

ref_bim <- ref_bim[ref_bim[,2] %in% sumstats$rsid,]

order_vec <- match(sumstats$rsid,ref_bim[,2])

ref_bim <- ref_bim[order_vec,]

a <- which((sumstats$a0 != ref_bim[,6]) & (sumstats$a1 != ref_bim[,6]))

b <- which((sumstats$a0 != ref_bim[,5]) & (sumstats$a1 != ref_bim[,5]))

sumstats$a0[a] <- ref_bim$V5[a]
sumstats$a1[a] <- ref_bim$V6[a]

b <- which((sumstats$a0 != ref_bim[,5]) & (sumstats$a1 != ref_bim[,5]))

if(length(b) > 0){
  sumstats$a0[b] <- ref_bim$V5[b]
  sumstats$a1[b] <- ref_bim$V6[b]
}

print(summary(sumstats$n_eff))

sumstats <- sumstats[sumstats$n_eff > (median(sumstats$n_eff)*.9),]

write.csv(sumstats,file = paste0(OUTPUT_PATH,"/",anc,"_",trait,"_GWAS_SumStats_PROSPER.csv"),row.names = FALSE)

Overwriting PROSPER_SumStats_Format.R


In [10]:
%%writefile PROSPER_SumStats_Format.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${sumstats} ${ref_bim} ${anc} ${trait} ${OUTPUT_PATH}

Overwriting PROSPER_SumStats_Format.sh


In [11]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

anc <- "AFR"
trait <- "LDL"

for(anc in c("EUR","AFR","AMR")){
   for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
    '--input sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats/",anc,"_",trait,"_GWAS_SumStats_RSIDs.csv"),
    '--input ref_bim'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/PROSPER/ref_bim.txt",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/PROSPER_SumStats_Format.R",
    '--env anc'=anc,
    '--env trait'=trait,
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER",
    check.names = FALSE
))   
   }
}



write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [12]:
!Rscript score_task.R

In [13]:
!gsutil -m cp PROSPER_SumStats_Format.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://PROSPER_SumStats_Format.R [Content-Type=application/octet-stream]...
/ [1/1 files][  1.4 KiB/  1.4 KiB] 100% Done                                    
Operation completed over 1 objects/1.4 KiB.                                      


In [14]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 200 \
  --boot-disk-size 25 \
  --min-ram 8 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script PROSPER_SumStats_Format.sh \
  --tasks score_task.txt

Job properties:
  job-id: prosper-su--williamsjacr--240721-124238-39
  job-name: prosper-sumstats-format
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/5943329446903232607
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10164156545079215933
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/6012435391604352405
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/5775686596901906735
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3353824575108734335
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/17537601535375617419
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/6819809204638678646
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/16083125786427904745
Prov

In [15]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-su--williamsjacr--240721-124238-39' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
prosper-sums...      18  Pending   07-21 12:42:39
prosper-sums...      17  Pending   07-21 12:42:39
prosper-sums...      16  Pending   07-21 12:42:39
prosper-sums...      15  Pending   07-21 12:42:39
prosper-sums...      14  Pending   07-21 12:42:39
prosper-sums...      13  Pending   07-21 12:42:39
prosper-sums...      12  Pending   07-21 12:42:39
prosper-sums...      11  Pending   07-21 12:42:39
prosper-sums...      10  Pending   07-21 12:42:39
prosper-sums...       9  Pending   07-21 12:42:39
prosper-sums...       8  Pending   07-21 12:42:39
prosper-sums...       7  Pending   07-21 12:42:38
prosper-sums...       6  Pending   07-21 12:42:38
prosper-sums...       5  Pending   07-21 12:42:38
prosper-sums...       4  Pending   07-21 12:42:38
prosper-sums...       3  Pending   07-21 12:42:38
prosper-sums...       2  Pending   07-21 12:42:38
prosper-sums...       1  Pending   07-21 12:42:38



In [10]:
%%writefile MUSSEL_SumStats_Format.R
rm(list = ls())

sumstats <- commandArgs(TRUE)[1]
print(sumstats)

ref_bim <- commandArgs(TRUE)[2]
print(ref_bim)

anc <- commandArgs(TRUE)[3]
print(anc)

trait <- commandArgs(TRUE)[4]
print(trait)

OUTPUT_PATH <- commandArgs(TRUE)[5]
print(OUTPUT_PATH)

library(data.table)
library(dplyr)

sumstats <- fread(sumstats)
    
sumstats <- as.data.frame(sumstats)
colnames(sumstats) <- c("CHR","SNP","BP","REF","ALT","A1","OBS_CT","BETA","SE","P","rs_ID")

sumstats <- sumstats[,c("rs_ID","CHR","ALT","REF","BETA","SE","OBS_CT")]
colnames(sumstats) <- c("rsid","chr","a1","a0","beta","beta_se","n_eff")

sumstats$n_eff <- as.numeric(sumstats$n_eff)

ref_bim <- fread(ref_bim)
ref_bim <- as.data.frame(ref_bim)

sumstats <- sumstats[sumstats$rsid %in% ref_bim[,2],]

ref_bim <- ref_bim[ref_bim[,2] %in% sumstats$rsid,]

order_vec <- match(sumstats$rsid,ref_bim[,2])

ref_bim <- ref_bim[order_vec,]

a <- which((sumstats$a0 != ref_bim[,6]) & (sumstats$a1 != ref_bim[,6]))

b <- which((sumstats$a0 != ref_bim[,5]) & (sumstats$a1 != ref_bim[,5]))

sumstats$a0[a] <- ref_bim$V5[a]
sumstats$a1[a] <- ref_bim$V6[a]

b <- which((sumstats$a0 != ref_bim[,5]) & (sumstats$a1 != ref_bim[,5]))

if(length(b) > 0){
  sumstats$a0[b] <- ref_bim$V5[b]
  sumstats$a1[b] <- ref_bim$V6[b]
}

print(summary(sumstats$n_eff))

sumstats <- sumstats[sumstats$n_eff > (median(sumstats$n_eff)*.9),]

write.csv(sumstats,file = paste0(OUTPUT_PATH,"/",anc,"_",trait,"_GWAS_SumStats_MUSSEL.csv"),row.names = FALSE)

Overwriting MUSSEL_SumStats_Format.R


In [11]:
%%writefile MUSSEL_SumStats_Format.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${sumstats} ${ref_bim} ${anc} ${trait} ${OUTPUT_PATH}

Overwriting MUSSEL_SumStats_Format.sh


In [12]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

anc <- "AFR"
trait <- "LDL"

for(anc in c("EUR","AFR","AMR")){
   for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
    '--input sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats/",anc,"_",trait,"_GWAS_SumStats_RSIDs.csv"),
    '--input ref_bim'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/MUSSEL/MUSSEL/ref_bim.txt",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/MUSSEL_SumStats_Format.R",
    '--env anc'=anc,
    '--env trait'=trait,
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/MUSSEL",
    check.names = FALSE
))   
   }
}



write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [13]:
!Rscript score_task.R

In [14]:
!gsutil -m cp MUSSEL_SumStats_Format.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://MUSSEL_SumStats_Format.R [Content-Type=application/octet-stream]...
/ [1/1 files][  1.4 KiB/  1.4 KiB] 100% Done                                    
Operation completed over 1 objects/1.4 KiB.                                      


In [15]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 200 \
  --boot-disk-size 25 \
  --min-ram 8 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script MUSSEL_SumStats_Format.sh \
  --tasks score_task.txt

Job properties:
  job-id: mussel-sum--williamsjacr--240722-122648-79
  job-name: mussel-sumstats-format
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/18368647893015607889
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11161361928737729861
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3139711350381230008
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/6110467123046620153
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/1005811247559210268
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10423943641961380668
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/15924590537914389518
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3826222295720045467
Prov

In [16]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'mussel-sum--williamsjacr--240722-122648-79' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
mussel-sumst...      18  Pending   07-22 12:26:50
mussel-sumst...      17  Pending   07-22 12:26:50
mussel-sumst...      16  Pending   07-22 12:26:49
mussel-sumst...      15  Pending   07-22 12:26:49
mussel-sumst...      14  Pending   07-22 12:26:49
mussel-sumst...      13  Pending   07-22 12:26:49
mussel-sumst...      12  Pending   07-22 12:26:49
mussel-sumst...      11  Pending   07-22 12:26:49
mussel-sumst...      10  Pending   07-22 12:26:49
mussel-sumst...       9  Pending   07-22 12:26:49
mussel-sumst...       8  Pending   07-22 12:26:49
mussel-sumst...       7  Pending   07-22 12:26:49
mussel-sumst...       6  Pending   07-22 12:26:49
mussel-sumst...       5  Pending   07-22 12:26:49
mussel-sumst...       4  Pending   07-22 12:26:49
mussel-sumst...       3  Pending   07-22 12:26:49
mussel-sumst...       2  Pending   07-22 12:26:49
mussel-sumst...       1  Pending   07-22 12:26:48



In [9]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/mussel-sum--williamsjacr--240721-124323-60.1.log

2024-07-21 12:46:42 INFO: gsutil -h Content-Type:text/plain -u terra-vpc-sc-804f445b -mq cp /tmp/continuous_logging_action/output gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/mussel-sum--williamsjacr--240721-124323-60.1.log
2024-07-21 12:46:42 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats
2024-07-21 12:46:42 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts
2024-07-21 12:46:42 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/MUSSEL
2024-07-21 12:46:42 INFO: mkdir -m 777 -p /mnt/data/output/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/MUSSEL/
2024-07-21 12:46:43 INFO: Localizing sumstats
2024-07-21 12:46:43 INFO: gsutil  -u terra-vpc-sc-804f445b -mq cp gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/

In [18]:
%%writefile CT_SLEB_dimCT_prseb.R
rm(list = ls())

library(devtools)
library(CTSLEB)
library(data.table)
library(dplyr)

BED_Tune_File <- commandArgs(TRUE)[1]
BED_Tune_File <- gsub(".bed","",BED_Tune_File)
print(BED_Tune_File)

BED_Full_File <- commandArgs(TRUE)[2]
BED_Full_File <- gsub(".bed","",BED_Full_File)
print(BED_Full_File)

#EUR
BED_Ref_pop1_File <- commandArgs(TRUE)[3]
BED_Ref_pop1_File <- gsub(".bed","",BED_Ref_pop1_File)
print(BED_Ref_pop1_File)

#ANC
BED_Ref_pop2_File <- commandArgs(TRUE)[4]
BED_Ref_pop2_File <- gsub(".bed","",BED_Ref_pop2_File)
print(BED_Ref_pop2_File)

pop1_sumstats <- commandArgs(TRUE)[5]
print(pop1_sumstats)

pop2_sumstats <- commandArgs(TRUE)[6]
print(pop2_sumstats)

all_phenotypes_file <- commandArgs(TRUE)[7]
print(all_phenotypes_file)

all_tune_file <- commandArgs(TRUE)[8]
print(all_tune_file)

anc <- commandArgs(TRUE)[9]
print(anc)

trait <- commandArgs(TRUE)[10]
print(trait)

OUTPUT_PATH <- commandArgs(TRUE)[11]
print(OUTPUT_PATH)

system("mkdir CTSLEB_TestCode")

sum_pop1 <- fread(pop1_sumstats)
    
sum_pop1 <- as.data.frame(sum_pop1)
sum_pop1 <- sum_pop1[,c("CHR","SNP","BP","REF","ALT","A1","OBS_CT","BETA","SE","P","SNP")]
colnames(sum_pop1) <- c("CHR","SNP","BP","REF","ALT","A1","OBS_CT","BETA","SE","P","rs_ID")
sum_pop1 <- sum_pop1[,c("CHR","SNP","BP","A1","BETA","SE","P","rs_ID")]

sum_pop1$P <- as.numeric(sum_pop1$P)
sum_pop1 <- sum_pop1[which(sum_pop1$P < 0.1),]

sum_pop2 <- fread(pop2_sumstats)
    
sum_pop2 <- as.data.frame(sum_pop2)
sum_pop2 <- sum_pop2[,c("CHR","SNP","BP","REF","ALT","A1","OBS_CT","BETA","SE","P","SNP")]
colnames(sum_pop2) <- c("CHR","SNP","BP","REF","ALT","A1","OBS_CT","BETA","SE","P","rs_ID")
sum_pop2 <- sum_pop2[,c("CHR","SNP","BP","A1","BETA","SE","P","rs_ID")]

sum_pop2$P <- as.numeric(sum_pop2$P)
sum_pop2 <- sum_pop2[which(sum_pop2$P < 0.1),]

print(nrow(sum_pop1));print(nrow(sum_pop2))

PRS_farm <- SetParamsFarm(plink19_exec = "plink",plink2_exec = "plink2",mem = 80490,pthres = c(5E-08,5E-07,5E-06,5E-05,5E-04,5E-03,0.05,0.1))

prs_mat <- dimCT(results_dir = "CTSLEB_TestCode/",sum_target = sum_pop2,sum_ref = sum_pop1,
                 ref_plink = BED_Ref_pop1_File,
                 target_plink = BED_Ref_pop2_File,
                 test_target_plink = BED_Tune_File,
                 out_prefix = "dimCT",
                 params_farm = PRS_farm)

# save(prs_mat,file = paste0(OUTPUT_PATH,"/",anc,"_",trait,"_prs_mat_dimCT.RData"))

pheno_tuning <- read.delim(all_tune_file)

pheno_tuning <- left_join(pheno_tuning,prs_mat,by = "IID")

model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_tuning)

n.total.prs <- length(pthres)^2*length(r2_vec)*length(wc_base_vec)
prs_r2_vec_test <- rep(0,n.total.prs)
indices <- (ncol(pheno_tuning) - n.total.prs + 1):ncol(pheno_tuning)

for(p_ind in 1:n.total.prs){
  prs <- pheno_tuning[!is.na(pheno_tuning[,trait]),indices[p_ind]]
  model <- lm(model.null$residual~prs)
  prs_r2_vec_test[p_ind] <- summary(model)$r.square
}

max_ind <- which.max(prs_r2_vec_test)

print(colnames(prs_mat)[indices[max_ind]])

best_snps <- colnames(prs_mat)[indices[max_ind]]
#calculate eb effect using EB coefficients
prs_mat_eb <- CalculateEBEffectSize(bfile = BED_Full_File,
                                   snp_ind = best_snps,
                                   plink_list = plink_list,
                                   out_prefix = "EB",
                                   results_dir = "CTSLEB_TestCode/",
                                   params_farm = PRS_farm)



score_full <- matrix(NA,ncol = ncol(prs_mat_eb),nrow = nrow(unique_infor_post))
names_score_full <- NULL
count <- 1
for(i in 1:length(pthres)){
  for(j in 1:length(pthres)){
    for(k in 1:24){
      tmp <- matrix(0,nrow = nrow(unique_infor_post),ncol = 1)
      tmp[which(unique_infor_post$P < pthres[j] | unique_infor_post$P_ref < pthres[i]),1] <- plink_list_eb$scores_eb[which(unique_infor_post$P < pthres[j] | unique_infor_post$P_ref < pthres[i]),k + 2]
      names_score_full <- c(names_score_full,paste0(colnames(plink_list_eb$scores_eb)[k + 2],"_p_other_",pthres[i],"_p_tar_",pthres[j]))
      score_full[,count] <- tmp[,1]
      count <- count + 1
    }
  }
}
  
final <- data.frame(SNP = unique_infor_post$SNP,A1 = unique_infor_post$A1,score_full)
colnames(final) <- c("SNP","A1",names_score_full)


save(final,file = paste0(OUTPUT_PATH,"/",anc,"_",trait,"_betas_eb.RData"))
save(prs_mat_eb,file = paste0(OUTPUT_PATH,"/",anc,"_",trait,"_prs_mat_eb.RData"))

Overwriting CT_SLEB_dimCT_prseb.R


In [19]:
%%writefile CT_SLEB_dimCT_prseb.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${BED_Tune_File} ${BED_Full_File} ${BED_Ref_pop1_File} ${BED_Ref_pop2_File} ${pop1_sumstats} ${pop2_sumstats} ${all_phenotypes_file} ${all_tune_file} ${anc} ${trait} ${OUTPUT_PATH}

Overwriting CT_SLEB_dimCT_prseb.sh


In [20]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

anc <- "AFR"
trait <- "LDL"

for(anc in c("AFR","AMR")){
   for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
    '--input BED_Tune_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_BedFiles/all_tune_",anc,".bed"),
    '--input BIM_Tune_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_BedFiles/all_tune_",anc,".bim"),
    '--input FAM_Tune_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_BedFiles/all_tune_",anc,".fam"),
    '--input BED_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.bed",
    '--input BIM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.bim",
    '--input FAM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr.fam",
    '--input BED_Ref_pop1_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr_reference_","EUR",".bed"),
    '--input BIM_Ref_pop1_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr_reference_","EUR",".bim"),
    '--input FAM_Ref_pop1_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr_reference_","EUR",".fam"),
    '--input BED_Ref_pop2_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr_reference_",anc,".bed"),
    '--input BIM_Ref_pop2_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr_reference_",anc,".bim"),
    '--input FAM_Ref_pop2_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/CommonVariants/all_chr_reference_",anc,".fam"),
    '--input pop1_sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats/EUR_",trait,"_GWAS_SumStats_Cleaned"),
    '--input pop2_sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GWAS_SummaryStats/",anc,"_",trait,"_GWAS_SumStats_Cleaned"),
    '--input all_phenotypes_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
    '--input all_tune_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/CT_SLEB_dimCT_prseb.R",
    '--env anc'=anc,
    '--env trait'=trait,
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/CTSLEB",
    check.names = FALSE
))   
   }
}



write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [21]:
!Rscript score_task.R

In [22]:
!gsutil -m cp CT_SLEB_dimCT_prseb.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://CT_SLEB_dimCT_prseb.R [Content-Type=application/octet-stream]...
/ [1/1 files][  4.4 KiB/  4.4 KiB] 100% Done                                    
Operation completed over 1 objects/4.4 KiB.                                      


In [23]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 800 \
  --boot-disk-size 25 \
  --min-ram 64 \
  --timeout "144h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script CT_SLEB_dimCT_prseb.sh \
  --tasks score_task.txt

Job properties:
  job-id: ct-sleb-di--williamsjacr--240722-123250-39
  job-name: ct-sleb-dimct-prseb
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3266674414708638102
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/17516157729654335075
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/15003487564096970620
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10431250898719543418
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/17480979131394875063
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/12974867493337158376
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/9396346449314849525
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/7046307428555817315
Provid

In [24]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'ct-sleb-di--williamsjacr--240722-123250-39' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
ct-sleb-dimc...      12  Pending   07-22 12:32:51
ct-sleb-dimc...      11  Pending   07-22 12:32:51
ct-sleb-dimc...      10  Pending   07-22 12:32:51
ct-sleb-dimc...       9  Pending   07-22 12:32:51
ct-sleb-dimc...       8  Pending   07-22 12:32:51
ct-sleb-dimc...       7  Pending   07-22 12:32:51
ct-sleb-dimc...       6  Pending   07-22 12:32:50
ct-sleb-dimc...       5  Pending   07-22 12:32:50
ct-sleb-dimc...       4  Pending   07-22 12:32:50
ct-sleb-dimc...       3  Pending   07-22 12:32:50
ct-sleb-dimc...       2  Pending   07-22 12:32:50
ct-sleb-dimc...       1  Pending   07-22 12:32:50



In [33]:
%%writefile PROSPER_LASSOSUM.sh
#!/bin/bash

path_plink='plink2'

set -o errexit
set -o nounset

tuning_bed_EUR=${tuning_bed_EUR%??}
tuning_bed_AFR=${tuning_bed_AFR%??}
tuning_bed_AMR=${tuning_bed_AMR%??}

valid_bed_EUR=${valid_bed_EUR%??}
valid_bed_AFR=${valid_bed_AFR%??}
valid_bed_AMR=${valid_bed_AMR%??}


Rscript ${PROSPER_Scripts}/scripts/lassosum2.R \
--PATH_package ${PROSPER_Scripts} \
--PATH_out ${OUTPUT_PATH} \
--PATH_plink ${path_plink} \
--FILE_sst ${EUR_Sumstats},${AFR_Sumstats},${AMR_Sumstats} \
--pop EUR,AFR,AMR \
--chrom 1-22 \
--bfile_tuning ${tuning_bed_EUR},${tuning_bed_AFR},${tuning_bed_AMR} \
--pheno_tuning ${tuning_y_EUR},${tuning_y_AFR},${tuning_y_AMR} \
--covar_tuning ${tuning_covar_EUR},${tuning_covar_AFR},${tuning_covar_AMR} \
--bfile_testing ${valid_bed_EUR},${valid_bed_AFR},${valid_bed_AMR} \
--pheno_testing ${valid_y_EUR},${valid_y_AFR},${valid_y_AMR} \
--covar_testing ${valid_covar_EUR},${valid_covar_AFR},${valid_covar_AMR} \
--testing TRUE \
--NCORES 1

Overwriting PROSPER_LASSOSUM.sh


In [34]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

trait <- "LDL"

for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
        '--input EUR_Sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/EUR_",trait,"_GWAS_SumStats_PROSPER.csv"),
        '--input AFR_Sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/AFR_",trait,"_GWAS_SumStats_PROSPER.csv"),
        '--input AMR_Sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/AMR_",trait,"_GWAS_SumStats_PROSPER.csv"),
        '--input-recursive PROSPER_Scripts'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/PROSPER",
        '--input tuning_bed_EUR'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_tune_JointPRS_MUSSEL_PROSPER_EUR.*",
        '--input tuning_bed_AMR'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_tune_JointPRS_MUSSEL_PROSPER_AMR.*",
        '--input tuning_bed_AFR'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_tune_JointPRS_MUSSEL_PROSPER_AFR.*",
        '--input valid_bed_EUR'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_Valid_JointPRS_MUSSEL_PROSPER_EUR.*",
        '--input valid_bed_AMR'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_Valid_JointPRS_MUSSEL_PROSPER_AMR.*",
        '--input valid_bed_AFR'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_Valid_JointPRS_MUSSEL_PROSPER_AFR.*",
        '--input tuning_y_EUR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/",trait,"_full_tune_y_EUR.txt"),
        '--input tuning_y_AMR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/",trait,"_full_tune_y_AMR.txt"),
        '--input tuning_y_AFR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/",trait,"_full_tune_y_AFR.txt"),
        '--input valid_y_EUR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/",trait,"_full_valid_y_EUR.txt"),
        '--input valid_y_AMR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/",trait,"_full_valid_y_AMR.txt"),
        '--input valid_y_AFR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/",trait,"_full_valid_y_AFR.txt"),
        '--input tuning_covar_EUR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/full_tune_covar_EUR.txt"),
        '--input tuning_covar_AMR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/full_tune_covar_AMR.txt"),
        '--input tuning_covar_AFR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/full_tune_covar_AFR.txt"),
        '--input valid_covar_EUR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/full_valid_covar_EUR.txt"),
        '--input valid_covar_AMR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/full_valid_covar_AMR.txt"),
        '--input valid_covar_AFR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/full_valid_covar_AFR.txt"),
        '--output-recursive OUTPUT_PATH'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/",trait),
    check.names = FALSE))   
}




write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [35]:
!Rscript score_task.R

In [36]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 60 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --min-cores 1 \
  --timeout "144h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script PROSPER_LASSOSUM.sh \
  --tasks score_task.txt

Job properties:
  job-id: prosper-la--williamsjacr--240722-180538-32
  job-name: prosper-lassosum
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/4638120276011749943
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/8663400788070535812
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/6956666000463151820
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/4609012853128535744
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10967767430252248676
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/5771711663660864596
Launched job-id: prosper-la--williamsjacr--240722-180538-32
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-la--williamsjacr--240722-1805

In [37]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-la--williamsjacr--240722-180538-32' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
prosper-lass...       6  Pending   07-22 18:05:38
prosper-lass...       5  Pending   07-22 18:05:38
prosper-lass...       4  Pending   07-22 18:05:38
prosper-lass...       3  Pending   07-22 18:05:38
prosper-lass...       2  Pending   07-22 18:05:38
prosper-lass...       1  Pending   07-22 18:05:38



In [14]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/prosper-la--williamsjacr--240722-180538-32.6-stdout.log


*************************
*** LASSOSUM2 FOR EUR ***
*************************

** Step 1. Preprocessing data **

** Step 2. Fitting models by chromosome **

** Step 3. Tuning **
Loaded 13 covariates
Calculating lassosum2 scores for tuning samples
lassosum2 optimal tuning parameters are saved in /mnt/data/output/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/Height/EUR/optimal_param.txt 
lassosum2 model is saved in /mnt/data/output/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/Height/EUR/lassosum2_prs_file.txt 

** Step 4. Testing **
Loaded 13 covariates
0.4559 variance in phenotype explained by covariates in testing samples 
** !COMPLETED! R2 is saved in /mnt/data/output/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/Height/EUR/R2.txt 

*************************
*** LASSOSUM2 FOR AFR ***
*************************

** Step 1. Preprocessing dat

In [38]:
%%writefile MUSSEL_LDPred2.sh
#!/bin/bash

set -o errexit
set -o nounset

tuning_bed_EUR=${tuning_bed_EUR%??}
tuning_bed_AFR=${tuning_bed_AFR%??}
tuning_bed_AMR=${tuning_bed_AMR%??}

Rscript ${MUSSEL_Scripts}/R/LDpred2.R \
--PATH_package ${MUSSEL_Scripts} \
--PATH_ref ${MUSSEL_LD} \
--PATH_out ${OUTPUT_PATH} \
--FILE_sst ${EUR_Sumstats},${AFR_Sumstats},${AMR_Sumstats} \
--pop EUR,AFR,AMR \
--bfile_tuning ${tuning_bed_EUR},${tuning_bed_AFR},${tuning_bed_AMR} \
--verbose 2 \
--NCORES 1

Overwriting MUSSEL_LDPred2.sh


In [39]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

trait <- "LDL"

for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
        '--input EUR_Sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/MUSSEL/EUR_",trait,"_GWAS_SumStats_MUSSEL.csv"),
        '--input AFR_Sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/MUSSEL/AFR_",trait,"_GWAS_SumStats_MUSSEL.csv"),
        '--input AMR_Sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/MUSSEL/AMR_",trait,"_GWAS_SumStats_MUSSEL.csv"),
        '--input-recursive MUSSEL_Scripts'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/MUSSEL/MUSSEL",
        '--input-recursive MUSSEL_LD'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/MUSSEL/MUSSEL_LD",
        '--input tuning_bed_EUR'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_tune_JointPRS_MUSSEL_PROSPER_EUR.*",
        '--input tuning_bed_AMR'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_tune_JointPRS_MUSSEL_PROSPER_AMR.*",
        '--input tuning_bed_AFR'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_tune_JointPRS_MUSSEL_PROSPER_AFR.*",
        '--output-recursive OUTPUT_PATH'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/MUSSEL/",trait),
    check.names = FALSE))   
}


write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [40]:
!Rscript score_task.R

In [41]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 200 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --min-cores 1 \
  --timeout "144h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script MUSSEL_LDPred2.sh \
  --tasks score_task.txt

Job properties:
  job-id: mussel-ldp--williamsjacr--240722-180603-76
  job-name: mussel-ldpred2
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/1876996133311691452
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/2094648451624967140
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/13690489926848506048
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/5508878527126587761
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/16970031399968614553
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/5105032732639826203
Launched job-id: mussel-ldp--williamsjacr--240722-180603-76
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'mussel-ldp--williamsjacr--240722-18060

In [30]:
!ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'mussel-ldp--williamsjacr--240722-180603-76' --users 'williamsjacr'

Delete running jobs:
  user:
    {'williamsjacr'}

  job-id:
    ['mussel-ldp--williamsjacr--240722-180603-76']

Found 6 tasks to delete.
1 job deleted (6 tasks)


In [29]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/mussel-ldp--williamsjacr--240722-180603-76.1-stdout.log


*************************
**** LDpred2 on EUR ****
*************************

** Step 1: data preparation **
[1] "Complete pre-processing GWAS summary data."
1.. [1] "Complete summarizing LD matrix info for CHR 1"
2.. [1] "Complete summarizing LD matrix info for CHR 2"
3.. [1] "Complete summarizing LD matrix info for CHR 3"
4.. [1] "Complete summarizing LD matrix info for CHR 4"
5.. [1] "Complete summarizing LD matrix info for CHR 5"
6.. [1] "Complete summarizing LD matrix info for CHR 6"
7.. [1] "Complete summarizing LD matrix info for CHR 7"
8.. [1] "Complete summarizing LD matrix info for CHR 8"
9.. [1] "Complete summarizing LD matrix info for CHR 9"
10.. [1] "Complete summarizing LD matrix info for CHR 10"
11.. [1] "Complete summarizing LD matrix info for CHR 11"
12.. [1] "Complete summarizing LD matrix info for CHR 12"
13.. [1] "Complete summarizing LD matrix info for CHR 13"
14.. [1] "Complete summarizing LD matrix info for CHR 14"
15.. [1] "Complete summarizing LD matrix info f

In [9]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/mussel-ldp--williamsjacr--240722-180603-76.1-stderr.log

Warning message:
NAs introduced by coercion 
Warning message:
NAs introduced by coercion 
1,545,398 variants to be matched.
63,059 ambiguous SNPs have been removed.
1,482,339 variants have been matched; 0 were flipped and 1,080,289 were reversed.
Loading required package: rngtools


In [16]:
%%writefile PROSPER_Prosper.sh

path_plink='plink2'

set -o errexit
set -o nounset

Rscript ${PROSPER_Scripts}/scripts/PROSPER.R \
--PATH_package ${PROSPER_Scripts} \
--PATH_out ${OUTPUT_PATH} \
--FILE_sst ${EUR_Sumstats},${AFR_Sumstats},${AMR_Sumstats} \
--pop EUR,AFR,AMR \
--lassosum_param ${EUR_Optim_Param},${AFR_Optim_Param},${AMR_Optim_Param} \
--chrom 1-22 \
--NCORES 1

Writing PROSPER_Prosper.sh


In [17]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

trait <- "LDL"

for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
        '--input EUR_Sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/EUR_",trait,"_GWAS_SumStats_PROSPER.csv"),
        '--input AFR_Sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/AFR_",trait,"_GWAS_SumStats_PROSPER.csv"),
        '--input AMR_Sumstats'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/AMR_",trait,"_GWAS_SumStats_PROSPER.csv"),
        '--input EUR_Optim_Param'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/",trait,"/EUR/optimal_param.txt"),
        '--input AFR_Optim_Param'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/",trait,"/AFR/optimal_param.txt"),
        '--input AMR_Optim_Param'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/",trait,"/AMR/optimal_param.txt"),
        '--input-recursive PROSPER_Scripts'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/PROSPER",
        '--output-recursive OUTPUT_PATH'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/",trait),
    check.names = FALSE))   
}

write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [18]:
!Rscript score_task.R

In [19]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 60 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --min-cores 1 \
  --timeout "144h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script PROSPER_Prosper.sh \
  --tasks score_task.txt

Job properties:
  job-id: prosper-pr--williamsjacr--240723-155606-10
  job-name: prosper-prosper
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/2200656647477152566
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10608213276173193313
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11366293823247759560
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3874828222003813483
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/18258188515994055652
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/5478996094867208468
Launched job-id: prosper-pr--williamsjacr--240723-155606-10
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-pr--williamsjacr--240723-155

In [22]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-pr--williamsjacr--240723-155606-10' --users 'williamsjacr' --status '*'

Job Name           Task  Status                          Last Update
---------------  ------  ------------------------------  --------------
prosper-prosper       6  Started running "user-command"  07-23 16:07:08
prosper-prosper       5  Started running "user-command"  07-23 16:07:16
prosper-prosper       4  Started running "user-command"  07-23 16:07:28
prosper-prosper       3  Started running "user-command"  07-23 16:08:49
prosper-prosper       2  Started running "user-command"  07-23 16:07:10
prosper-prosper       1  Started running "user-command"  07-23 16:07:17



In [31]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/prosper-pr--williamsjacr--240723-155606-10.1-stdout.log


** Step 1. Preprocessing data **

** Step 2. Fitting models by chromosome **
PRSs in all tuning parameter settings are saved in /mnt/data/output/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/BMI/before_ensemble/score_file.txt 
Their corresponding tuning parameter settings are saved in /mnt/data/output/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/BMI/before_ensemble/score_param.txt 


In [24]:
%%writefile PROSPER_SL.sh

path_plink='plink2'

set -o errexit
set -o nounset

tuning_bed=${tuning_bed%??}

valid_bed=${valid_bed%??}

Rscript ${PROSPER_Scripts}/scripts/tuning_testing.R \
--PATH_plink ${path_plink} \
--PATH_out ${OUTPUT_PATH} \
--PATH_input ${INPUT_PATH} \
--prefix 'full_results' \
--SL_library 'SL.glmnet,SL.ridge,SL.lm' \
--testing TRUE \
--linear_score TRUE \
--bfile_tuning ${tuning_bed} \
--pheno_tuning ${tuning_y} \
--covar_tuning ${tuning_covar} \
--bfile_testing ${valid_bed} \
--pheno_testing ${valid_y} \
--covar_testing ${valid_covar} \
--cleanup F \
--verbose 2 \
--NCORES 1

Overwriting PROSPER_SL.sh


In [25]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

trait <- "LDL"

for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
        '--input-recursive PROSPER_Scripts'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/PROSPER",
        '--input-recursive INPUT_PATH'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/",trait),
        '--input tuning_bed'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_tune_JointPRS_MUSSEL_PROSPER.*",
        '--input valid_bed'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_valid_JointPRS_MUSSEL_PROSPER.*",
        '--input tuning_y'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/",trait,"_full_tune_y.txt"),
        '--input valid_y'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/",trait,"_full_valid_y.txt"),
        '--input tuning_covar'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/full_tune_covar.txt"),
        '--input valid_covar'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/Tune_Validation_Phenotypes/full_valid_covar.txt"),
        '--output-recursive OUTPUT_PATH'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/",trait),
    check.names = FALSE))   
}

write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [26]:
!Rscript score_task.R

In [27]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 60 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --min-cores 1 \
  --timeout "144h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script PROSPER_SL.sh \
  --tasks score_task.txt

Job properties:
  job-id: prosper-sl--williamsjacr--240725-203220-36
  job-name: prosper-sl
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3198059155794176789
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/9150731821324572012
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/18392933430793886820
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/17610843321921514412
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3790050121890991472
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10535344993978084634
Launched job-id: prosper-sl--williamsjacr--240725-203220-36
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-sl--williamsjacr--240725-203220-3

In [28]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-sl--williamsjacr--240725-203220-36' --users 'williamsjacr' --status '*'

Job Name      Task  Status    Last Update
----------  ------  --------  --------------
prosper-sl       6  Pending   07-25 20:32:20
prosper-sl       5  Pending   07-25 20:32:20
prosper-sl       4  Pending   07-25 20:32:20
prosper-sl       3  Pending   07-25 20:32:20
prosper-sl       2  Pending   07-25 20:32:20
prosper-sl       1  Pending   07-25 20:32:20



In [15]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/prosper-sl--williamsjacr--240724-145841-65.6-stderr.log

codes.
(Add the 'list-variants' modifier to see which variants were actually used for
scoring.)
codes.
(Add the 'list-variants' modifier to see which variants were actually used for
scoring.)


In [9]:
%%writefile JointPRS_Score.R
rm(list = ls())

INPUT_PATH <- commandArgs(TRUE)[1]
print(INPUT_PATH)

BED_Full_File <- commandArgs(TRUE)[2]
BED_Full_File <- gsub(".bed","",BED_Full_File)
print(BED_Full_File)

OUTPUT_PATH <- commandArgs(TRUE)[3]
print(OUTPUT_PATH)


for(trait in c("BMI","LDL","HDL","logTG","TC","Height")){
    print(trait)
    a <- read.csv(paste0(INPUT_PATH,"/JointPRS_META_Score_",trait,".csv"))
    colnames(a) <- c("CHR", "rsID", "BP", "A1", "A2", "Post_EffectSize")
    a <- data.frame(SNP = a$rsID,A1 = a$A1,BETA = a$Post_EffectSize)
    write.table(a,file = "Final_PRS_Coefficients",col.names = T,row.names = F,quote=F)
    
    system(paste0("plink2 --threads 2 --score Final_PRS_Coefficients cols=+scoresums,-scoreavgs header no-mean-imputation  --bfile ",BED_Full_File," --out ",OUTPUT_PATH,"/PRS_META_",trait))
} 


Overwriting JointPRS_Score.R


In [10]:
%%writefile JointPRS_Score.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${INPUT_PATH} ${BED_Full_File} ${OUTPUT_PATH}

Overwriting JointPRS_Score.sh


In [11]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

tasks <- rbind(tasks, data.frame(
    '--input-recursive INPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/JointPRS/Scores",
    '--input BED_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_JointPRS_MUSSEL_PROSPER.bed",
    '--input BIM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_JointPRS_MUSSEL_PROSPER.bim",
    '--input FAM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_JointPRS_MUSSEL_PROSPER.fam",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/JointPRS_Score.R",
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/JointPRS/PRSs",
    check.names = FALSE
))   

write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Writing score_task.R


In [12]:
!Rscript score_task.R

In [13]:
!gsutil -m cp JointPRS_Score.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://JointPRS_Score.R [Content-Type=application/octet-stream]...
/ [1/1 files][  812.0 B/  812.0 B] 100% Done                                    
Operation completed over 1 objects/812.0 B.                                      


In [14]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 200 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "144h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script JointPRS_Score.sh \
  --tasks score_task.txt

Job properties:
  job-id: jointprs-s--williamsjacr--240731-121008-31
  job-name: jointprs-score
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/12804961307672585039
Launched job-id: jointprs-s--williamsjacr--240731-121008-31
1 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'jointprs-s--williamsjacr--240731-121008-31' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'jointprs-s--williamsjacr--240731-121008-31' --users 'williamsjacr'


In [15]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'jointprs-s--williamsjacr--240731-121008-31' --users 'williamsjacr' --status '*'

Job Name          Task  Status    Last Update
--------------  ------  --------  --------------
jointprs-score       1  Pending   07-31 12:10:08



In [48]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/jointprs-s--williamsjacr--240725-211311-32.1-stderr.log

Error: object 'anc' not found
Execution halted


In [59]:
%%writefile PROSPER_Score.R
rm(list = ls())

Score_BMI <- commandArgs(TRUE)[1]
print(Score_BMI)

Score_LDL <- commandArgs(TRUE)[2]
print(Score_LDL)

Score_HDL <- commandArgs(TRUE)[3]
print(Score_HDL)

Score_logTG <- commandArgs(TRUE)[4]
print(Score_logTG)

Score_TC <- commandArgs(TRUE)[5]
print(Score_TC)

Score_Height <- commandArgs(TRUE)[6]
print(Score_Height)

BED_Full_File <- commandArgs(TRUE)[7]
BED_Full_File <- gsub(".bed","",BED_Full_File)
print(BED_Full_File)

OUTPUT_PATH <- commandArgs(TRUE)[8]
print(OUTPUT_PATH)


for(trait in c("BMI","LDL","HDL","logTG","TC","Height")){
    
    if(trait == "BMI"){
        Score_File <- Score_BMI
    }else if(trait == "LDL"){
        Score_File <- Score_LDL
    }else if(trait == "HDL"){
        Score_File <- Score_HDL
    }else if(trait == "logTG"){
        Score_File <- Score_logTG
    }else if(trait == "TC"){
        Score_File <- Score_TC
    }else {
        Score_File <- Score_Height
    }
    
    b <- read.delim(Score_File)
    b <- data.frame(SNP = b$rsid,A1 = b$a1,BETA = b$weight)
    write.table(b,file = "Final_PRS_Coefficients",col.names = T,row.names = F,quote=F)
    
    system(paste0("plink2 --threads 2 --score Final_PRS_Coefficients cols=+scoresums,-scoreavgs header no-mean-imputation  --bfile ",BED_Full_File," --out ",OUTPUT_PATH,"/PRS_",trait))
} 

Overwriting PROSPER_Score.R


In [60]:
%%writefile PROSPER_Score.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${Score_BMI} ${Score_LDL} ${Score_HDL} ${Score_logTG} ${Score_TC} ${Score_Height} ${BED_Full_File} ${OUTPUT_PATH}

Overwriting PROSPER_Score.sh


In [61]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

tasks <- rbind(tasks, data.frame(
    '--input Score_BMI'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/BMI/after_ensemble_full_results/PROSPER_prs_file.txt",
    '--input Score_LDL'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/LDL/after_ensemble_full_results/PROSPER_prs_file.txt",
    '--input Score_HDL'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/HDL/after_ensemble_full_results/PROSPER_prs_file.txt",
    '--input Score_logTG'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/logTG/after_ensemble_full_results/PROSPER_prs_file.txt",
    '--input Score_TC'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/TC/after_ensemble_full_results/PROSPER_prs_file.txt",
    '--input Score_Height'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/Height/after_ensemble_full_results/PROSPER_prs_file.txt",
    '--input BED_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_JointPRS_MUSSEL_PROSPER.bed",
    '--input BIM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_JointPRS_MUSSEL_PROSPER.bim",
    '--input FAM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_JointPRS_MUSSEL_PROSPER.fam",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/PROSPER_Score.R",
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/PRSs",
    check.names = FALSE
))   

write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [62]:
!Rscript score_task.R

In [63]:
!gsutil -m cp PROSPER_Score.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://PROSPER_Score.R [Content-Type=application/octet-stream]...
\ [1/1 files][  1.3 KiB/  1.3 KiB] 100% Done                                    
Operation completed over 1 objects/1.3 KiB.                                      


In [64]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 200 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "144h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script PROSPER_Score.sh \
  --tasks score_task.txt

Job properties:
  job-id: prosper-sc--williamsjacr--240725-225501-88
  job-name: prosper-score
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/7038750270459992951
Launched job-id: prosper-sc--williamsjacr--240725-225501-88
1 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-sc--williamsjacr--240725-225501-88' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-sc--williamsjacr--240725-225501-88' --users 'williamsjacr'


In [65]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-sc--williamsjacr--240725-225501-88' --users 'williamsjacr' --status '*'

Job Name         Task  Status    Last Update
-------------  ------  --------  --------------
prosper-score       1  Pending   07-25 22:55:02



In [11]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/prosper-sc--williamsjacr--240725-121247-15.1-stderr.log

codes.
(Add the 'list-variants' modifier to see which variants were actually used for
scoring.)
codes.
(Add the 'list-variants' modifier to see which variants were actually used for
scoring.)
codes.
(Add the 'list-variants' modifier to see which variants were actually used for
scoring.)
codes.
(Add the 'list-variants' modifier to see which variants were actually used for
scoring.)
codes.
(Add the 'list-variants' modifier to see which variants were actually used for
scoring.)
Error in data.frame(SNP = b$rsid, A1 = b$a1, BETA = b$weight) : 
  arguments imply differing number of rows: 1456203, 0
Execution halted


In [59]:
%%writefile PROSPER_Score_All.R
rm(list = ls())

trait <- commandArgs(TRUE)[1]
print(trait)

Score_Trait <- commandArgs(TRUE)[2]
print(Score_Trait)

BED_Full_File <- commandArgs(TRUE)[3]
BED_Full_File <- gsub(".bed","",BED_Full_File)
print(BED_Full_File)

OUTPUT_PATH <- commandArgs(TRUE)[4]
print(OUTPUT_PATH)

Score_File <- Score_Trait
    
b <- read.delim(Score_File)

system(paste0("plink2 --threads 2 --score ",Score_Trait," cols=+scoresums,-scoreavgs header no-mean-imputation --score-col-nums 4-",ncol(b)," --bfile ",BED_Full_File," --out ",OUTPUT_PATH,"/PRS_All_",trait))

Overwriting PROSPER_Score_All.R


In [60]:
%%writefile PROSPER_Score_All.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${trait} ${Score_Trait} ${BED_Full_File} ${OUTPUT_PATH}

Overwriting PROSPER_Score_All.sh


In [61]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(trait in c("BMI","LDL","HDL","logTG","TC","Height")){
 tasks <- rbind(tasks, data.frame(
    '--env trait'=trait,
    '--input Score_Trait'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/",trait,"/before_ensemble/score_file.txt"),
    '--input BED_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_JointPRS_MUSSEL_PROSPER.bed",
    '--input BIM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_JointPRS_MUSSEL_PROSPER.bim",
    '--input FAM_Full_File'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED/all_JointPRS_MUSSEL_PROSPER.fam",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/PROSPER_Score_All.R",
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/PRSs",
    check.names = FALSE
))   
   
}
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [62]:
!Rscript score_task.R

In [63]:
!gsutil -m cp PROSPER_Score_All.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://PROSPER_Score_All.R [Content-Type=application/octet-stream]...
/ [1/1 files][  548.0 B/  548.0 B] 100% Done                                    
Operation completed over 1 objects/548.0 B.                                      


In [64]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 200 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "144h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script PROSPER_Score_All.sh \
  --tasks score_task.txt

Job properties:
  job-id: prosper-sc--williamsjacr--240726-131839-85
  job-name: prosper-score-all
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/16919446399436233190
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/16216689967082005827
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/519434646358723671
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/14020884406289596863
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/855727823133521907
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/16841389287832901535
Launched job-id: prosper-sc--williamsjacr--240726-131839-85
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-sc--williamsjacr--240726-13

In [65]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-sc--williamsjacr--240726-131839-85' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
prosper-scor...       6  Pending   07-26 13:18:40
prosper-scor...       5  Pending   07-26 13:18:40
prosper-scor...       4  Pending   07-26 13:18:40
prosper-scor...       3  Pending   07-26 13:18:40
prosper-scor...       2  Pending   07-26 13:18:40
prosper-scor...       1  Pending   07-26 13:18:40



In [58]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/prosper-sc--williamsjacr--240726-122424-49.6.log

2024-07-26 12:27:37 INFO: gsutil -h Content-Type:text/plain -u terra-vpc-sc-804f445b -mq cp /tmp/continuous_logging_action/output gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/prosper-sc--williamsjacr--240726-122424-49.6.log
2024-07-26 12:27:37 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED
2024-07-26 12:27:37 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED
2024-07-26 12:27:37 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Data/JointPRS_MUSSEL_PROSPER_BED
2024-07-26 12:27:37 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/Height/before_ensemble
2024-07-26 12:27:37 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d

In [72]:
%%writefile CT_SLEB_SL.R
rm(list = ls())

library(caret)
library(ranger)
library(SuperLearner)
library(dplyr)
library(boot)
library(stringr)

AFR_PRS <- commandArgs(TRUE)[1]
print(AFR_PRS)

AMR_PRS <- commandArgs(TRUE)[2]
print(AMR_PRS)

all_phenotypes_file <- commandArgs(TRUE)[3]
print(all_phenotypes_file)

all_train_file <- commandArgs(TRUE)[4]
print(all_train_file)

all_tune_file <- commandArgs(TRUE)[5]
print(all_tune_file)

all_validation_file <- commandArgs(TRUE)[6]
print(all_validation_file)

trait <- commandArgs(TRUE)[7]
print(trait)

OUTPUT_PATH <- commandArgs(TRUE)[8]
print(OUTPUT_PATH)



load(AFR_PRS)
colnames(prs_mat_eb)[3:ncol(prs_mat_eb)] <- paste0("AFR_",colnames(prs_mat_eb)[3:ncol(prs_mat_eb)])

PRS_Mat_AFR <- prs_mat_eb
rm(prs_mat_eb)

load(AMR_PRS)
colnames(prs_mat_eb)[3:ncol(prs_mat_eb)] <- paste0("AMR_",colnames(prs_mat_eb)[3:ncol(prs_mat_eb)])

PRS_Mat_AMR <- prs_mat_eb
rm(prs_mat_eb)

PRS_Mat_All <- inner_join(PRS_Mat_AFR,PRS_Mat_AMR)

rm(PRS_Mat_AFR);rm(PRS_Mat_AMR)



pheno_training <- read.delim(all_train_file)

PRS_Mat_Train <- PRS_Mat_All[PRS_Mat_All$IID %in% pheno_training$IID,]

pheno_training <- left_join(pheno_training,PRS_Mat_Train,by = "IID")

prs_train_all <- pheno_training[!is.na(pheno_training[,trait]),-c(1:22)]



pheno_tuning <- read.delim(all_tune_file)

PRS_Mat_Tune <- PRS_Mat_All[PRS_Mat_All$IID %in% pheno_tuning$IID,]

pheno_tuning <- left_join(pheno_tuning,PRS_Mat_Tune,by = "IID")

prs_tune_all <- pheno_tuning[!is.na(pheno_tuning[,trait]),-c(1:22)]



pheno_valid <- read.delim(all_validation_file)

PRS_Mat_Validation <- PRS_Mat_All[PRS_Mat_All$IID %in% pheno_valid$IID,]

pheno_valid <- left_join(pheno_valid,PRS_Mat_Validation,by = "IID")

prs_validation_all <- pheno_valid[,-c(1:22)]


model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_tuning)
y_tune <- model.null$residual

model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_valid)
y_validation <- model.null$residual


drop <- caret::findLinearCombos(prs_tune_all)$remove
drop <- names(prs_tune_all)[drop]

prs_train_all = prs_train_all %>% 
  select(-all_of(drop))
prs_tune_all = prs_tune_all %>% 
  select(-all_of(drop))
prs_validation_all = prs_validation_all %>% 
  select(-all_of(drop))
    

mtx <- cor(prs_tune_all)
drop <- findCorrelation(mtx,cutoff=0.98)
drop <- names(prs_tune_all)[drop]

prs_train_all = prs_train_all %>% 
  select(-all_of(drop))
prs_tune_all = prs_tune_all %>% 
  select(-all_of(drop))
prs_validation_all = prs_validation_all %>% 
  select(-all_of(drop))    


pheno_tuning$y_tune <- NA
pheno_tuning$y_tune[!is.na(pheno_tuning[,trait])] <- y_tune

pheno_valid$y_validation <- NA
pheno_valid$y_validation[!is.na(pheno_valid[,trait])] <- y_validation



SL.library <- c(
  "SL.glmnet",
  "SL.ridge"
)
sl <- SuperLearner(Y = y_tune, X = prs_tune_all, family = gaussian(),
                   # For a real analysis we would use V = 10.
                   # V = 3,
                   SL.library = SL.library,cvControl = list(V = 20))
cvsl <- CV.SuperLearner(Y = y_tune, X = prs_tune_all, family = gaussian(),
                        # For a real analysis we would use V = 10.
                        # V = 3,
                        SL.library = SL.library,cvControl = list(V = 20))

best_algorithm <- summary(cvsl)$Table$Algorithm[which.min(summary(cvsl)$Table$Ave)]

a <- predict(sl, prs_validation_all, onlySL = FALSE)

prs_best_validation_sl <- a$pred

prs_best_validation <- data.frame(IID = pheno_valid$IID,prs = prs_best_validation_sl)


a <- predict(sl, prs_train_all, onlySL = FALSE)

prs_best_train_sl <- a$pred

prs_best_train <- data.frame(IID = pheno_training$IID[!is.na(pheno_training[,trait])],prs = prs_best_train_sl)


a <- predict(sl, prs_tune_all, onlySL = FALSE)

prs_best_tune_sl <- a$pred

prs_best_tune <- data.frame(IID = pheno_tuning$IID[!is.na(pheno_tuning[,trait])],prs = prs_best_tune_sl)

pheno_training <- left_join(pheno_training,prs_best_train)
pheno_valid <- left_join(pheno_valid,prs_best_validation)
pheno_tuning <- left_join(pheno_tuning,prs_best_tune)


beta_lasso <- data.frame(Coef = row.names(coef(sl$fitLibrary$SL.glmnet_All$object)),Beta = as.numeric(coef(sl$fitLibrary$SL.glmnet_All$object)))
beta_ridge <- data.frame(Coef = row.names(sl$fitLibrary$SL.ridge_All$bestCoef),Beta = as.numeric(sl$fitLibrary$SL.ridge_All$bestCoef[,1]))
    
beta_sl <- data.frame(Coef = row.names(coef(sl$fitLibrary$SL.glmnet_All$object)), Beta = 0)
beta_sl$Beta <- as.numeric(sl$coef[names(sl$coef) == "SL.glmnet_All"]) * beta_lasso$Beta + 
      as.numeric(sl$coef[names(sl$coef) == "SL.ridge_All"]) * beta_ridge$Beta 
  
write.csv(beta_sl,file = paste0(OUTPUT_PATH,"/",trait,"_Final_Coefficients.csv"),row.names = FALSE)


prs_best_train <- data.frame(IID = pheno_training$IID,prs = pheno_training[,"prs"])

prs_best_tune <- data.frame(IID = pheno_tuning$IID,prs = pheno_tuning[,"prs"])

prs_best_validation <- data.frame(IID = pheno_valid$IID,prs = pheno_valid[,"prs"])

write.table(prs_best_train,file=paste0(OUTPUT_PATH,"/",trait,"_Best_Train_All.txt"),sep = "\t",row.names = FALSE)
write.table(prs_best_tune,file=paste0(OUTPUT_PATH,"/",trait,"_Best_Tune_All.txt"),sep = "\t",row.names = FALSE)
write.table(prs_best_validation,file=paste0(OUTPUT_PATH,"/",trait,"_Best_Validation_All.txt"),sep = "\t",row.names = FALSE)


all_phenotypes <- read.csv(all_phenotypes_file)

pheno_validation_raw <- pheno_valid
pheno_validation_adjusted <- pheno_valid


tmp <- data.frame(y = pheno_validation_adjusted[,"prs"],pheno_validation_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
mod <- lm(y~.,data = tmp)
R <- mod$residuals
tmp <- data.frame(y = R^2,pheno_validation_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
mod <- lm(y~.,data = tmp)
y_hat <- predict(mod,tmp)
if(sum(sqrt(y_hat)) == 0){
  pheno_validation_adjusted[,"prs"] <- 0
}else{
  pheno_validation_adjusted[,"prs"] <- R/sqrt(y_hat)
}


pheno_validation_raw_EUR <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
pheno_validation_raw_SAS <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
pheno_validation_raw_AMR <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
pheno_validation_raw_AFR <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
pheno_validation_raw_EAS <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
pheno_validation_raw_MID <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]


pheno_validation_adjusted_EUR <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
pheno_validation_adjusted_SAS <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
pheno_validation_adjusted_AMR <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
pheno_validation_adjusted_AFR <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
pheno_validation_adjusted_EAS <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
pheno_validation_adjusted_MID <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]


pheno_validation_raw_EUR$y_validation <- scale(pheno_validation_raw_EUR$y_validation)
pheno_validation_raw_SAS$y_validation <- scale(pheno_validation_raw_SAS$y_validation)
pheno_validation_raw_AMR$y_validation <- scale(pheno_validation_raw_AMR$y_validation)
pheno_validation_raw_AFR$y_validation <- scale(pheno_validation_raw_AFR$y_validation)
pheno_validation_raw_EAS$y_validation <- scale(pheno_validation_raw_EAS$y_validation)
pheno_validation_raw_MID$y_validation <- scale(pheno_validation_raw_MID$y_validation)

pheno_validation_adjusted_EUR$y_validation <- scale(pheno_validation_adjusted_EUR$y_validation)
pheno_validation_adjusted_SAS$y_validation <- scale(pheno_validation_adjusted_SAS$y_validation)
pheno_validation_adjusted_AMR$y_validation <- scale(pheno_validation_adjusted_AMR$y_validation)
pheno_validation_adjusted_AFR$y_validation <- scale(pheno_validation_adjusted_AFR$y_validation)
pheno_validation_adjusted_EAS$y_validation <- scale(pheno_validation_adjusted_EAS$y_validation)
pheno_validation_adjusted_MID$y_validation <- scale(pheno_validation_adjusted_MID$y_validation)


pheno_validation_raw_EUR[,"prs"] <- scale(pheno_validation_raw_EUR[,"prs"])
pheno_validation_raw_SAS[,"prs"] <- scale(pheno_validation_raw_SAS[,"prs"])
pheno_validation_raw_AMR[,"prs"] <- scale(pheno_validation_raw_AMR[,"prs"])
pheno_validation_raw_AFR[,"prs"] <- scale(pheno_validation_raw_AFR[,"prs"])
pheno_validation_raw_EAS[,"prs"] <- scale(pheno_validation_raw_EAS[,"prs"])
pheno_validation_raw_MID[,"prs"] <- scale(pheno_validation_raw_MID[,"prs"])


beta_validation_raw_EUR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EUR))[2]
se_validation_raw_EUR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EUR))$coefficients[2,2]
beta_validation_raw_SAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_SAS))[2]
se_validation_raw_SAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_SAS))$coefficients[2,2]
beta_validation_raw_AMR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AMR))[2]
se_validation_raw_AMR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AMR))$coefficients[2,2]
beta_validation_raw_AFR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AFR))[2]
se_validation_raw_AFR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AFR))$coefficients[2,2]
beta_validation_raw_EAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EAS))[2]
se_validation_raw_EAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EAS))$coefficients[2,2]
beta_validation_raw_MID <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_MID))[2]
se_validation_raw_MID <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_MID))$coefficients[2,2]

beta_validation_adjusted_EUR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EUR))[2]
se_validation_adjusted_EUR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EUR))$coefficients[2,2]
beta_validation_adjusted_SAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_SAS))[2]
se_validation_adjusted_SAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_SAS))$coefficients[2,2]
beta_validation_adjusted_AMR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AMR))[2]
se_validation_adjusted_AMR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AMR))$coefficients[2,2]
beta_validation_adjusted_AFR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AFR))[2]
se_validation_adjusted_AFR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AFR))$coefficients[2,2]
beta_validation_adjusted_EAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EAS))[2]
se_validation_adjusted_EAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EAS))$coefficients[2,2]
beta_validation_adjusted_MID <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_MID))[2]
se_validation_adjusted_MID <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_MID))$coefficients[2,2]

CV_Results <- data.frame(trait = trait,ancestry = c("EUR","SAS","AMR","AFR","EAS","MID"), 
                         beta_raw = c(beta_validation_raw_EUR,beta_validation_raw_SAS,beta_validation_raw_AMR,beta_validation_raw_AFR,beta_validation_raw_EAS,beta_validation_raw_MID), 
                         se_raw = c(se_validation_raw_EUR,se_validation_raw_SAS,se_validation_raw_AMR,se_validation_raw_AFR,se_validation_raw_EAS,se_validation_raw_MID), 
                         beta_adjusted = c(beta_validation_adjusted_EUR,beta_validation_adjusted_SAS,beta_validation_adjusted_AMR,beta_validation_adjusted_AFR,beta_validation_adjusted_EAS,beta_validation_adjusted_MID), 
                         se_adjusted = c(se_validation_adjusted_EUR,se_validation_adjusted_SAS,se_validation_adjusted_AMR,se_validation_adjusted_AFR,se_validation_adjusted_EAS,se_validation_adjusted_MID))

write.csv(CV_Results,file = paste0(OUTPUT_PATH,"/",trait,"Best_Betas.csv"),row.names = FALSE)

Overwriting CT_SLEB_SL.R


In [73]:
%%writefile CT_SLEB_SL.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${AFR_PRS} ${AMR_PRS} ${all_phenotypes_file} ${all_train_file} ${all_tune_file} ${all_validation_file} ${trait} ${OUTPUT_PATH}

Overwriting CT_SLEB_SL.sh


In [74]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)


for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
    '--input AFR_PRS'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/CTSLEB/AFR_",trait,"_prs_mat_eb.RData"),
    '--input AMR_PRS'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/CTSLEB/AMR_",trait,"_prs_mat_eb.RData"),
    '--input all_phenotypes_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
    '--input all_train_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Train.txt",
    '--input all_tune_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt",
    '--input all_validation_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/CT_SLEB_SL.R",
    '--env trait'=trait,
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/CTSLEB/Results/",
    check.names = FALSE
))   
   }

write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [75]:
!Rscript score_task.R

In [76]:
!gsutil -m cp CT_SLEB_SL.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://CT_SLEB_SL.R [Content-Type=application/octet-stream]...
/ [1/1 files][ 12.9 KiB/ 12.9 KiB] 100% Done                                    
Operation completed over 1 objects/12.9 KiB.                                     


In [77]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 64 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script CT_SLEB_SL.sh \
  --tasks score_task.txt

Job properties:
  job-id: ct-sleb-sl--williamsjacr--240726-135410-37
  job-name: ct-sleb-sl
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/12577038072026570578
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10747239263645617330
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/8395246878476471389
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/14450160083624827693
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/14140011373402086818
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/7320237539570158463
Launched job-id: ct-sleb-sl--williamsjacr--240726-135410-37
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'ct-sleb-sl--williamsjacr--240726-135410-

In [8]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'ct-sleb-sl--williamsjacr--240726-135410-37' --users 'williamsjacr' --status '*'

Job Name      Task  Status                          Last Update
----------  ------  ------------------------------  --------------
ct-sleb-sl       6  Started running "user-command"  07-26 13:57:47
ct-sleb-sl       5  Started running "user-command"  07-26 13:57:57
ct-sleb-sl       4  Started running "user-command"  07-26 13:57:32
ct-sleb-sl       3  Started running "user-command"  07-26 13:57:36
ct-sleb-sl       2  Started running "user-command"  07-26 13:58:03
ct-sleb-sl       1  Started running "user-command"  07-26 13:57:37



In [71]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/ct-sleb-sl--williamsjacr--240726-123906-81.1-stderr.log

Loading required package: ggplot2
Loading required package: lattice
Loading required package: nnls
Loading required package: gam
Loading required package: splines
Loading required package: foreach
Loaded gam 1.22-3

Super Learner
Version: 2.0-29
Package created on 2024-02-06


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘boot’

The following object is masked from ‘package:lattice’:

    melanoma

Joining with `by = join_by(`#FID`, IID)`
Loading required namespace: glmnet
/mnt/data/script/CT_SLEB_SL.sh: line 6:     7 Killed                  Rscript ${R_Script} ${AFR_PRS} ${AMR_PRS} ${all_phenotypes_file} ${all_train_file} ${all_tune_file} ${all_validation_file} ${trait} ${OUTPUT_PATH}


In [8]:
%%writefile Joint_PRS_Results.R
rm(list = ls())

library(caret)
library(ranger)
library(SuperLearner)
library(dplyr)
library(boot)
library(stringr)

PRS <- commandArgs(TRUE)[1]
print(PRS)

all_phenotypes_file <- commandArgs(TRUE)[2]
print(all_phenotypes_file)

all_train_file <- commandArgs(TRUE)[3]
print(all_train_file)

all_tune_file <- commandArgs(TRUE)[4]
print(all_tune_file)

all_validation_file <- commandArgs(TRUE)[5]
print(all_validation_file)

trait <- commandArgs(TRUE)[6]
print(trait)

OUTPUT_PATH <- commandArgs(TRUE)[7]
print(OUTPUT_PATH)


PRS_Mat_All <- read.delim(PRS)
colnames(PRS_Mat_All) <- c("FID","IID","ALLELE_CT","NAMED_ALLELE_DOSAGE_SUM","prs")
PRS_Mat_All <- PRS_Mat_All[,c("IID","prs")]


pheno_training <- read.delim(all_train_file)

PRS_Mat_Train <- PRS_Mat_All[PRS_Mat_All$IID %in% pheno_training$IID,]

pheno_training <- left_join(pheno_training,PRS_Mat_Train,by = "IID")



pheno_tuning <- read.delim(all_tune_file)

PRS_Mat_Tune <- PRS_Mat_All[PRS_Mat_All$IID %in% pheno_tuning$IID,]

pheno_tuning <- left_join(pheno_tuning,PRS_Mat_Tune,by = "IID")



pheno_valid <- read.delim(all_validation_file)

PRS_Mat_Validation <- PRS_Mat_All[PRS_Mat_All$IID %in% pheno_valid$IID,]

pheno_valid <- left_join(pheno_valid,PRS_Mat_Validation,by = "IID")


model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_tuning)
y_tune <- model.null$residual

model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_valid)
y_validation <- model.null$residual

pheno_tuning$y_tune <- NA
pheno_tuning$y_tune[!is.na(pheno_tuning[,trait])] <- y_tune

pheno_valid$y_validation <- NA
pheno_valid$y_validation[!is.na(pheno_valid[,trait])] <- y_validation


all_phenotypes <- read.csv(all_phenotypes_file)

pheno_validation_raw <- pheno_valid
pheno_validation_adjusted <- pheno_valid


tmp <- data.frame(y = pheno_validation_adjusted[,"prs"],pheno_validation_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
mod <- lm(y~.,data = tmp)
R <- mod$residuals
tmp <- data.frame(y = R^2,pheno_validation_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
mod <- lm(y~.,data = tmp)
y_hat <- predict(mod,tmp)
if(sum(sqrt(y_hat)) == 0){
  pheno_validation_adjusted[,"prs"] <- 0
}else{
  pheno_validation_adjusted[,"prs"] <- R/sqrt(y_hat)
}


pheno_validation_raw_EUR <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
pheno_validation_raw_SAS <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
pheno_validation_raw_AMR <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
pheno_validation_raw_AFR <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
pheno_validation_raw_EAS <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
pheno_validation_raw_MID <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]


pheno_validation_adjusted_EUR <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
pheno_validation_adjusted_SAS <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
pheno_validation_adjusted_AMR <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
pheno_validation_adjusted_AFR <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
pheno_validation_adjusted_EAS <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
pheno_validation_adjusted_MID <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]


pheno_validation_raw_EUR$y_validation <- scale(pheno_validation_raw_EUR$y_validation)
pheno_validation_raw_SAS$y_validation <- scale(pheno_validation_raw_SAS$y_validation)
pheno_validation_raw_AMR$y_validation <- scale(pheno_validation_raw_AMR$y_validation)
pheno_validation_raw_AFR$y_validation <- scale(pheno_validation_raw_AFR$y_validation)
pheno_validation_raw_EAS$y_validation <- scale(pheno_validation_raw_EAS$y_validation)
pheno_validation_raw_MID$y_validation <- scale(pheno_validation_raw_MID$y_validation)

pheno_validation_adjusted_EUR$y_validation <- scale(pheno_validation_adjusted_EUR$y_validation)
pheno_validation_adjusted_SAS$y_validation <- scale(pheno_validation_adjusted_SAS$y_validation)
pheno_validation_adjusted_AMR$y_validation <- scale(pheno_validation_adjusted_AMR$y_validation)
pheno_validation_adjusted_AFR$y_validation <- scale(pheno_validation_adjusted_AFR$y_validation)
pheno_validation_adjusted_EAS$y_validation <- scale(pheno_validation_adjusted_EAS$y_validation)
pheno_validation_adjusted_MID$y_validation <- scale(pheno_validation_adjusted_MID$y_validation)


pheno_validation_raw_EUR[,"prs"] <- scale(pheno_validation_raw_EUR[,"prs"])
pheno_validation_raw_SAS[,"prs"] <- scale(pheno_validation_raw_SAS[,"prs"])
pheno_validation_raw_AMR[,"prs"] <- scale(pheno_validation_raw_AMR[,"prs"])
pheno_validation_raw_AFR[,"prs"] <- scale(pheno_validation_raw_AFR[,"prs"])
pheno_validation_raw_EAS[,"prs"] <- scale(pheno_validation_raw_EAS[,"prs"])
pheno_validation_raw_MID[,"prs"] <- scale(pheno_validation_raw_MID[,"prs"])


beta_validation_raw_EUR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EUR))[2]
se_validation_raw_EUR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EUR))$coefficients[2,2]
beta_validation_raw_SAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_SAS))[2]
se_validation_raw_SAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_SAS))$coefficients[2,2]
beta_validation_raw_AMR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AMR))[2]
se_validation_raw_AMR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AMR))$coefficients[2,2]
beta_validation_raw_AFR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AFR))[2]
se_validation_raw_AFR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AFR))$coefficients[2,2]
beta_validation_raw_EAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EAS))[2]
se_validation_raw_EAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EAS))$coefficients[2,2]
beta_validation_raw_MID <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_MID))[2]
se_validation_raw_MID <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_MID))$coefficients[2,2]

beta_validation_adjusted_EUR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EUR))[2]
se_validation_adjusted_EUR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EUR))$coefficients[2,2]
beta_validation_adjusted_SAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_SAS))[2]
se_validation_adjusted_SAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_SAS))$coefficients[2,2]
beta_validation_adjusted_AMR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AMR))[2]
se_validation_adjusted_AMR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AMR))$coefficients[2,2]
beta_validation_adjusted_AFR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AFR))[2]
se_validation_adjusted_AFR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AFR))$coefficients[2,2]
beta_validation_adjusted_EAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EAS))[2]
se_validation_adjusted_EAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EAS))$coefficients[2,2]
beta_validation_adjusted_MID <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_MID))[2]
se_validation_adjusted_MID <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_MID))$coefficients[2,2]

CV_Results <- data.frame(trait = trait,ancestry = c("EUR","SAS","AMR","AFR","EAS","MID"), 
                         beta_raw = c(beta_validation_raw_EUR,beta_validation_raw_SAS,beta_validation_raw_AMR,beta_validation_raw_AFR,beta_validation_raw_EAS,beta_validation_raw_MID), 
                         se_raw = c(se_validation_raw_EUR,se_validation_raw_SAS,se_validation_raw_AMR,se_validation_raw_AFR,se_validation_raw_EAS,se_validation_raw_MID), 
                         beta_adjusted = c(beta_validation_adjusted_EUR,beta_validation_adjusted_SAS,beta_validation_adjusted_AMR,beta_validation_adjusted_AFR,beta_validation_adjusted_EAS,beta_validation_adjusted_MID), 
                         se_adjusted = c(se_validation_adjusted_EUR,se_validation_adjusted_SAS,se_validation_adjusted_AMR,se_validation_adjusted_AFR,se_validation_adjusted_EAS,se_validation_adjusted_MID))

write.csv(CV_Results,file = paste0(OUTPUT_PATH,"/",trait,"Best_Betas.csv"),row.names = FALSE)

Writing Joint_PRS_Results.R


In [9]:
%%writefile Joint_PRS_Results.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${PRS} ${all_phenotypes_file} ${all_train_file} ${all_tune_file} ${all_validation_file} ${trait} ${OUTPUT_PATH}

Writing Joint_PRS_Results.sh


In [10]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)


for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
    '--input PRS'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/JointPRS/PRSs/PRS_META_",trait,".sscore"),
    '--input all_phenotypes_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
    '--input all_train_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Train.txt",
    '--input all_tune_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt",
    '--input all_validation_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/Joint_PRS_Results.R",
    '--env trait'=trait,
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/JointPRS/Results/",
    check.names = FALSE
))   
   }

write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [11]:
!Rscript score_task.R

In [12]:
!gsutil -m cp Joint_PRS_Results.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://Joint_PRS_Results.R [Content-Type=application/octet-stream]...
/ [1/1 files][  9.4 KiB/  9.4 KiB] 100% Done                                    
Operation completed over 1 objects/9.4 KiB.                                      


In [13]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Joint_PRS_Results.sh \
  --tasks score_task.txt

Job properties:
  job-id: joint-prs---williamsjacr--240731-191801-94
  job-name: joint-prs-results
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3712880415493156591
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/13846036390793409358
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/7413637980004994758
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/2494075548292990445
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/4063203274783889299
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/17990280119822650816
Launched job-id: joint-prs---williamsjacr--240731-191801-94
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'joint-prs---williamsjacr--240731-19

In [14]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'joint-prs---williamsjacr--240731-191801-94' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
joint-prs-re...       6  Pending   07-31 19:18:02
joint-prs-re...       5  Pending   07-31 19:18:02
joint-prs-re...       4  Pending   07-31 19:18:02
joint-prs-re...       3  Pending   07-31 19:18:02
joint-prs-re...       2  Pending   07-31 19:18:02
joint-prs-re...       1  Pending   07-31 19:18:02



In [101]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/joint-prs---williamsjacr--240726-155335-82.1-stderr.log

Loading required package: ggplot2
Loading required package: lattice
Loading required package: nnls
Loading required package: gam
Loading required package: splines
Loading required package: foreach
Loaded gam 1.22-3

Super Learner
Version: 2.0-29
Package created on 2024-02-06


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘boot’

The following object is masked from ‘package:lattice’:

    melanoma

Error in array(x, c(length(x), 1L), if (!is.null(names(x))) list(names(x),  : 
  'data' must be of a vector type, was 'NULL'
Calls: scale ... scale.default -> as.matrix -> as.matrix.default -> array
Execution halted


In [9]:
%%writefile PROSPER_Results.R
rm(list = ls())

library(caret)
library(ranger)
library(SuperLearner)
library(dplyr)
library(boot)
library(stringr)

PRS <- commandArgs(TRUE)[1]
print(PRS)

all_phenotypes_file <- commandArgs(TRUE)[2]
print(all_phenotypes_file)

all_train_file <- commandArgs(TRUE)[3]
print(all_train_file)

all_tune_file <- commandArgs(TRUE)[4]
print(all_tune_file)

all_validation_file <- commandArgs(TRUE)[5]
print(all_validation_file)

trait <- commandArgs(TRUE)[6]
print(trait)

OUTPUT_PATH <- commandArgs(TRUE)[7]
print(OUTPUT_PATH)


PRS_Mat_All <- read.delim(PRS)
colnames(PRS_Mat_All) <- c("FID","IID","ALLELE_CT","NAMED_ALLELE_DOSAGE_SUM","prs")
PRS_Mat_All <- PRS_Mat_All[,c("IID","prs")]


pheno_training <- read.delim(all_train_file)

PRS_Mat_Train <- PRS_Mat_All[PRS_Mat_All$IID %in% pheno_training$IID,]

pheno_training <- left_join(pheno_training,PRS_Mat_Train,by = "IID")



pheno_tuning <- read.delim(all_tune_file)

PRS_Mat_Tune <- PRS_Mat_All[PRS_Mat_All$IID %in% pheno_tuning$IID,]

pheno_tuning <- left_join(pheno_tuning,PRS_Mat_Tune,by = "IID")



pheno_valid <- read.delim(all_validation_file)

PRS_Mat_Validation <- PRS_Mat_All[PRS_Mat_All$IID %in% pheno_valid$IID,]

pheno_valid <- left_join(pheno_valid,PRS_Mat_Validation,by = "IID")


model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_tuning)
y_tune <- model.null$residual

model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_valid)
y_validation <- model.null$residual

pheno_tuning$y_tune <- NA
pheno_tuning$y_tune[!is.na(pheno_tuning[,trait])] <- y_tune

pheno_valid$y_validation <- NA
pheno_valid$y_validation[!is.na(pheno_valid[,trait])] <- y_validation


all_phenotypes <- read.csv(all_phenotypes_file)

pheno_validation_raw <- pheno_valid
pheno_validation_adjusted <- pheno_valid


tmp <- data.frame(y = pheno_validation_adjusted[,"prs"],pheno_validation_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
mod <- lm(y~.,data = tmp)
R <- mod$residuals
tmp <- data.frame(y = R^2,pheno_validation_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
mod <- lm(y~.,data = tmp)
y_hat <- predict(mod,tmp)
if(sum(sqrt(y_hat)) == 0){
  pheno_validation_adjusted[,"prs"] <- 0
}else{
  pheno_validation_adjusted[,"prs"] <- R/sqrt(y_hat)
}


pheno_validation_raw_EUR <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
pheno_validation_raw_SAS <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
pheno_validation_raw_AMR <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
pheno_validation_raw_AFR <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
pheno_validation_raw_EAS <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
pheno_validation_raw_MID <- pheno_validation_raw[pheno_validation_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]


pheno_validation_adjusted_EUR <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
pheno_validation_adjusted_SAS <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
pheno_validation_adjusted_AMR <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
pheno_validation_adjusted_AFR <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
pheno_validation_adjusted_EAS <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
pheno_validation_adjusted_MID <- pheno_validation_adjusted[pheno_validation_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]


pheno_validation_raw_EUR$y_validation <- scale(pheno_validation_raw_EUR$y_validation)
pheno_validation_raw_SAS$y_validation <- scale(pheno_validation_raw_SAS$y_validation)
pheno_validation_raw_AMR$y_validation <- scale(pheno_validation_raw_AMR$y_validation)
pheno_validation_raw_AFR$y_validation <- scale(pheno_validation_raw_AFR$y_validation)
pheno_validation_raw_EAS$y_validation <- scale(pheno_validation_raw_EAS$y_validation)
pheno_validation_raw_MID$y_validation <- scale(pheno_validation_raw_MID$y_validation)

pheno_validation_adjusted_EUR$y_validation <- scale(pheno_validation_adjusted_EUR$y_validation)
pheno_validation_adjusted_SAS$y_validation <- scale(pheno_validation_adjusted_SAS$y_validation)
pheno_validation_adjusted_AMR$y_validation <- scale(pheno_validation_adjusted_AMR$y_validation)
pheno_validation_adjusted_AFR$y_validation <- scale(pheno_validation_adjusted_AFR$y_validation)
pheno_validation_adjusted_EAS$y_validation <- scale(pheno_validation_adjusted_EAS$y_validation)
pheno_validation_adjusted_MID$y_validation <- scale(pheno_validation_adjusted_MID$y_validation)


pheno_validation_raw_EUR[,"prs"] <- scale(pheno_validation_raw_EUR[,"prs"])
pheno_validation_raw_SAS[,"prs"] <- scale(pheno_validation_raw_SAS[,"prs"])
pheno_validation_raw_AMR[,"prs"] <- scale(pheno_validation_raw_AMR[,"prs"])
pheno_validation_raw_AFR[,"prs"] <- scale(pheno_validation_raw_AFR[,"prs"])
pheno_validation_raw_EAS[,"prs"] <- scale(pheno_validation_raw_EAS[,"prs"])
pheno_validation_raw_MID[,"prs"] <- scale(pheno_validation_raw_MID[,"prs"])


beta_validation_raw_EUR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EUR))[2]
se_validation_raw_EUR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EUR))$coefficients[2,2]
beta_validation_raw_SAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_SAS))[2]
se_validation_raw_SAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_SAS))$coefficients[2,2]
beta_validation_raw_AMR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AMR))[2]
se_validation_raw_AMR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AMR))$coefficients[2,2]
beta_validation_raw_AFR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AFR))[2]
se_validation_raw_AFR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_AFR))$coefficients[2,2]
beta_validation_raw_EAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EAS))[2]
se_validation_raw_EAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_EAS))$coefficients[2,2]
beta_validation_raw_MID <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_MID))[2]
se_validation_raw_MID <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_raw_MID))$coefficients[2,2]

beta_validation_adjusted_EUR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EUR))[2]
se_validation_adjusted_EUR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EUR))$coefficients[2,2]
beta_validation_adjusted_SAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_SAS))[2]
se_validation_adjusted_SAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_SAS))$coefficients[2,2]
beta_validation_adjusted_AMR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AMR))[2]
se_validation_adjusted_AMR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AMR))$coefficients[2,2]
beta_validation_adjusted_AFR <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AFR))[2]
se_validation_adjusted_AFR <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_AFR))$coefficients[2,2]
beta_validation_adjusted_EAS <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EAS))[2]
se_validation_adjusted_EAS <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_EAS))$coefficients[2,2]
beta_validation_adjusted_MID <- coef(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_MID))[2]
se_validation_adjusted_MID <- summary(lm(as.formula(paste0("y_validation~","prs")),data = pheno_validation_adjusted_MID))$coefficients[2,2]

CV_Results <- data.frame(trait = trait,ancestry = c("EUR","SAS","AMR","AFR","EAS","MID"), 
                         beta_raw = c(beta_validation_raw_EUR,beta_validation_raw_SAS,beta_validation_raw_AMR,beta_validation_raw_AFR,beta_validation_raw_EAS,beta_validation_raw_MID), 
                         se_raw = c(se_validation_raw_EUR,se_validation_raw_SAS,se_validation_raw_AMR,se_validation_raw_AFR,se_validation_raw_EAS,se_validation_raw_MID), 
                         beta_adjusted = c(beta_validation_adjusted_EUR,beta_validation_adjusted_SAS,beta_validation_adjusted_AMR,beta_validation_adjusted_AFR,beta_validation_adjusted_EAS,beta_validation_adjusted_MID), 
                         se_adjusted = c(se_validation_adjusted_EUR,se_validation_adjusted_SAS,se_validation_adjusted_AMR,se_validation_adjusted_AFR,se_validation_adjusted_EAS,se_validation_adjusted_MID))

write.csv(CV_Results,file = paste0(OUTPUT_PATH,"/",trait,"Best_Betas.csv"),row.names = FALSE)

Overwriting OneCommonPRS.R


In [10]:
%%writefile PROSPER_Results.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${PRS} ${all_phenotypes_file} ${all_train_file} ${all_tune_file} ${all_validation_file} ${trait} ${OUTPUT_PATH}

Writing PROSPER_Results.sh


In [11]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)


for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
    '--input PRS'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/PRSs/PRS_",trait,".sscore"),
    '--input all_phenotypes_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
    '--input all_train_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Train.txt",
    '--input all_tune_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt",
    '--input all_validation_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/PROSPER_Results.R",
    '--env trait'=trait,
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/Results/",
    check.names = FALSE
))   
   }

write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [12]:
!Rscript score_task.R

In [13]:
!gsutil -m cp PROSPER_Results.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

CommandException: No URLs matched: PROSPER_Results.R
CommandException: 1 file/object could not be transferred.


In [117]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script PROSPER_Results.sh \
  --tasks score_task.txt

Job properties:
  job-id: prosper-re--williamsjacr--240726-164526-12
  job-name: prosper-results
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/14514243183779343518
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11259223612414206690
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/12115918917554650631
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/6019796038400644284
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10214161872501419410
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/8329389636366739508
Launched job-id: prosper-re--williamsjacr--240726-164526-12
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-re--williamsjacr--240726-16

In [118]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'prosper-re--williamsjacr--240726-164526-12' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
prosper-results       6  Pending   07-26 16:45:26
prosper-results       5  Pending   07-26 16:45:26
prosper-results       4  Pending   07-26 16:45:26
prosper-results       3  Pending   07-26 16:45:26
prosper-results       2  Pending   07-26 16:45:26
prosper-results       1  Pending   07-26 16:45:26



In [15]:
%%writefile OneCommonPRS.R
rm(list = ls())
library(caret)
library(ranger)
library(SuperLearner)
library(dplyr)
library(boot)
library(stringr)

PROSPER_PRSs <- commandArgs(TRUE)[1]
print(PROSPER_PRSs)

JointPRS_PRS <- commandArgs(TRUE)[2]
print(JointPRS_PRS)

CTSLEB_AFR <- commandArgs(TRUE)[3]
print(CTSLEB_AFR)

CTSLEB_AMR <- commandArgs(TRUE)[4]
print(CTSLEB_AMR)

all_phenotypes_file <- commandArgs(TRUE)[5]
print(all_phenotypes_file)

all_train_file <- commandArgs(TRUE)[6]
print(all_train_file)

all_tune_file <- commandArgs(TRUE)[7]
print(all_tune_file)

all_validation_file <- commandArgs(TRUE)[8]
print(all_validation_file)

trait <- commandArgs(TRUE)[9]
print(trait)

OUTPUT_PATH <- commandArgs(TRUE)[10]
print(OUTPUT_PATH)

PROSPER_PRSs <- read.delim(PROSPER_PRSs)
colnames(PROSPER_PRSs) <- c("FID","IID","ALLELE_CT","NAMED_ALLELE_DOSAGE_SUM",paste0("PROSPER_PRS_",1:(ncol(PROSPER_PRSs) - 4)))
PROSPER_PRSs <- PROSPER_PRSs[,c("IID",paste0("PROSPER_PRS_",1:(ncol(PROSPER_PRSs) - 4)))]

JointPRS_PRS <- read.delim(JointPRS_PRS)
colnames(JointPRS_PRS) <- c("FID","IID","ALLELE_CT","NAMED_ALLELE_DOSAGE_SUM","JointPRS_PRS")
JointPRS_PRS <- JointPRS_PRS[,c("IID","JointPRS_PRS")]

load(CTSLEB_AFR)
CTSLEB_PRS_AFR <- prs_mat_eb;rm(prs_mat_eb)
colnames(CTSLEB_PRS_AFR) <- c("FID","IID",paste0("AFR_",colnames(CTSLEB_PRS_AFR)[3:ncol(CTSLEB_PRS_AFR)]))
colnames(CTSLEB_PRS_AFR) <- gsub("-","_",colnames(CTSLEB_PRS_AFR))
CTSLEB_PRS_AFR <- CTSLEB_PRS_AFR[,c("IID",colnames(CTSLEB_PRS_AFR)[3:ncol(CTSLEB_PRS_AFR)])]

load(CTSLEB_AMR)
CTSLEB_PRS_AMR <- prs_mat_eb;rm(prs_mat_eb)
colnames(CTSLEB_PRS_AMR) <- c("FID","IID",paste0("AMR_",colnames(CTSLEB_PRS_AMR)[3:ncol(CTSLEB_PRS_AMR)]))
colnames(CTSLEB_PRS_AMR) <- gsub("-","_",colnames(CTSLEB_PRS_AMR))
CTSLEB_PRS_AMR <- CTSLEB_PRS_AMR[,c("IID",colnames(CTSLEB_PRS_AMR)[3:ncol(CTSLEB_PRS_AMR)])]

PRS_All <- inner_join(PROSPER_PRSs,JointPRS_PRS)
PRS_All <- inner_join(PRS_All,CTSLEB_PRS_AFR)
PRS_All <- inner_join(PRS_All,CTSLEB_PRS_AMR)

pheno_training <- read.delim(all_train_file)

PRS_Mat_Train <- PRS_All[PRS_All$IID %in% pheno_training$IID,]

pheno_training <- left_join(pheno_training,PRS_Mat_Train,by = "IID")

prs_train_all <- pheno_training[!is.na(pheno_training[,trait]),-c(1:22)]



pheno_tuning <- read.delim(all_tune_file)

PRS_Mat_Tune <- PRS_All[PRS_All$IID %in% pheno_tuning$IID,]

pheno_tuning <- left_join(pheno_tuning,PRS_Mat_Tune,by = "IID")

prs_tune_all <- pheno_tuning[!is.na(pheno_tuning[,trait]),-c(1:22)]



pheno_valid <- read.delim(all_validation_file)

PRS_Mat_Validation <- PRS_All[PRS_All$IID %in% pheno_valid$IID,]

pheno_valid <- left_join(pheno_valid,PRS_Mat_Validation,by = "IID")

prs_validation_all <- pheno_valid[,-c(1:22)]


model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_tuning)
y_tune <- model.null$residual

model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_valid)
y_validation <- model.null$residual


drop <- caret::findLinearCombos(prs_tune_all)$remove
drop <- names(prs_tune_all)[drop]

prs_train_all = prs_train_all %>% 
  select(-all_of(drop))
prs_tune_all = prs_tune_all %>% 
  select(-all_of(drop))
prs_validation_all = prs_validation_all %>% 
  select(-all_of(drop))


mtx <- cor(prs_tune_all)
drop <- findCorrelation(mtx,cutoff=0.98)
drop <- names(prs_tune_all)[drop]

prs_train_all = prs_train_all %>% 
  select(-all_of(drop))
prs_tune_all = prs_tune_all %>% 
  select(-all_of(drop))
prs_validation_all = prs_validation_all %>% 
  select(-all_of(drop))    


pheno_tuning$y_tune <- NA
pheno_tuning$y_tune[!is.na(pheno_tuning[,trait])] <- y_tune

pheno_valid$y_validation <- NA
pheno_valid$y_validation[!is.na(pheno_valid[,trait])] <- y_validation

## SL

SL.library <- c(
  "SL.glmnet",
  "SL.ridge",
  "SL.glm",
  "SL.mean"
)
sl <- SuperLearner(Y = y_tune, X = prs_tune_all, family = gaussian(),
                   # For a real analysis we would use V = 10.
                   # V = 3,
                   SL.library = SL.library,cvControl = list(V = 20))
cvsl <- CV.SuperLearner(Y = y_tune, X = prs_tune_all, family = gaussian(),
                        # For a real analysis we would use V = 10.
                        # V = 3,
                        SL.library = SL.library,cvControl = list(V = 20))

best_algorithm <- summary(cvsl)$Table$Algorithm[which.min(summary(cvsl)$Table$Ave)]

a <- predict(sl, prs_validation_all, onlySL = FALSE)

prs_best_validation_sl <- a$pred
prs_best_validation_glmnet <- a$library.predict[,1]
prs_best_validation_ridge <- a$library.predict[,2]
prs_best_validation_glm <- a$library.predict[,3]

if(best_algorithm == "SL.glmnet_All"){
  #final
  prs_best_validation <- prs_best_validation_glmnet
}else if(best_algorithm == "SL.ridge_All"){
  #final
  prs_best_validation <- prs_best_validation_ridge
}else if(best_algorithm == "SL.glm_All"){
  #final
  prs_best_validation <- prs_best_validation_glm
}else{
  #final
  prs_best_validation <- prs_best_validation_sl
}

prs_best_validation <- data.frame(IID = pheno_valid$IID,prs = prs_best_validation)


a <- predict(sl, prs_train_all, onlySL = FALSE)

prs_best_train_sl <- a$pred
prs_best_train_glmnet <- a$library.predict[,1]
prs_best_train_ridge <- a$library.predict[,2]
prs_best_train_glm <- a$library.predict[,3]

if(best_algorithm == "SL.glmnet_All"){
  #final
  prs_best_train <- prs_best_train_glmnet
}else if(best_algorithm == "SL.ridge_All"){
  #final
  prs_best_train <- prs_best_train_ridge
}else if(best_algorithm == "SL.glm_All"){
  #final
  prs_best_train <- prs_best_train_glm
}else{
  #final
  prs_best_train <- prs_best_train_sl
}
prs_best_train <- data.frame(IID = pheno_training$IID[!is.na(pheno_training[,trait])],prs = prs_best_train)


a <- predict(sl, prs_tune_all, onlySL = FALSE)

prs_best_tune_sl <- a$pred
prs_best_tune_glmnet <- a$library.predict[,1]
prs_best_tune_ridge <- a$library.predict[,2]
prs_best_tune_glm <- a$library.predict[,3]

if(best_algorithm == "SL.glmnet_All"){
  #final
  prs_best_tune <- prs_best_tune_glmnet
}else if(best_algorithm == "SL.ridge_All"){
  #final
  prs_best_tune <- prs_best_tune_ridge
}else if(best_algorithm == "SL.glm_All"){
  #final
  prs_best_tune <- prs_best_tune_glm
}else{
  #final
  prs_best_tune <- prs_best_tune_sl
}
prs_best_tune <- data.frame(IID = pheno_tuning$IID[!is.na(pheno_tuning[,trait])],prs = prs_best_tune)

pheno_training <- left_join(pheno_training,prs_best_train)
pheno_valid <- left_join(pheno_valid,prs_best_validation)
pheno_tuning <- left_join(pheno_tuning,prs_best_tune)

prs_columns <- c(which(str_detect(colnames(pheno_tuning),"PROSPER_PRS_")),which(str_detect(colnames(pheno_tuning),"JointPRS_PRS")),which(str_detect(colnames(pheno_tuning),"AFR_")),which(str_detect(colnames(pheno_tuning),"AMR_")),which(str_detect(colnames(pheno_tuning),"prs")))

r2_tune <- vector()
for(i in 1:length(prs_columns)){
  r2_tune[i] <- summary(lm(as.formula(paste0("y_tune ~",colnames(pheno_tuning)[prs_columns[i]])),data = pheno_tuning))$r.squared
}

if(colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]] == "prs"){
  if(best_algorithm == "SL.glmnet_All"){
    beta_sl <- data.frame(Coef = row.names(coef(sl$fitLibrary$SL.glmnet_All$object)),Beta = as.numeric(coef(sl$fitLibrary$SL.glmnet_All$object)))
  }else if(best_algorithm == "SL.ridge_All"){
    beta_sl <- data.frame(Coef = row.names(sl$fitLibrary$SL.ridge_All$bestCoef),Beta = as.numeric(sl$fitLibrary$SL.ridge_All$bestCoef[,1]))
  }else if(best_algorithm == "SL.glm_All"){
    beta_sl <- data.frame(Coef = names(coef(sl$fitLibrary$SL.glm_All$object)),Beta = as.numeric(coef(sl$fitLibrary$SL.glm_All$object)))
  }else{
    beta_lasso <- data.frame(Coef = row.names(coef(sl$fitLibrary$SL.glmnet_All$object)),Beta = as.numeric(coef(sl$fitLibrary$SL.glmnet_All$object)))
    beta_ridge <- data.frame(Coef = row.names(sl$fitLibrary$SL.ridge_All$bestCoef),Beta = as.numeric(sl$fitLibrary$SL.ridge_All$bestCoef[,1]))
    beta_glm <- data.frame(Coef = names(coef(sl$fitLibrary$SL.glm_All$object)),Beta = as.numeric(coef(sl$fitLibrary$SL.glm_All$object)))
    beta_mean <- data.frame(Coef = names(coef(sl$fitLibrary$SL.glm_All$object)), Beta = 0)
    beta_mean$Beta[1] <- sl$fitLibrary$SL.mean_All$object
    
    beta_sl <- data.frame(Coef = names(coef(sl$fitLibrary$SL.glm_All$object)), Beta = 0)
    beta_sl$Beta <- as.numeric(sl$coef[names(sl$coef) == "SL.glmnet_All"]) * beta_lasso$Beta + 
      as.numeric(sl$coef[names(sl$coef) == "SL.ridge_All"]) * beta_ridge$Beta + 
      as.numeric(sl$coef[names(sl$coef) == "SL.glm_All"]) * beta_glm$Beta + 
      as.numeric(sl$coef[names(sl$coef) == "SL.mean_All"]) * beta_mean$Beta 
  }
  write.csv(beta_sl,file = paste0(OUTPUT_PATH,"/",trait,"_final_coef.csv"),row.names = FALSE)
  
}else{
  beta_coefs <- data.frame(Coef = colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]],Beta = 1)
  write.csv(beta_coefs,file = paste0(OUTPUT_PATH,"/",trait,"_final_coef.csv"),row.names = FALSE)
}



prs_best_train <- data.frame(IID = pheno_training$IID,prs = pheno_training[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]])

prs_best_tune <- data.frame(IID = pheno_tuning$IID,prs = pheno_tuning[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]])

prs_best_validation <- data.frame(IID = pheno_valid$IID,prs = pheno_valid[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]])

write.table(prs_best_train,file=paste0(OUTPUT_PATH,"/",trait,"_Best_Train_All.txt"),sep = "\t",row.names = FALSE)
write.table(prs_best_tune,file=paste0(OUTPUT_PATH,"/",trait,"_Best_Tune_All.txt"),sep = "\t",row.names = FALSE)
write.table(prs_best_validation,file=paste0(OUTPUT_PATH,"/",trait,"_Best_Validation_All.txt"),sep = "\t",row.names = FALSE)

all_phenotypes <- read.csv(all_phenotypes_file)


pheno_valid_raw <- pheno_valid
pheno_valid_adjusted <- pheno_valid


tmp <- data.frame(y = pheno_valid_adjusted[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]],pheno_valid_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
mod <- lm(y~.,data = tmp)
R <- mod$residuals
tmp <- data.frame(y = R^2,pheno_valid_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
mod <- lm(y~.,data = tmp)
y_hat <- predict(mod,tmp)
if(sum(sqrt(y_hat)) == 0){
  pheno_valid_adjusted[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]] <- 0
}else{
  pheno_valid_adjusted[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]] <- R/sqrt(y_hat)
}


pheno_valid_raw_EUR <- pheno_valid_raw[pheno_valid_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
pheno_valid_raw_SAS <- pheno_valid_raw[pheno_valid_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
pheno_valid_raw_AMR <- pheno_valid_raw[pheno_valid_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
pheno_valid_raw_AFR <- pheno_valid_raw[pheno_valid_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
pheno_valid_raw_EAS <- pheno_valid_raw[pheno_valid_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
pheno_valid_raw_MID <- pheno_valid_raw[pheno_valid_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]

pheno_valid_adjusted_EUR <- pheno_valid_adjusted[pheno_valid_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
pheno_valid_adjusted_SAS <- pheno_valid_adjusted[pheno_valid_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
pheno_valid_adjusted_AMR <- pheno_valid_adjusted[pheno_valid_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
pheno_valid_adjusted_AFR <- pheno_valid_adjusted[pheno_valid_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
pheno_valid_adjusted_EAS <- pheno_valid_adjusted[pheno_valid_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
pheno_valid_adjusted_MID <- pheno_valid_adjusted[pheno_valid_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]

pheno_valid_raw_EUR$y_validation <- scale(pheno_valid_raw_EUR$y_validation)
pheno_valid_raw_SAS$y_validation <- scale(pheno_valid_raw_SAS$y_validation)
pheno_valid_raw_AMR$y_validation <- scale(pheno_valid_raw_AMR$y_validation)
pheno_valid_raw_AFR$y_validation <- scale(pheno_valid_raw_AFR$y_validation)
pheno_valid_raw_EAS$y_validation <- scale(pheno_valid_raw_EAS$y_validation)
pheno_valid_raw_MID$y_validation <- scale(pheno_valid_raw_MID$y_validation)

pheno_valid_adjusted_EUR$y_validation <- scale(pheno_valid_adjusted_EUR$y_validation)
pheno_valid_adjusted_SAS$y_validation <- scale(pheno_valid_adjusted_SAS$y_validation)
pheno_valid_adjusted_AMR$y_validation <- scale(pheno_valid_adjusted_AMR$y_validation)
pheno_valid_adjusted_AFR$y_validation <- scale(pheno_valid_adjusted_AFR$y_validation)
pheno_valid_adjusted_EAS$y_validation <- scale(pheno_valid_adjusted_EAS$y_validation)
pheno_valid_adjusted_MID$y_validation <- scale(pheno_valid_adjusted_MID$y_validation)

pheno_valid_raw_EUR[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]] <- scale(pheno_valid_raw_EUR[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]])
pheno_valid_raw_SAS[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]] <- scale(pheno_valid_raw_SAS[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]])
pheno_valid_raw_AMR[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]] <- scale(pheno_valid_raw_AMR[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]])
pheno_valid_raw_AFR[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]] <- scale(pheno_valid_raw_AFR[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]])
pheno_valid_raw_EAS[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]] <- scale(pheno_valid_raw_EAS[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]])
pheno_valid_raw_MID[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]] <- scale(pheno_valid_raw_MID[,colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]]])

beta_validation_raw_EUR <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_EUR))[2]
se_validation_raw_EUR <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_EUR))$coefficients[2,2]
beta_validation_raw_SAS <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_SAS))[2]
se_validation_raw_SAS <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_SAS))$coefficients[2,2]
beta_validation_raw_AMR <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_AMR))[2]
se_validation_raw_AMR <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_AMR))$coefficients[2,2]
beta_validation_raw_AFR <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_AFR))[2]
se_validation_raw_AFR <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_AFR))$coefficients[2,2]
beta_validation_raw_EAS <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_EAS))[2]
se_validation_raw_EAS <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_EAS))$coefficients[2,2]
beta_validation_raw_MID <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_MID))[2]
se_validation_raw_MID <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_raw_MID))$coefficients[2,2]

beta_validation_adjusted_EUR <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_EUR))[2]
se_validation_adjusted_EUR <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_EUR))$coefficients[2,2]
beta_validation_adjusted_SAS <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_SAS))[2]
se_validation_adjusted_SAS <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_SAS))$coefficients[2,2]
beta_validation_adjusted_AMR <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_AMR))[2]
se_validation_adjusted_AMR <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_AMR))$coefficients[2,2]
beta_validation_adjusted_AFR <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_AFR))[2]
se_validation_adjusted_AFR <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_AFR))$coefficients[2,2]
beta_validation_adjusted_EAS <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_EAS))[2]
se_validation_adjusted_EAS <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_EAS))$coefficients[2,2]
beta_validation_adjusted_MID <- coef(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_MID))[2]
se_validation_adjusted_MID <- summary(lm(as.formula(paste0("y_validation~",colnames(pheno_tuning)[prs_columns[which.max(r2_tune)]])),data = pheno_valid_adjusted_MID))$coefficients[2,2]

CV_Results <- data.frame(trait = trait,ancestry = c("EUR","SAS","AMR","AFR","EAS","MID"), 
                                                    beta_raw = c(beta_validation_raw_EUR,beta_validation_raw_SAS,beta_validation_raw_AMR,beta_validation_raw_AFR,beta_validation_raw_EAS,beta_validation_raw_MID), 
                                                    se_raw = c(se_validation_raw_EUR,se_validation_raw_SAS,se_validation_raw_AMR,se_validation_raw_AFR,se_validation_raw_EAS,se_validation_raw_MID), 
                                                    beta_adjusted = c(beta_validation_adjusted_EUR,beta_validation_adjusted_SAS,beta_validation_adjusted_AMR,beta_validation_adjusted_AFR,beta_validation_adjusted_EAS,beta_validation_adjusted_MID), 
                                                    se_adjusted = c(se_validation_adjusted_EUR,se_validation_adjusted_SAS,se_validation_adjusted_AMR,se_validation_adjusted_AFR,se_validation_adjusted_EAS,se_validation_adjusted_MID))
                         
write.csv(CV_Results,file = paste0(OUTPUT_PATH,"/",trait,"Best_Betas.csv"),row.names = FALSE)

Writing OneCommonPRS.R


In [16]:
%%writefile OneCommonPRS.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${PROSPER_PRSs} ${JointPRS_PRS} ${CTSLEB_AFR} ${CTSLEB_AMR} ${all_phenotypes_file} ${all_train_file} ${all_tune_file} ${all_validation_file} ${trait} ${OUTPUT_PATH}

Writing OneCommonPRS.sh


In [17]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)


for(trait in c("BMI","LDL","HDL","TC","logTG","Height")){
    tasks <- rbind(tasks, data.frame(
    '--input PROSPER_PRSs'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/PROSPER/PRSs/PRS_All_",trait,".sscore"),
    '--input JointPRS_PRS'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/JointPRS/PRSs/PRS_META_",trait,".sscore"),
    '--input CTSLEB_AFR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/CTSLEB/AFR_",trait,"_prs_mat_eb.RData"),
    '--input CTSLEB_AMR'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/CTSLEB/AMR_",trait,"_prs_mat_eb.RData"),
    '--input all_phenotypes_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
    '--input all_train_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Train.txt",
    '--input all_tune_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt",
    '--input all_validation_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/OneCommonPRS.R",
    '--env trait'=trait,
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/OneCommonPRS/",
    check.names = FALSE
))   
   }

write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [18]:
!Rscript score_task.R

In [19]:
!gsutil -m cp OneCommonPRS.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://OneCommonPRS.R [Content-Type=application/octet-stream]...
/ [1/1 files][ 18.7 KiB/ 18.7 KiB] 100% Done                                    
Operation completed over 1 objects/18.7 KiB.                                     


In [20]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 64 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script OneCommonPRS.sh \
  --tasks score_task.txt

Job properties:
  job-id: onecommonp--williamsjacr--240731-191832-07
  job-name: onecommonprs
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/5269959543922159773
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/4104167026320805035
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/17363237223867866755
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/14958567778078369968
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/6334697042897950361
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/12828164815721406103
Launched job-id: onecommonp--williamsjacr--240731-191832-07
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'onecommonp--williamsjacr--240731-191832

In [21]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'onecommonp--williamsjacr--240731-191832-07' --users 'williamsjacr' --status '*'

Job Name        Task  Status    Last Update
------------  ------  --------  --------------
onecommonprs       6  Pending   07-31 19:18:32
onecommonprs       5  Pending   07-31 19:18:32
onecommonprs       4  Pending   07-31 19:18:32
onecommonprs       3  Pending   07-31 19:18:32
onecommonprs       2  Pending   07-31 19:18:32
onecommonprs       1  Pending   07-31 19:18:32



In [8]:
%%writefile NullModel.R
rm(list = ls())

all_train_file <- commandArgs(TRUE)[1]
print(all_train_file)

all_tune_file <- commandArgs(TRUE)[2]
print(all_tune_file)

all_validation_file <- commandArgs(TRUE)[3]
print(all_validation_file)

INPUT_PATH <- commandArgs(TRUE)[4]
print(INPUT_PATH)

OUTPUT_PATH <- commandArgs(TRUE)[5]
print(OUTPUT_PATH)

library(gdsfmt)
library(SeqArray)
library(SeqVarTools)
library(dplyr)
library(STAAR)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(Matrix)
library(SCANG)
library(STAARpipeline)

trait <- "BMI"

for(trait in c("BMI","TC","LDL","HDL","logTG","Height")){
  pheno_train <- read.delim(all_train_file)
  
  common_prs <- read.delim(paste0(INPUT_PATH,"/",trait,"_Best_Train_All.txt"))
  
  pheno_train <- inner_join(pheno_train,common_prs,by = "IID")
  
  obj.STAAR.UKB <- fit_nullmodel(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10 + prs")), data = pheno_train,id = "IID",kins = NULL,family = gaussian(link = "identity"))
  
  save(obj.STAAR.UKB,file = paste0(OUTPUT_PATH,"/",trait,"_Train_Null_Model.RData"))
  
  
  
  
  pheno_tune <- read.delim(all_tune_file)
  
  common_prs <- read.delim(paste0(INPUT_PATH,"/",trait,"_Best_Tune_All.txt"))
  
  pheno_tune <- inner_join(pheno_tune,common_prs,by = "IID")
  
  obj.STAAR.UKB <- fit_nullmodel(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10 + prs")), data = pheno_tune,id = "IID",kins = NULL,family = gaussian(link = "identity"))
  
  save(obj.STAAR.UKB,file = paste0(OUTPUT_PATH,"/",trait,"_Tune_Null_Model.RData"))
  
  
  
  
  pheno_validation <- read.delim(all_validation_file)
  
  common_prs <- read.delim(paste0(INPUT_PATH,"/",trait,"_Best_Validation_All.txt"))
  
  pheno_validation <- inner_join(pheno_validation,common_prs,by = "IID")
  
  obj.STAAR.UKB <- fit_nullmodel(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10 + prs")), data = pheno_validation,id = "IID",kins = NULL,family = gaussian(link = "identity"))
  
  save(obj.STAAR.UKB,file = paste0(OUTPUT_PATH,"/",trait,"_Validation_Null_Model.RData"))
  
  
}

Overwriting NullModel.R


In [9]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

tasks <- rbind(tasks, data.frame(
    '--input all_train_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Train.txt",
    '--input all_tune_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt",
    '--input all_validation_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt",
    '--input-recursive INPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/OneCommonPRS",
    '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/NullModel.R",
    '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels",
    check.names = FALSE
))   
   
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [10]:
%%writefile NullModel.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${all_train_file} ${all_tune_file} ${all_validation_file} ${INPUT_PATH} ${OUTPUT_PATH}

Overwriting NullModel.sh


In [11]:
!Rscript score_task.R

In [12]:
!gsutil -m cp NullModel.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://NullModel.R [Content-Type=application/octet-stream]...
/ [1/1 files][  2.0 KiB/  2.0 KiB] 100% Done                                    
Operation completed over 1 objects/2.0 KiB.                                      


In [13]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script NullModel.sh \
  --tasks score_task.txt

Job properties:
  job-id: nullmodel--williamsjacr--240801-214711-99
  job-name: nullmodel
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/4654998437295274832
Launched job-id: nullmodel--williamsjacr--240801-214711-99
1 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'nullmodel--williamsjacr--240801-214711-99' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'nullmodel--williamsjacr--240801-214711-99' --users 'williamsjacr'


In [15]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'nullmodel--williamsjacr--240801-214711-99' --users 'williamsjacr' --status '*'

Job Name      Task  Status    Last Update
----------  ------  --------  --------------
nullmodel        1  Success   08-01 21:52:06



In [29]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/nullmodel--williamsjacr--240801-131108-38.1-stderr.log

Error in library(SeqArray) : there is no package called ‘SeqArray’
Execution halted


In [33]:
%%writefile ~/aou_dsub.bash
#!/bin/bash
function aou_dsub () {

  # Get a shorter username to leave more characters for the job name.
  local DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

  # For AoU RWB projects network name is "network".
  local AOU_NETWORK=network
  local AOU_SUBNETWORK=subnetwork

  dsub \
      --provider google-cls-v2 \
      --user-project "${GOOGLE_PROJECT}"\
      --project "${GOOGLE_PROJECT}"\
      --image 'marketplace.gcr.io/google/ubuntu1804:latest' \
      --network "${AOU_NETWORK}" \
      --subnetwork "${AOU_SUBNETWORK}" \
      --service-account "$(gcloud config get-value account)" \
      --user "${DSUB_USER_NAME}" \
      --regions us-central1 \
      --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
      "$@"
}

Overwriting /home/jupyter/aou_dsub.bash


In [34]:
!echo source ~/aou_dsub.bash >> ~/.bashrc

In [91]:
%%writefile ncRNA_Analysis.R
rm(list=ls())
gc()

arrayid <- as.numeric(commandArgs(TRUE)[1])

trait <- commandArgs(TRUE)[2]
print(trait)

Train_Null_Model <- commandArgs(TRUE)[3]
print(Train_Null_Model)

Annotation_name_catalog <- commandArgs(TRUE)[4]
print(Annotation_name_catalog)

GDS_File <- commandArgs(TRUE)[5]
print(GDS_File)

OUTPUT_PATH <- commandArgs(TRUE)[6]
print(OUTPUT_PATH)

## load required packages
library(gdsfmt)
library(SeqArray)
library(SeqVarTools)
library(STAAR)
library(STAARpipeline)
library(STAARpipelineSummary)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(readr)
library(dplyr)
library(stringr)

obj_nullmodel <- get(load(Train_Null_Model))

## Parameter
QC_label <- "annotation/filter"
geno_missing_imputation <- "mean"
variant_type <- "SNV"

## Annotation_dir
Annotation_dir <- "annotation/info/FunctionalAnnotation"
## Annotation channel
Annotation_name_catalog <- read.csv(Annotation_name_catalog)
## Use_annotation_weights
Use_annotation_weights <- TRUE
## Annotation name
Annotation_name <- c("CADD","LINSIGHT","FATHMM.XF","aPC.EpigeneticActive","aPC.EpigeneticRepressed","aPC.EpigeneticTranscription",
                     "aPC.Conservation","aPC.LocalDiversity","aPC.Mappability","aPC.TF","aPC.Protein")

## output file name
output_file_name <- paste0(OUTPUT_PATH,"/",trait,"_AoU_WGS_ncRNA_Train")

## gene number in job
gene_num_in_array <- 100 
group.num.allchr <- ceiling(table(ncRNA_gene[,1])/gene_num_in_array)

chr <- which.max(arrayid <= cumsum(group.num.allchr))
group.num <- group.num.allchr[chr]

if (chr == 1){
  groupid <- arrayid
}else{
  groupid <- arrayid - cumsum(group.num.allchr)[chr-1]
}

ncRNA_gene_chr <- ncRNA_gene[ncRNA_gene[,1]==chr,]
ncRNA_longmasks <- c("KCNQ1OT1","AC006548.28","RP3-394A18.1","RP3-323A16.1","RP5-1039K5.19","SNHG14","LL22NC03-86G7.1")
ncRNA_gene_chr <- ncRNA_gene_chr[!ncRNA_gene_chr[,2] %in% ncRNA_longmasks,]
sub_seq_num <- dim(ncRNA_gene_chr)[1]

if(groupid < group.num){
  sub_seq_id <- ((groupid - 1)*gene_num_in_array + 1):(groupid*gene_num_in_array)
}else{
  sub_seq_id <- ((groupid - 1)*gene_num_in_array + 1):sub_seq_num
}

###########################################################
#           Main Function 
###########################################################

ncRNA_nolongmask <- function(chr,gene_name,genofile,obj_nullmodel,
                             rare_maf_cutoff=0.01,rv_num_cutoff=2,
                             QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                             Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                             Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                             SPA_p_filter=TRUE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA)){
    use_SPA <- obj_nullmodel$use_SPA
  }else{
    use_SPA <- FALSE
  }
  
  ## get SNV id
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant"){
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV"){
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel"){
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ## ncRNA SNVs
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  is.in <- ((GENCODE.Category=="ncRNA_exonic")|(GENCODE.Category=="ncRNA_exonic;splicing")|(GENCODE.Category=="ncRNA_splicing"))&(SNVlist)
  
  variant.id.ncRNA <- variant.id[is.in]
  
  rm(GENCODE.Category)
  gc()
  
  seqSetFilter(genofile,variant.id=variant.id.ncRNA,sample.id=phenotype.id)
  
  rm(variant.id.ncRNA)
  gc()
  
  GENCODE.Info <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Info")]))
  GENCODE.Info.split <- strsplit(GENCODE.Info, split = "[;]")
  Gene <- as.character(sapply(GENCODE.Info.split,function(z) gsub("\\(.*\\)","",z[1])))
  
  Gene_list_1 <- as.character(sapply(strsplit(Gene,','),'[',1))
  Gene_list_2 <- as.character(sapply(strsplit(Gene,','),'[',2))
  Gene_list_3 <- as.character(sapply(strsplit(Gene,','),'[',3))
  
  rm(GENCODE.Info)
  gc()
  
  rm(GENCODE.Info.split)
  gc()
  
  variant.id.ncRNA <- seqGetData(genofile, "variant.id")
  
  seqResetFilter(genofile)
  
  ### Gene
  is.in <- union(which(Gene_list_1==gene_name),which(Gene_list_2==gene_name))
  is.in <- union(is.in,which(Gene_list_3==gene_name))
  
  variant.is.in <- variant.id.ncRNA[is.in]
  
  if(length(variant.is.in) > 10000){
    return(NULL)
  }
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity"){
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list")){
    results_temp <- rep(NA,4)
    results_temp[3] <- "ncRNA"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results)){
    if(!use_SPA){
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

agds.path <- GDS_File

genofile <- seqOpen(agds.path)

gene_centric_ncRNA_dnanexus <- function(ncRNA_gene_chr,gene_name,chr,genofile,obj_nullmodel,rare_maf_cutoff,
                                        QC_label,variant_type,geno_missing_imputation,
                                        Annotation_dir,Annotation_name_catalog,
                                        Use_annotation_weights,Annotation_name,silent){
  results <- try(ncRNA_nolongmask(chr=chr,gene_name=gene_name,genofile=genofile,obj_nullmodel=obj_nullmodel,rare_maf_cutoff=rare_maf_cutoff,
                                  QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                  Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                  Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,silent=silent),silent=TRUE)
  return(results)
}

results_ncRNA <- c()
for(kk in sub_seq_id){
  print(kk)
  gene_name <- ncRNA_gene_chr[kk,2]
  results <- gene_centric_ncRNA_dnanexus(ncRNA_gene_chr=ncRNA_gene_chr,gene_name = gene_name,chr=chr,genofile=genofile,obj_nullmodel=obj_nullmodel,rare_maf_cutoff=max.maf,
                                         QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                         Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                         Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,silent=TRUE)
  
  results_ncRNA <- append(results_ncRNA,results)
}

seqClose(genofile)

rm(list=setdiff(ls(), c("results_ncRNA", "output_file_name","arrayid","agds.path","trait"))); gc()
save(results_ncRNA, file = paste0(output_file_name,"_",arrayid,".Rdata"))

Overwriting ncRNA_Analysis.R


In [92]:
%%writefile ncRNA_Analysis.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${arrayid} ${trait} ${Train_Null_Model} ${Annotation_name_catalog} ${GDS_File} ${OUTPUT_PATH}

Overwriting ncRNA_Analysis.sh


In [93]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(trait in c("BMI","LDL","HDL","logTG","Height","TC")){
    for(arrayid in 222){
        if(arrayid < 19){
            chr <- 1
        }else if(arrayid < 35){
            chr <- 2
        }else if(arrayid < 46){
            chr <- 3
        }else if(arrayid < 56){
            chr <- 4
        }else if(arrayid < 69){
            chr <- 5
        }else if(arrayid < 79){
            chr <- 6
        }else if(arrayid < 89){
            chr <- 7
        }else if(arrayid < 100){
            chr <- 8
        }else if(arrayid < 108){
            chr <- 9
        }else if(arrayid < 117){
            chr <- 10
        }else if(arrayid < 128){
            chr <- 11
        }else if(arrayid < 141){
            chr <- 12
        }else if(arrayid < 147){
            chr <- 13
        }else if(arrayid < 156){
            chr <- 14
        }else if(arrayid < 166){
            chr <- 15
        }else if(arrayid < 178){
            chr <- 16
        }else if(arrayid < 191){
            chr <- 17
        }else if(arrayid < 198){
            chr <- 18
        }else if(arrayid < 208){
            chr <- 19
        }else if(arrayid < 214){
            chr <- 20
        }else if(arrayid < 218){
            chr <- 21
        }else{
            chr <- 22
        }
        
        tasks <- rbind(tasks, data.frame(
            '--env arrayid'=arrayid,
            '--env trait'=trait,
            '--input Train_Null_Model'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels/",trait,"_Train_Null_Model.RData"),
            '--input Annotation_name_catalog'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataGDS/acaf_threshold_v7/Annotation_name_catalog.csv",
            '--input GDS_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataAGDS/acaf_threshold_v7/acaf_threshold.chr",chr,".gds"),
            '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/ncRNA_Analysis.R",
            '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GeneCentric_ncRNA",
            check.names = FALSE
        )) 
    }
}  
   
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [94]:
!Rscript score_task.R

In [95]:
!gsutil -m cp ncRNA_Analysis.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://ncRNA_Analysis.R [Content-Type=application/octet-stream]...
/ [1/1 files][ 10.5 KiB/ 10.5 KiB] 100% Done                                    
Operation completed over 1 objects/10.5 KiB.                                     


In [96]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 32 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script ncRNA_Analysis.sh \
  --tasks score_task.txt

Job properties:
  job-id: ncrna-anal--williamsjacr--240802-141249-95
  job-name: ncrna-analysis
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/15945330792918160714
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/12474725655731960867
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10526454129034710544
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/2471146011125752468
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/16532595984810075093
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/6231998838750485626
Launched job-id: ncrna-anal--williamsjacr--240802-141249-95
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'ncrna-anal--williamsjacr--240802-141

In [113]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'ncrna-anal--williamsjacr--240802-141249-95' --users 'williamsjacr' --status '*'

Job Name          Task  Status    Last Update
--------------  ------  --------  --------------
ncrna-analysis       6  Success   08-02 14:27:00
ncrna-analysis       5  Success   08-02 14:24:56
ncrna-analysis       4  Success   08-02 14:24:33
ncrna-analysis       3  Success   08-02 14:24:57
ncrna-analysis       2  Success   08-02 14:24:48
ncrna-analysis       1  Success   08-02 14:25:09



In [114]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/ncrna-anal--williamsjacr--240802-141249-95.5-stdout.log

         used (Mb) gc trigger (Mb) max used (Mb)
Ncells 294492 15.8     692571   37   480244 25.7
Vcells 510864  3.9    8388608   64  1926841 14.8
[1] "Height"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels/Height_Train_Null_Model.RData"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataGDS/acaf_threshold_v7/Annotation_name_catalog.csv"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataAGDS/acaf_threshold_v7/acaf_threshold.chr22.gds"
[1] "/mnt/data/output/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GeneCentric_ncRNA"
[1] 401
# of selected samples: 155,611
# of selected variants: 18,254
# of selected samples: 245,394
# of selected variants: 1,069,452
# of selected samples: 155,611
# of selected variants: 0
# of selected samples: 245,394
# of selected variants: 1,069,452
[1] 402
# of selected samples: 155,611
# of selected va

In [98]:
%%writefile Noncoding_Analysis.R
rm(list=ls())
gc()

arrayid <- as.numeric(commandArgs(TRUE)[1])

trait <- commandArgs(TRUE)[2]
print(trait)

Train_Null_Model <- commandArgs(TRUE)[3]
print(Train_Null_Model)

Annotation_name_catalog <- commandArgs(TRUE)[4]
print(Annotation_name_catalog)

GDS_File <- commandArgs(TRUE)[5]
print(GDS_File)

OUTPUT_PATH <- commandArgs(TRUE)[6]
print(OUTPUT_PATH)

## load required packages
library(gdsfmt)
library(SeqArray)
library(SeqVarTools)
library(STAAR)
library(STAARpipeline)
library(STAARpipelineSummary)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(readr)
library(dplyr)
library(stringr)

obj_nullmodel <- get(load(Train_Null_Model))

## Parameter
QC_label <- "annotation/filter"
geno_missing_imputation <- "mean"
variant_type <- "SNV"	

## Annotation_dir
Annotation_dir <- "annotation/info/FunctionalAnnotation"
## Annotation channel
Annotation_name_catalog <- read.csv(Annotation_name_catalog)
## Use_annotation_weights
Use_annotation_weights <- TRUE
## Annotation name
Annotation_name <- c("CADD","LINSIGHT","FATHMM.XF","aPC.EpigeneticActive","aPC.EpigeneticRepressed","aPC.EpigeneticTranscription",
                     "aPC.Conservation","aPC.LocalDiversity","aPC.Mappability","aPC.TF","aPC.Protein")

## output file name
output_file_name <- paste0(OUTPUT_PATH,"/",trait,"_AoU_WGS_Noncoding_Train")

###########################################################
#           Main Function 
###########################################################


## gene number in job
gene_num_in_array <- 50 
group.num.allchr <- ceiling(table(genes_info[,2])/gene_num_in_array)

chr <- which.max(arrayid <= cumsum(group.num.allchr))
group.num <- group.num.allchr[chr]

if(chr == 1){
  groupid <- arrayid
}else{
  groupid <- arrayid - cumsum(group.num.allchr)[chr-1]
}

genes_info_chr <- genes_info[genes_info[,2]==chr,]
noncoding_longmasks <- c("ARID4B","RERE","CRIM1","EML4","EML6","GLI2","ITGB6","FNDC3B","AFAP1","TRIO",
                         "NR3C1","SGK1","PRDM1","TNS3","GNA12","HOXA2","PLEC","ASAP1","CSGALNACT1","RAPGEF1",
                         "KLF6","DNAJB12","ABLIM1","ETV6","IGF1R","RMI2","CMIP","HOXB2","SEPT9","SPECC1",
                         "CIRBP","ADNP","CDC42EP3","EPAS1","MGAT5")
genes_info_chr <- genes_info_chr[!genes_info_chr[,1] %in% noncoding_longmasks,]
sub_seq_num <- dim(genes_info_chr)[1]

if(groupid < group.num){
  sub_seq_id <- ((groupid - 1)*gene_num_in_array + 1):(groupid*gene_num_in_array)
}else{
  sub_seq_id <- ((groupid - 1)*gene_num_in_array + 1):sub_seq_num
}

Gene_Centric_Noncoding_preload <- function(chr,gene_name,category=c("all_categories","downstream","upstream","UTR","promoter_CAGE","promoter_DHS","enhancer_CAGE","enhancer_DHS"),
                                           genofile,obj_nullmodel,
                                           dfPromCAGEVarGene.SNV,variant.id.SNV.PromCAGE,
                                           dfPromrOCRsVarGene.SNV,variant.id.SNV.PromrOCRs,
                                           dfHancerCAGEVarGene.SNV,variant.id.SNV.HancerCAGE,
                                           dfHancerrOCRsVarGene.SNV,variant.id.SNV.HancerrOCRs,
                                           rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                           QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                                           Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                                           Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                                           SPA_p_filter=TRUE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  category <- match.arg(category)
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  if(category=="all_categories"){
    results <- noncoding_preload(chr,gene_name,genofile,obj_nullmodel,
                                 dfPromCAGEVarGene.SNV=dfPromCAGEVarGene.SNV,variant.id.SNV.PromCAGE=variant.id.SNV.PromCAGE,
                                 dfPromrOCRsVarGene.SNV=dfPromrOCRsVarGene.SNV,variant.id.SNV.PromrOCRs=variant.id.SNV.PromrOCRs,
                                 dfHancerCAGEVarGene.SNV=dfHancerCAGEVarGene.SNV,variant.id.SNV.HancerCAGE=variant.id.SNV.HancerCAGE,
                                 dfHancerrOCRsVarGene.SNV=dfHancerrOCRsVarGene.SNV,variant.id.SNV.HancerrOCRs=variant.id.SNV.HancerrOCRs,
                                 rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                 Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                 Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                                 SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="downstream"){
    results <- downstream(chr,gene_name,genofile,obj_nullmodel,
                          rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                          QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                          Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                          Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                          SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="upstream"){
    results <- upstream(chr,gene_name,genofile,obj_nullmodel,
                        rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                        QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                        Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                        Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                        SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="UTR"){
    results <- UTR(chr,gene_name,genofile,obj_nullmodel,
                   rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                   QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                   Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                   Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                   SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="promoter_CAGE"){
    results <- promoter_CAGE_preload(chr,gene_name,genofile,obj_nullmodel,
                                     dfPromCAGEVarGene.SNV=dfPromCAGEVarGene.SNV,variant.id.SNV.PromCAGE=variant.id.SNV.PromCAGE,
                                     rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                                     QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                     Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                     Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                                     SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="promoter_DHS"){
    results <- promoter_DHS_preload(chr,gene_name,genofile,obj_nullmodel,
                                    dfPromrOCRsVarGene.SNV=dfPromrOCRsVarGene.SNV,variant.id.SNV.PromrOCRs=variant.id.SNV.PromrOCRs,
                                    rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                                    QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                    Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                    Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                                    SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="enhancer_CAGE"){
    results <- enhancer_CAGE_preload(chr,gene_name,genofile,obj_nullmodel,
                                     dfHancerCAGEVarGene.SNV=dfHancerCAGEVarGene.SNV,variant.id.SNV.HancerCAGE=variant.id.SNV.HancerCAGE,
                                     rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                                     QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                     Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                     Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                                     SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="enhancer_DHS"){
    results <- enhancer_DHS_preload(chr,gene_name,genofile,obj_nullmodel,
                                    dfHancerrOCRsVarGene.SNV=dfHancerrOCRsVarGene.SNV,variant.id.SNV.HancerrOCRs=variant.id.SNV.HancerrOCRs,
                                    rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                                    QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                    Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                    Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                                    SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  return(results)
}

noncoding_preload <- function(chr,gene_name,genofile,obj_nullmodel,
                              dfPromCAGEVarGene.SNV,variant.id.SNV.PromCAGE,
                              dfPromrOCRsVarGene.SNV,variant.id.SNV.PromrOCRs,
                              dfHancerCAGEVarGene.SNV,variant.id.SNV.HancerCAGE,
                              dfHancerrOCRsVarGene.SNV,variant.id.SNV.HancerrOCRs,
                              rare_maf_cutoff=0.01,rv_num_cutoff=2,
                              QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                              Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                              Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                              SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA)){
    use_SPA <- obj_nullmodel$use_SPA
  }else{
    use_SPA <- FALSE
  }
  
  #####################################
  #   Gene Info
  ## get SNV id
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant"){
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV"){
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel"){
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ########################################
  #   Downstream
  ## downstream SNVs
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  is.in <- (GENCODE.Category=="downstream")&(SNVlist)
  variant.id.downstream <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.downstream,sample.id=phenotype.id)
  
  rm(variant.id.downstream)
  gc()
  
  GENCODE.Info <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Info")]))
  GENCODE.Info.split <- strsplit(GENCODE.Info, split = "[,]")
  variant_gene_num <- sapply(GENCODE.Info.split,function(z) length(z))
  
  variant.id.SNV <- seqGetData(genofile, "variant.id")
  variant.id.SNV <- rep(variant.id.SNV,variant_gene_num)
  
  rm(GENCODE.Info)
  gc()
  
  rm(variant_gene_num)
  gc()
  
  Gene <- as.character(unlist(GENCODE.Info.split))
  
  rm(GENCODE.Info.split)
  gc()
  
  seqResetFilter(genofile)
  
  ### Gene
  is.in <- which(Gene==gene_name)
  variant.is.in <- variant.id.SNV[is.in]
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity"){
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_downstream <- c()
  if(inherits(pvalues, "list")){
    results_temp <- rep(NA,4)
    results_temp[3] <- "downstream"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_downstream <- rbind(results_downstream,results_temp)
  }
  
  if(!is.null(results_downstream)){
    if(!use_SPA){
      colnames(results_downstream) <- colnames(results_downstream, do.NULL = FALSE, prefix = "col")
      colnames(results_downstream)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_downstream)[(dim(results_downstream)[2]-1):dim(results_downstream)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results_downstream) <- colnames(results_downstream, do.NULL = FALSE, prefix = "col")
      colnames(results_downstream)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_downstream)[dim(results_downstream)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  ########################################
  #   Upstream
  
  is.in <- (GENCODE.Category=="upstream")&(SNVlist)
  variant.id.upstream <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.upstream,sample.id=phenotype.id)
  
  rm(variant.id.upstream)
  gc()
  
  GENCODE.Info <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Info")]))
  GENCODE.Info.split <- strsplit(GENCODE.Info, split = "[,]")
  variant_gene_num <- sapply(GENCODE.Info.split,function(z) length(z))
  
  variant.id.SNV <- seqGetData(genofile, "variant.id")
  variant.id.SNV <- rep(variant.id.SNV,variant_gene_num)
  
  rm(GENCODE.Info)
  gc()
  
  rm(variant_gene_num)
  gc()
  
  Gene <- as.character(unlist(GENCODE.Info.split))
  
  rm(GENCODE.Info.split)
  gc()
  
  seqResetFilter(genofile)
  
  ### Gene
  is.in <- which(Gene==gene_name)
  variant.is.in <- variant.id.SNV[is.in]
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity"){
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_upstream <- c()
  if(inherits(pvalues, "list")){
    results_temp <- rep(NA,4)
    results_temp[3] <- "upstream"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_upstream <- rbind(results_upstream,results_temp)
  }
  
  if(!is.null(results_upstream)){
    if(!use_SPA){
      colnames(results_upstream) <- colnames(results_upstream, do.NULL = FALSE, prefix = "col")
      colnames(results_upstream)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_upstream)[(dim(results_upstream)[2]-1):dim(results_upstream)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results_upstream) <- colnames(results_upstream, do.NULL = FALSE, prefix = "col")
      colnames(results_upstream)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_upstream)[dim(results_upstream)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  ########################################################
  #                UTR
  
  is.in <- ((GENCODE.Category=="UTR3")|(GENCODE.Category=="UTR5")|(GENCODE.Category=="UTR5;UTR3"))&(SNVlist)
  variant.id.UTR <- variant.id[is.in]
  
  rm(GENCODE.Category)
  gc()
  
  seqSetFilter(genofile,variant.id=variant.id.UTR,sample.id=phenotype.id)
  
  rm(variant.id.UTR)
  gc()
  
  GENCODE.Info <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Info")]))
  GENCODE.Info.split <- strsplit(GENCODE.Info, split = "[(]")
  
  rm(GENCODE.Info)
  gc()
  
  # Gene <- as.character(sapply(GENCODE.Info.split,function(z) z[seq(1,length(z),2)]))
  Gene <- as.character(sapply(GENCODE.Info.split,function(z) z[1]))
  
  rm(GENCODE.Info.split)
  gc()
  
  variant.id.SNV <- seqGetData(genofile, "variant.id")
  
  seqResetFilter(genofile)
  
  ### Gene
  is.in <- which(Gene==gene_name)
  variant.is.in <- variant.id.SNV[is.in]
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity"){
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_UTR <- c()
  if(inherits(pvalues, "list")){
    results_temp <- rep(NA,4)
    results_temp[3] <- "UTR"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_UTR <- rbind(results_UTR,results_temp)
  }
  
  if(!is.null(results_UTR)){
    if(!use_SPA){
      colnames(results_UTR) <- colnames(results_UTR, do.NULL = FALSE, prefix = "col")
      colnames(results_UTR)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_UTR)[(dim(results_UTR)[2]-1):dim(results_UTR)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results_UTR) <- colnames(results_UTR, do.NULL = FALSE, prefix = "col")
      colnames(results_UTR)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_UTR)[dim(results_UTR)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  #############################################
  #   Promoter-CAGE
  
  ### Gene
  is.in <- which(dfPromCAGEVarGene.SNV[,5]==gene_name)
  variant.is.in <- variant.id.SNV.PromCAGE[is.in]
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity"){
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_promoter_CAGE <- c()
  if(inherits(pvalues, "list")){
    results_temp <- dfPromCAGEVarGene.SNV[1,1:4]
    results_temp[3] <- "promoter_CAGE"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_promoter_CAGE <- rbind(results_promoter_CAGE,results_temp)
  }
  
  if(!is.null(results_promoter_CAGE)){
    if(!use_SPA){
      colnames(results_promoter_CAGE) <- colnames(results_promoter_CAGE, do.NULL = FALSE, prefix = "col")
      colnames(results_promoter_CAGE)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_promoter_CAGE)[(dim(results_promoter_CAGE)[2]-1):dim(results_promoter_CAGE)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results_promoter_CAGE) <- colnames(results_promoter_CAGE, do.NULL = FALSE, prefix = "col")
      colnames(results_promoter_CAGE)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_promoter_CAGE)[dim(results_promoter_CAGE)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  ##################################################
  #       Promoter-DHS
  
  ### Gene
  is.in <- which(dfPromrOCRsVarGene.SNV[,5]==gene_name)
  variant.is.in <- variant.id.SNV.PromrOCRs[is.in]
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity")
          {
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_promoter_DHS <- c()
  if(inherits(pvalues, "list")){
    results_temp <- dfPromrOCRsVarGene.SNV[1,1:4]
    results_temp[3] <- "promoter_DHS"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_promoter_DHS <- rbind(results_promoter_DHS ,results_temp)
  }
  
  if(!is.null(results_promoter_DHS)){
    if(!use_SPA){
      colnames(results_promoter_DHS) <- colnames(results_promoter_DHS, do.NULL = FALSE, prefix = "col")
      colnames(results_promoter_DHS)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_promoter_DHS)[(dim(results_promoter_DHS)[2]-1):dim(results_promoter_DHS)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results_promoter_DHS) <- colnames(results_promoter_DHS, do.NULL = FALSE, prefix = "col")
      colnames(results_promoter_DHS)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_promoter_DHS)[dim(results_promoter_DHS)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  ###########################################
  #        Enhancer-CAGE
  
  ### Gene
  is.in <- which(dfHancerCAGEVarGene.SNV[,5]==gene_name)
  variant.is.in <- variant.id.SNV.HancerCAGE[is.in]
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity"){
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_enhancer_CAGE <- c()
  if(inherits(pvalues, "list")){
    results_temp <- dfHancerCAGEVarGene.SNV[1,1:4]
    results_temp[3] <- "enhancer_CAGE"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_enhancer_CAGE <- rbind(results_enhancer_CAGE,results_temp)
  }
  
  if(!is.null(results_enhancer_CAGE)){
    if(!use_SPA){
      colnames(results_enhancer_CAGE) <- colnames(results_enhancer_CAGE, do.NULL = FALSE, prefix = "col")
      colnames(results_enhancer_CAGE)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_enhancer_CAGE)[(dim(results_enhancer_CAGE)[2]-1):dim(results_enhancer_CAGE)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results_enhancer_CAGE) <- colnames(results_enhancer_CAGE, do.NULL = FALSE, prefix = "col")
      colnames(results_enhancer_CAGE)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_enhancer_CAGE)[dim(results_enhancer_CAGE)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  ##################################################
  #       Enhancer-DHS
  
  ### Gene
  is.in <- which(dfHancerrOCRsVarGene.SNV[,5]==gene_name)
  variant.is.in <- variant.id.SNV.HancerrOCRs[is.in]
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity"){
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_enhancer_DHS <- c()
  if(inherits(pvalues, "list")){
    results_temp <- dfHancerrOCRsVarGene.SNV[1,1:4]
    results_temp[3] <- "enhancer_DHS"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_enhancer_DHS <- rbind(results_enhancer_DHS,results_temp)
  }
  
  if(!is.null(results_enhancer_DHS)){
    if(!use_SPA){
      colnames(results_enhancer_DHS) <- colnames(results_enhancer_DHS, do.NULL = FALSE, prefix = "col")
      colnames(results_enhancer_DHS)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_enhancer_DHS)[(dim(results_enhancer_DHS)[2]-1):dim(results_enhancer_DHS)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results_enhancer_DHS) <- colnames(results_enhancer_DHS, do.NULL = FALSE, prefix = "col")
      colnames(results_enhancer_DHS)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_enhancer_DHS)[dim(results_enhancer_DHS)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  ############################################
  #           results
  
  results_noncoding <- list(upstream=results_upstream,downstream=results_downstream,UTR=results_UTR,
                            promoter_CAGE=results_promoter_CAGE,promoter_DHS=results_promoter_DHS,
                            enhancer_CAGE=results_enhancer_CAGE,enhancer_DHS=results_enhancer_DHS)
  
  return(results_noncoding)
  
}

promoter_CAGE_preload <- function(chr,gene_name,genofile,obj_nullmodel,
                                  dfPromCAGEVarGene.SNV,variant.id.SNV.PromCAGE,
                                  rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                  QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                                  Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                                  Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                                  SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA)){
    use_SPA <- obj_nullmodel$use_SPA
  }else{
    use_SPA <- FALSE
  }
  
  ### Gene
  is.in <- which(dfPromCAGEVarGene.SNV[,5]==gene_name)
  variant.is.in <- variant.id.SNV.PromCAGE[is.in]
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity"){
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list")){
    results_temp <- dfPromCAGEVarGene.SNV[1,1:4]
    results_temp[3] <- "promoter_CAGE"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results)){
    if(!use_SPA){
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

promoter_DHS_preload <- function(chr,gene_name,genofile,obj_nullmodel,
                                 dfPromrOCRsVarGene.SNV,variant.id.SNV.PromrOCRs,
                                 rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                 QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                                 Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                                 Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                                 SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA)){
    use_SPA <- obj_nullmodel$use_SPA
  }else{
    use_SPA <- FALSE
  }
  
  ### Gene
  is.in <- which(dfPromrOCRsVarGene.SNV[,5]==gene_name)
  variant.is.in <- variant.id.SNV.PromrOCRs[is.in]
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity"){
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list")){
    results_temp <- dfPromrOCRsVarGene.SNV[1,1:4]
    results_temp[3] <- "promoter_DHS"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  
  if(!is.null(results)){
    if(!use_SPA){
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

enhancer_CAGE_preload <- function(chr,gene_name,genofile,obj_nullmodel,
                                  dfHancerCAGEVarGene.SNV,variant.id.SNV.HancerCAGE,
                                  rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                  QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                                  Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                                  Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                                  SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA)){
    use_SPA <- obj_nullmodel$use_SPA
  }else{
    use_SPA <- FALSE
  }
  
  ### Gene
  is.in <- which(dfHancerCAGEVarGene.SNV[,5]==gene_name)
  variant.is.in <- variant.id.SNV.HancerCAGE[is.in]
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity"){
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list")){
    results_temp <- dfHancerCAGEVarGene.SNV[1,1:4]
    results_temp[3] <- "enhancer_CAGE"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results)){
    if(!use_SPA){
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

enhancer_DHS_preload <- function(chr,gene_name,genofile,obj_nullmodel,
                                 dfHancerrOCRsVarGene.SNV,variant.id.SNV.HancerrOCRs,
                                 rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                 QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                                 Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                                 Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                                 SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA)){
    use_SPA <- obj_nullmodel$use_SPA
  }else{
    use_SPA <- FALSE
  }
  
  ### Gene
  is.in <- which(dfHancerrOCRsVarGene.SNV[,5]==gene_name)
  variant.is.in <- variant.id.SNV.HancerrOCRs[is.in]
  
  seqSetFilter(genofile,variant.id=variant.is.in,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno))){
    if(dim(Geno)[2]>0){
      if(geno_missing_imputation=="mean"){
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor"){
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV"){
    if(Use_annotation_weights){
      for(k in 1:length(Annotation_name)){
        if(Annotation_name[k]%in%Annotation_name_catalog$name){
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD"){
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity"){
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1){
    if(!use_SPA){
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else{
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list")){
    results_temp <- dfHancerrOCRsVarGene.SNV[1,1:4]
    results_temp[3] <- "enhancer_DHS"
    results_temp[2] <- chr
    results_temp[1] <- as.character(gene_name)
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA){
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else{
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results)){
    if(!use_SPA){
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else{
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

gene_centric_noncoding_dnanexus <- function(genes_info_chr,gene_name,chr,genofile,obj_nullmodel,
                                            dfPromCAGEVarGene.SNV,variant.id.SNV.PromCAGE,
                                            dfPromrOCRsVarGene.SNV,variant.id.SNV.PromrOCRs,
                                            dfHancerCAGEVarGene.SNV,variant.id.SNV.HancerCAGE,
                                            dfHancerrOCRsVarGene.SNV,variant.id.SNV.HancerrOCRs,
                                            rare_maf_cutoff,QC_label,variant_type,geno_missing_imputation,
                                            Annotation_dir,Annotation_name_catalog,
                                            Use_annotation_weights,Annotation_name,silent){
  results <- try(Gene_Centric_Noncoding_preload(chr=chr,gene_name=gene_name,genofile=genofile,obj_nullmodel=obj_nullmodel,
                                                dfPromCAGEVarGene.SNV=dfPromCAGEVarGene.SNV,variant.id.SNV.PromCAGE=variant.id.SNV.PromCAGE,
                                                dfPromrOCRsVarGene.SNV=dfPromrOCRsVarGene.SNV,variant.id.SNV.PromrOCRs=variant.id.SNV.PromrOCRs,
                                                dfHancerCAGEVarGene.SNV=dfHancerCAGEVarGene.SNV,variant.id.SNV.HancerCAGE=variant.id.SNV.HancerCAGE,
                                                dfHancerrOCRsVarGene.SNV=dfHancerrOCRsVarGene.SNV,variant.id.SNV.HancerrOCRs=variant.id.SNV.HancerrOCRs,
                                                rare_maf_cutoff=rare_maf_cutoff,QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                                Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                                Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,silent=silent),silent=TRUE)
  return(results)
}

#########################################################
#             Promoter_CAGE
#########################################################

agds.path <- GDS_File

genofile <- seqOpen(agds.path)

varid <- seqGetData(genofile, "variant.id")
txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene
promGobj <- promoters(genes(txdb), upstream = 3000, downstream = 3000)

#Subsetting Promoters that within +/-3kb of TSS and have CAGE signals
CAGEAnno <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="CAGE")]))
CAGEBvt <- CAGEAnno!=""
CAGEidx <- which(CAGEBvt,useNames=TRUE)
seqSetFilter(genofile,variant.id=varid[CAGEidx])
seqSetFilter(genofile,promGobj,intersect=TRUE)
CAGEpromgene <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Info")]))
CAGEGene <- unlist(lapply(strsplit(CAGEpromgene,"\\(|\\,|;|-"),`[[`,1))
##obtain variants info
CAGEvchr <- as.numeric(seqGetData(genofile,"chromosome"))
CAGEvpos <- as.numeric(seqGetData(genofile,"position"))
CAGEvref <- as.character(seqGetData(genofile,"$ref"))
CAGEvalt <- as.character(seqGetData(genofile,"$alt"))
dfPromCAGEVarGene <- data.frame(CAGEvchr,CAGEvpos,CAGEvref,CAGEvalt,CAGEGene)

rm(varid)
gc()

## get SNV id
filter <- seqGetData(genofile, QC_label)
if(variant_type=="variant"){
  SNVlist <- filter == "PASS"
}

if(variant_type=="SNV"){
  SNVlist <- (filter == "PASS") & isSNV(genofile)
}

if(variant_type=="Indel"){
  SNVlist <- (filter == "PASS") & (!isSNV(genofile))
}

variant.id <- seqGetData(genofile, "variant.id")
variant.id.SNV.PromCAGE <- variant.id[SNVlist]

dfPromCAGEVarGene.SNV <- dfPromCAGEVarGene[SNVlist,]
dfPromCAGEVarGene.SNV$CAGEvpos <- as.character(dfPromCAGEVarGene.SNV$CAGEvpos)
dfPromCAGEVarGene.SNV$CAGEvref <- as.character(dfPromCAGEVarGene.SNV$CAGEvref)
dfPromCAGEVarGene.SNV$CAGEvalt <- as.character(dfPromCAGEVarGene.SNV$CAGEvalt)

seqResetFilter(genofile)

rm(dfPromCAGEVarGene)
gc()

#########################################################
#             Promoter_DHS
#########################################################

varid <- seqGetData(genofile, "variant.id")
txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene
promGobj <- promoters(genes(txdb), upstream = 3000, downstream = 3000)

# Subsetting Promoters that within +/-3kb of TSS and have rOCRs signals
rOCRsAnno <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="DHS")]))
rOCRsBvt <- rOCRsAnno!=""
rOCRsidx <- which(rOCRsBvt,useNames=TRUE)
seqSetFilter(genofile,variant.id=varid[rOCRsidx])

seqSetFilter(genofile,promGobj,intersect=TRUE)
rOCRspromgene <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Info")]))
rOCRsGene <- unlist(lapply(strsplit(rOCRspromgene,"\\(|\\,|;|-"),`[[`,1))
## obtain variants info
rOCRsvchr <- as.numeric(seqGetData(genofile,"chromosome"))
rOCRsvpos <- as.numeric(seqGetData(genofile,"position"))
rOCRsvref <- as.character(seqGetData(genofile,"$ref"))
rOCRsvalt <- as.character(seqGetData(genofile,"$alt"))
dfPromrOCRsVarGene <- data.frame(rOCRsvchr,rOCRsvpos,rOCRsvref,rOCRsvalt,rOCRsGene)

rm(varid)
gc()

## get SNV id
filter <- seqGetData(genofile, QC_label)
if(variant_type=="variant"){
  SNVlist <- filter == "PASS"
}

if(variant_type=="SNV"){
  SNVlist <- (filter == "PASS") & isSNV(genofile)
}

if(variant_type=="Indel"){
  SNVlist <- (filter == "PASS") & (!isSNV(genofile))
}

variant.id <- seqGetData(genofile, "variant.id")
variant.id.SNV.PromrOCRs <- variant.id[SNVlist]

dfPromrOCRsVarGene.SNV <- dfPromrOCRsVarGene[SNVlist,]
dfPromrOCRsVarGene.SNV$rOCRsvpos <- as.character(dfPromrOCRsVarGene.SNV$rOCRsvpos)
dfPromrOCRsVarGene.SNV$rOCRsvref <- as.character(dfPromrOCRsVarGene.SNV$rOCRsvref)
dfPromrOCRsVarGene.SNV$rOCRsvalt <- as.character(dfPromrOCRsVarGene.SNV$rOCRsvalt)

seqResetFilter(genofile)

rm(dfPromrOCRsVarGene)
gc()

#########################################################
#             Enhancer_CAGE
#########################################################

varid <- seqGetData(genofile, "variant.id")

#Now extract the GeneHancer with CAGE Signal Overlay
genehancerAnno <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GeneHancer")]))
genehancer <- genehancerAnno!=""

CAGEAnno <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="CAGE")]))
CAGE <- CAGEAnno!=""
CAGEGeneHancervt <- CAGEAnno!=""&genehancerAnno!=""
CAGEGeneHanceridx <- which(CAGEGeneHancervt,useNames=TRUE)
seqSetFilter(genofile,variant.id=varid[CAGEGeneHanceridx])

# variants that covered by whole GeneHancer without CAGE overlap.
genehancerSet <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GeneHancer")]))
enhancerGene <- unlist(lapply(strsplit(genehancerSet,"="),`[[`,4))
enhancer2GENE <- unlist(lapply(strsplit(enhancerGene,";"),`[[`,1))
enhancervchr <- as.numeric(seqGetData(genofile,"chromosome"))
enhancervpos <- as.numeric(seqGetData(genofile,"position"))
enhancervref <- as.character(seqGetData(genofile,"$ref"))
enhancervalt <- as.character(seqGetData(genofile,"$alt"))
dfHancerCAGEVarGene <- data.frame(enhancervchr,enhancervpos,enhancervref,enhancervalt,enhancer2GENE)

rm(varid)
gc()

## get SNV id
filter <- seqGetData(genofile, QC_label)
if(variant_type=="variant"){
  SNVlist <- filter == "PASS"
}

if(variant_type=="SNV"){
  SNVlist <- (filter == "PASS") & isSNV(genofile)
}

if(variant_type=="Indel"){
  SNVlist <- (filter == "PASS") & (!isSNV(genofile))
}

variant.id <- seqGetData(genofile, "variant.id")
variant.id.SNV.HancerCAGE <- variant.id[SNVlist]

dfHancerCAGEVarGene.SNV <- dfHancerCAGEVarGene[SNVlist,]
dfHancerCAGEVarGene.SNV$enhancervpos <- as.character(dfHancerCAGEVarGene.SNV$enhancervpos)
dfHancerCAGEVarGene.SNV$enhancervref <- as.character(dfHancerCAGEVarGene.SNV$enhancervref)
dfHancerCAGEVarGene.SNV$enhancervalt <- as.character(dfHancerCAGEVarGene.SNV$enhancervalt)

seqResetFilter(genofile)

rm(dfHancerCAGEVarGene)
gc()

#########################################################
#             Enhancer_DHS
#########################################################

varid <- seqGetData(genofile, "variant.id")

#Now extract the GeneHancer with rOCRs Signal Overlay
genehancerAnno <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GeneHancer")]))
genehancer <- genehancerAnno!=""

rOCRsAnno <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="DHS")]))
rOCRs <- rOCRsAnno!=""
rOCRsGeneHancervt <- rOCRsAnno!=""&genehancerAnno!=""
rOCRsGeneHanceridx <- which(rOCRsGeneHancervt,useNames=TRUE)
seqSetFilter(genofile,variant.id=varid[rOCRsGeneHanceridx])
# variants that covered by whole GeneHancer without rOCRs overlap.

genehancerSet <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GeneHancer")]))
enhancerGene <- unlist(lapply(strsplit(genehancerSet,"="),`[[`,4))
enhancer2GENE <- unlist(lapply(strsplit(enhancerGene,";"),`[[`,1))
enhancervchr <- as.numeric(seqGetData(genofile,"chromosome"))
enhancervpos <- as.numeric(seqGetData(genofile,"position"))
enhancervref <- as.character(seqGetData(genofile,"$ref"))
enhancervalt <- as.character(seqGetData(genofile,"$alt"))
dfHancerrOCRsVarGene <- data.frame(enhancervchr,enhancervpos,enhancervref,enhancervalt,enhancer2GENE)

rm(varid)
gc()

## get SNV id
filter <- seqGetData(genofile, QC_label)
if(variant_type=="variant"){
  SNVlist <- filter == "PASS"
}

if(variant_type=="SNV"){
  SNVlist <- (filter == "PASS") & isSNV(genofile)
}

if(variant_type=="Indel"){
  SNVlist <- (filter == "PASS") & (!isSNV(genofile))
}

variant.id <- seqGetData(genofile, "variant.id")
variant.id.SNV.HancerrOCRs <- variant.id[SNVlist]

dfHancerrOCRsVarGene.SNV <- dfHancerrOCRsVarGene[SNVlist,]
dfHancerrOCRsVarGene.SNV$enhancervpos <- as.character(dfHancerrOCRsVarGene.SNV$enhancervpos)
dfHancerrOCRsVarGene.SNV$enhancervref <- as.character(dfHancerrOCRsVarGene.SNV$enhancervref)
dfHancerrOCRsVarGene.SNV$enhancervalt <- as.character(dfHancerrOCRsVarGene.SNV$enhancervalt)

seqResetFilter(genofile)

rm(dfHancerrOCRsVarGene)
gc()

results_noncoding <- list()
count <- 1
for(kk in sub_seq_id){
  print(kk)
  gene_name <- genes_info_chr[kk,1]
  results <- gene_centric_noncoding_dnanexus(genes_info_chr=genes_info_chr,gene_name = gene_name,chr=chr,genofile=genofile,obj_nullmodel=obj_nullmodel,
                                             dfPromCAGEVarGene.SNV=dfPromCAGEVarGene.SNV,variant.id.SNV.PromCAGE=variant.id.SNV.PromCAGE,
                                             dfPromrOCRsVarGene.SNV=dfPromrOCRsVarGene.SNV,variant.id.SNV.PromrOCRs=variant.id.SNV.PromrOCRs,
                                             dfHancerCAGEVarGene.SNV=dfHancerCAGEVarGene.SNV,variant.id.SNV.HancerCAGE=variant.id.SNV.HancerCAGE,
                                             dfHancerrOCRsVarGene.SNV=dfHancerrOCRsVarGene.SNV,variant.id.SNV.HancerrOCRs=variant.id.SNV.HancerrOCRs,
                                             rare_maf_cutoff=0.01,QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                             Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                             Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,silent=TRUE)
  
  results_noncoding[[count]] <-results
  count <- count + 1
}

seqClose(genofile)

rm(list=setdiff(ls(), c("results_noncoding", "output_file_name","arrayid","agds.path","trait"))); gc()
save(results_noncoding, file = paste0(output_file_name,"_",arrayid,".Rdata"))

Overwriting Noncoding_Analysis.R


In [99]:
%%writefile Noncoding_Analysis.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${arrayid} ${trait} ${Train_Null_Model} ${Annotation_name_catalog} ${GDS_File} ${OUTPUT_PATH}

Overwriting Noncoding_Analysis.sh


In [100]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(trait in c("BMI","LDL","HDL","logTG","Height","TC")){
    for(arrayid in 379){
        if(arrayid < 41){
            chr <- 1
        }else if(arrayid < 66){
            chr <- 2
        }else if(arrayid < 87){
            chr <- 3
        }else if(arrayid < 102){
            chr <- 4
        }else if(arrayid < 120){
            chr <- 5
        }else if(arrayid < 141){
            chr <- 6
        }else if(arrayid < 159){
            chr <- 7
        }else if(arrayid < 173){
            chr <- 8
        }else if(arrayid < 189){
            chr <- 9
        }else if(arrayid < 204){
            chr <- 10
        }else if(arrayid < 230){
            chr <- 11
        }else if(arrayid < 250){
            chr <- 12
        }else if(arrayid < 257){
            chr <- 13
        }else if(arrayid < 269){
            chr <- 14
        }else if(arrayid < 281){
            chr <- 15
        }else if(arrayid < 298){
            chr <- 16
        }else if(arrayid < 321){
            chr <- 17
        }else if(arrayid < 327){
            chr <- 18
        }else if(arrayid < 355){
            chr <- 19
        }else if(arrayid < 366){
            chr <- 20
        }else if(arrayid < 371){
            chr <- 21
        }else{
            chr <- 22
        }
        
        tasks <- rbind(tasks, data.frame(
            '--env arrayid'=arrayid,
            '--env trait'=trait,
            '--input Train_Null_Model'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels/",trait,"_Train_Null_Model.RData"),
            '--input Annotation_name_catalog'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataGDS/acaf_threshold_v7/Annotation_name_catalog.csv",
            '--input GDS_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataAGDS/acaf_threshold_v7/acaf_threshold.chr",chr,".gds"),
            '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/Noncoding_Analysis.R",
            '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GeneCentricNoncoding",
            check.names = FALSE
        )) 
    }
}  
   
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [101]:
!Rscript score_task.R

In [102]:
!gsutil -m cp Noncoding_Analysis.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://Noncoding_Analysis.R [Content-Type=application/octet-stream]...
/ [1/1 files][ 75.8 KiB/ 75.8 KiB] 100% Done                                    
Operation completed over 1 objects/75.8 KiB.                                     


In [110]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 32 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Noncoding_Analysis.sh \
  --tasks score_task.txt

Traceback (most recent call last):
  File "/opt/conda/bin/dsub", line 8, in <module>
    sys.exit(main())
  File "/home/jupyter/.local/lib/python3.10/site-packages/dsub/commands/dsub.py", line 1150, in main
    dsub_main(prog, argv)
  File "/home/jupyter/.local/lib/python3.10/site-packages/dsub/commands/dsub.py", line 1135, in dsub_main
    launched_job = run_main(args)
  File "/home/jupyter/.local/lib/python3.10/site-packages/dsub/commands/dsub.py", line 1212, in run_main
    return run(
  File "/home/jupyter/.local/lib/python3.10/site-packages/dsub/commands/dsub.py", line 1284, in run
    raise ValueError('Requesting retries requires requesting wait')
ValueError: Requesting retries requires requesting wait


CalledProcessError: Command 'b'\nsource ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.\n\naou_dsub \\\n  --image willja16/r_with_plink \\\n  --disk-size 100 \\\n  --boot-disk-size 25 \\\n  --min-ram 32 \\\n  --preemptible \\\n  --retries 1 \\\n  --timeout "96h" \\\n  --logging "${WORKSPACE_BUCKET}/data/logging" \\\n  --script Noncoding_Analysis.sh \\\n  --tasks score_task.txt\n'' returned non-zero exit status 1.

In [111]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'noncoding---williamsjacr--240802-141312-91' --users 'williamsjacr' --status '*'

Job Name           Task  Status                          Last Update
---------------  ------  ------------------------------  --------------
noncoding-an...       6  Success                         08-02 14:28:26
noncoding-an...       5  Success                         08-02 14:35:16
noncoding-an...       4  Success                         08-02 14:28:42
noncoding-an...       3  Success                         08-02 14:29:28
noncoding-an...       2  Success                         08-02 14:28:26
noncoding-an...       1  Started running "user-command"  08-02 14:18:37



In [112]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/noncoding---williamsjacr--240802-141312-91.6-stdout.log

         used (Mb) gc trigger (Mb) max used (Mb)
Ncells 294492 15.8     692571   37   480244 25.7
Vcells 510864  3.9    8388608   64  1926841 14.8
[1] "TC"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels/TC_Train_Null_Model.RData"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataGDS/acaf_threshold_v7/Annotation_name_catalog.csv"
[1] "/mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataAGDS/acaf_threshold_v7/acaf_threshold.chr22.gds"
[1] "/mnt/data/output/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GeneCentricNoncoding"
# of selected variants: 19,697
# of selected variants: 6,975
           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells 10008001 534.5   16402044 876.0 12330846 658.6
Vcells 24815785 189.4   42645776 325.4 35469499 270.7
# of selected samples: 245,394
# of selected variants: 1,069,452
           used  (Mb) gc trig

In [7]:
%%writefile Coding_Analysis.R
rm(list=ls())
gc()

arrayid <- as.numeric(commandArgs(TRUE)[1])

trait <- commandArgs(TRUE)[2]
print(trait)

Train_Null_Model <- commandArgs(TRUE)[3]
print(Train_Null_Model)

Annotation_name_catalog <- commandArgs(TRUE)[4]
print(Annotation_name_catalog)

GDS_File <- commandArgs(TRUE)[5]
print(GDS_File)

OUTPUT_PATH <- commandArgs(TRUE)[6]
print(OUTPUT_PATH)

## load required packages
library(gdsfmt)
library(SeqArray)
library(SeqVarTools)
library(STAAR)
library(STAARpipeline)
library(STAARpipelineSummary)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(readr)
library(dplyr)
library(stringr)

coding <- function(chr,gene_name,genofile,obj_nullmodel,genes,
                   rare_maf_cutoff=0.01,rv_num_cutoff=2,
                   QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                   Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                   Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                   SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA))
  {
    use_SPA <- obj_nullmodel$use_SPA
  }else
  {
    use_SPA <- FALSE
  }
  
  ## get SNV id, position, REF, ALT (whole genome)
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant")
  {
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV")
  {
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel")
  {
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  position <- as.numeric(seqGetData(genofile, "position"))
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ### Gene
  kk <- which(genes[,1]==gene_name)
  
  sub_start_loc <- genes[kk,3]
  sub_end_loc <- genes[kk,4]
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  rm(position)
  gc()
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Gencode
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  ## Meta.SVM.Pred
  MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
  
  ################################################
  #           Coding
  ################################################
  variant.id.gene <- seqGetData(genofile, "variant.id")
  lof.in.coding <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")|(GENCODE.EXONIC.Category=="nonsynonymous SNV")|(GENCODE.EXONIC.Category=="synonymous SNV")
  variant.id.gene <- variant.id.gene[lof.in.coding]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Gencode
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  ## Meta.SVM.Pred
  MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV")
  {
    if(Use_annotation_weights)
    {
      for(k in 1:length(Annotation_name))
      {
        if(Annotation_name[k]%in%Annotation_name_catalog$name)
        {
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD")
          {
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity")
          {
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  ################################################
  #                  plof_ds
  ################################################
  variant.id.gene <- seqGetData(genofile, "variant.id")
  lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
  variant.id.gene.category <- variant.id.gene[lof.in.plof]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.plof,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_plof_ds <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "plof_ds"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_plof_ds <- rbind(results_plof_ds,results_temp)
  }
  
  if(!is.null(results_plof_ds))
  {
    if(!use_SPA)
    {
      colnames(results_plof_ds) <- colnames(results_plof_ds, do.NULL = FALSE, prefix = "col")
      colnames(results_plof_ds)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_plof_ds)[(dim(results_plof_ds)[2]-1):dim(results_plof_ds)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results_plof_ds) <- colnames(results_plof_ds, do.NULL = FALSE, prefix = "col")
      colnames(results_plof_ds)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_plof_ds)[dim(results_plof_ds)[2]] <- c("STAAR-B")
      
    }
  }
  
  #####################################################
  #                      plof
  #####################################################
  lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")
  variant.id.gene.category <- variant.id.gene[lof.in.plof]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.plof,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_plof <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "plof"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_plof <- rbind(results_plof,results_temp)
  }
  
  if(!is.null(results_plof))
  {
    if(!use_SPA)
    {
      colnames(results_plof) <- colnames(results_plof, do.NULL = FALSE, prefix = "col")
      colnames(results_plof)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_plof)[(dim(results_plof)[2]-1):dim(results_plof)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results_plof) <- colnames(results_plof, do.NULL = FALSE, prefix = "col")
      colnames(results_plof)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_plof)[dim(results_plof)[2]] <- c("STAAR-B")
    }
  }
  
  #############################################
  #             synonymous
  #############################################
  lof.in.synonymous <- (GENCODE.EXONIC.Category=="synonymous SNV")
  variant.id.gene.category <- variant.id.gene[lof.in.synonymous]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.synonymous,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_synonymous <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "synonymous"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_synonymous <- rbind(results_synonymous,results_temp)
  }
  
  if(!is.null(results_synonymous))
  {
    if(!use_SPA)
    {
      colnames(results_synonymous) <- colnames(results_synonymous, do.NULL = FALSE, prefix = "col")
      colnames(results_synonymous)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_synonymous)[(dim(results_synonymous)[2]-1):dim(results_synonymous)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results_synonymous) <- colnames(results_synonymous, do.NULL = FALSE, prefix = "col")
      colnames(results_synonymous)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_synonymous)[dim(results_synonymous)[2]] <- c("STAAR-B")
    }
    
  }
  
  #################################################
  #        missense
  #################################################
  lof.in.missense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")
  variant.id.gene.category <- variant.id.gene[lof.in.missense]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.missense,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "missense"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  #################################################
  #         disruptive missense
  #################################################
  lof.in.dmissense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D")
  variant.id.gene.category <- variant.id.gene[lof.in.dmissense]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.dmissense,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "disruptive_missense"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results))
  {
    if(!use_SPA)
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
    
    if(dim(results)[1]==1)
    {
      if(results[3]!="disruptive_missense")
      {
        if(!use_SPA)
        {
          results <- cbind(results,matrix(1,1,6))
          colnames(results)[(dim(results)[2]-5):dim(results)[2]] <- c("SKAT(1,25)-Disruptive","SKAT(1,1)-Disruptive","Burden(1,25)-Disruptive","Burden(1,1)-Disruptive","ACAT-V(1,25)-Disruptive","ACAT-V(1,1)-Disruptive")
          results_missense <- results
          results_ds <- c()
        }else{
          results <- cbind(results,matrix(1,1,2))
          colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("Burden(1,25)-Disruptive","Burden(1,1)-Disruptive")
          results_missense <- results
          results_ds <- c()
        }
      }else
      {
        results_missense <- c()
        results_ds <- results
        results <- c()
      }
    }
    
    if(!is.null(results))
    {
      if(dim(results)[1]==2)
      {
        if(!use_SPA)
        {
          results_m <- c(results[1,],rep(0,6))
          names(results_m)[(length(results_m)-5):length(results_m)] <- c("SKAT(1,25)-Disruptive","SKAT(1,1)-Disruptive","Burden(1,25)-Disruptive","Burden(1,1)-Disruptive","ACAT-V(1,25)-Disruptive","ACAT-V(1,1)-Disruptive")
          results_m[(length(results_m)-5):length(results_m)] <- results[2,c("SKAT(1,25)","SKAT(1,1)","Burden(1,25)","Burden(1,1)","ACAT-V(1,25)","ACAT-V(1,1)")]
          apc_num <- (length(results_m)-19)/6
          p_seq <- c(1:apc_num,1:apc_num+(apc_num+1),1:apc_num+2*(apc_num+1),1:apc_num+3*(apc_num+1),1:apc_num+4*(apc_num+1),1:apc_num+5*(apc_num+1),(6*apc_num+9):(6*apc_num+14))
          results_m["STAAR-O"] <- CCT(as.numeric(results_m[6:length(results_m)][p_seq]))
          results_m["STAAR-S(1,25)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num,6*apc_num+9)]))
          results_m["STAAR-S(1,1)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+(apc_num+1),6*apc_num+10)]))
          results_m["STAAR-B(1,25)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+2*(apc_num+1),6*apc_num+11)]))
          results_m["STAAR-B(1,1)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+3*(apc_num+1),6*apc_num+12)]))
          results_m["STAAR-A(1,25)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+4*(apc_num+1),6*apc_num+13)]))
          results_m["STAAR-A(1,1)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+5*(apc_num+1),6*apc_num+14)]))
          
          results_ds <- c()
          results_ds <- rbind(results_ds,results[2,])
          
          results <- c()
          results <- rbind(results,results_m)
        }else
        {
          results_m <- c(results[1,],rep(0,2))
          names(results_m)[(length(results_m)-1):length(results_m)] <- c("Burden(1,25)-Disruptive","Burden(1,1)-Disruptive")
          results_m[(length(results_m)-1):length(results_m)] <- results[2,c("Burden(1,25)","Burden(1,1)")]
          
          ## check whether the p-values is NA. If so, set NA equals 1.
          if(is.na(results_m[(length(results_m)-1)]))
          {
            results_m[(length(results_m)-1)] <- 1
          }
          
          if(is.na(results_m[length(results_m)]))
          {
            results_m[length(results_m)] <- 1
          }
          
          apc_num <- (length(results_m)-10)/2
          p_seq <- c(1:apc_num,1:apc_num+(apc_num+1),(length(results_m)-6):(length(results_m)-5))
          
          ## calculate STAAR-B
          pvalues_sub <- as.numeric(results_m[6:length(results_m)][p_seq])
          if(sum(is.na(pvalues_sub))>0)
          {
            if(sum(is.na(pvalues_sub))==length(pvalues_sub))
            {
              results_m["STAAR-B"] <- 1
            }else
            {
              ## not all NAs
              pvalues_sub <- pvalues_sub[!is.na(pvalues_sub)]
              if(sum(pvalues_sub[pvalues_sub<1])>0)
              {
                ## not all ones
                results_m["STAAR-B"] <- CCT(pvalues_sub[pvalues_sub<1])
                
              }else
              {
                results_m["STAAR-B"] <- 1
                
              }
            }
          }else
          {
            if(sum(pvalues_sub[pvalues_sub<1])>0)
            {
              results_m["STAAR-B"] <- CCT(pvalues_sub[pvalues_sub<1])
            }else
            {
              results_m["STAAR-B"] <- 1
            }
          }
          
          ## calculate STAAR-B(1,25)
          pvalues_sub <- as.numeric(results_m[6:length(results_m)][c(1:apc_num,(length(results_m)-6))])
          if(sum(is.na(pvalues_sub))>0)
          {
            if(sum(is.na(pvalues_sub))==length(pvalues_sub))
            {
              results_m["STAAR-B(1,25)"] <- 1
            }else
            {
              ## not all NAs
              pvalues_sub <- pvalues_sub[!is.na(pvalues_sub)]
              if(sum(pvalues_sub[pvalues_sub<1])>0)
              {
                ## not all ones
                results_m["STAAR-B(1,25)"] <- CCT(pvalues_sub[pvalues_sub<1])
                
              }else
              {
                results_m["STAAR-B(1,25)"] <- 1
                
              }
            }
          }else
          {
            if(sum(pvalues_sub[pvalues_sub<1])>0)
            {
              results_m["STAAR-B(1,25)"] <- CCT(pvalues_sub[pvalues_sub<1])
            }else
            {
              results_m["STAAR-B(1,25)"] <- 1
            }
          }
          
          ## calculate STAAR-B(1,1)
          pvalues_sub <- as.numeric(results_m[6:length(results_m)][c(1:apc_num+(apc_num+1),(length(results_m)-5))])
          if(sum(is.na(pvalues_sub))>0)
          {
            if(sum(is.na(pvalues_sub))==length(pvalues_sub))
            {
              results_m["STAAR-B(1,1)"] <- 1
            }else
            {
              ## not all NAs
              pvalues_sub <- pvalues_sub[!is.na(pvalues_sub)]
              if(sum(pvalues_sub[pvalues_sub<1])>0)
              {
                ## not all ones
                results_m["STAAR-B(1,1)"] <- CCT(pvalues_sub[pvalues_sub<1])
                
              }else
              {
                results_m["STAAR-B(1,1)"] <- 1
                
              }
            }
          }else
          {
            if(sum(pvalues_sub[pvalues_sub<1])>0)
            {
              results_m["STAAR-B(1,1)"] <- CCT(pvalues_sub[pvalues_sub<1])
            }else
            {
              results_m["STAAR-B(1,1)"] <- 1
            }
          }
          
          results_ds <- c()
          results_ds <- rbind(results_ds,results[2,])
          
          results <- c()
          results <- rbind(results,results_m)
        }
      }
    }
  }else
  {
    results <- c()
    results_ds <- c()
  }
  
  results_coding <- list(plof=results_plof,plof_ds=results_plof_ds,missense=results,disruptive_missense=results_ds,synonymous=results_synonymous)
  
  seqResetFilter(genofile)
  
  return(results_coding)
}

plof <- function(chr,gene_name,genofile,obj_nullmodel,genes,
                 rare_maf_cutoff=0.01,rv_num_cutoff=2,
                 QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                 Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                 Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                 SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA))
  {
    use_SPA <- obj_nullmodel$use_SPA
  }else
  {
    use_SPA <- FALSE
  }
  
  ## get SNV id, position, REF, ALT (whole genome)
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant")
  {
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV")
  {
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel")
  {
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  position <- as.numeric(seqGetData(genofile, "position"))
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ### Gene
  kk <- which(genes[,1]==gene_name)
  
  sub_start_loc <- genes[kk,3]
  sub_end_loc <- genes[kk,4]
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## plof
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Gencode
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  
  variant.id.gene <- seqGetData(genofile, "variant.id")
  lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")
  variant.id.gene <- variant.id.gene[lof.in.plof]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV")
  {
    if(Use_annotation_weights)
    {
      for(k in 1:length(Annotation_name))
      {
        if(Annotation_name[k]%in%Annotation_name_catalog$name)
        {
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD")
          {
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity")
          {
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "plof"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results))
  {
    if(!use_SPA)
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

plof_ds <- function(chr,gene_name,genofile,obj_nullmodel,genes,
                    rare_maf_cutoff=0.01,rv_num_cutoff=2,
                    QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                    Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                    Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                    SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA))
  {
    use_SPA <- obj_nullmodel$use_SPA
  }else
  {
    use_SPA <- FALSE
  }
  
  ## get SNV id, position, REF, ALT (whole genome)
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant")
  {
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV")
  {
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel")
  {
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  position <- as.numeric(seqGetData(genofile, "position"))
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ### Gene
  kk <- which(genes[,1]==gene_name)
  
  sub_start_loc <- genes[kk,3]
  sub_end_loc <- genes[kk,4]
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## plof_ds
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Gencode
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  ## Meta.SVM.Pred
  MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
  
  variant.id.gene <- seqGetData(genofile, "variant.id")
  lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
  variant.id.gene <- variant.id.gene[lof.in.plof]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV")
  {
    if(Use_annotation_weights)
    {
      for(k in 1:length(Annotation_name))
      {
        if(Annotation_name[k]%in%Annotation_name_catalog$name)
        {
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD")
          {
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity")
          {
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else
    {
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "plof_ds"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results))
  {
    if(!use_SPA)
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

missense <- function(chr,gene_name,genofile,obj_nullmodel,genes,
                     rare_maf_cutoff=0.01,rv_num_cutoff=2,
                     QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                     Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                     Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                     SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA))
  {
    use_SPA <- obj_nullmodel$use_SPA
  }else
  {
    use_SPA <- FALSE
  }
  
  ## get SNV id, position, REF, ALT (whole genome)
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant")
  {
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV")
  {
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel")
  {
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  position <- as.numeric(seqGetData(genofile, "position"))
  variant.id <- seqGetData(genofile, "variant.id")
  
  ### Gene
  kk <- which(genes[,1]==gene_name)
  
  sub_start_loc <- genes[kk,3]
  sub_end_loc <- genes[kk,4]
  
  ############################################################
  #                      missense
  ############################################################
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  
  variant.id.gene <- seqGetData(genofile, "variant.id")
  lof.in.missense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")
  variant.id.gene <- variant.id.gene[lof.in.missense]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV")
  {
    if(Use_annotation_weights)
    {
      for(k in 1:length(Annotation_name))
      {
        if(Annotation_name[k]%in%Annotation_name_catalog$name)
        {
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD")
          {
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity")
          {
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "missense"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  ############################################################
  #                      disruptive missense
  ############################################################
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Meta.SVM.Pred
  MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
  
  variant.id.gene <- seqGetData(genofile, "variant.id")
  lof.in.dmissense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D")
  variant.id.gene <- variant.id.gene[lof.in.dmissense]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV")
  {
    if(Use_annotation_weights)
    {
      for(k in 1:length(Annotation_name))
      {
        if(Annotation_name[k]%in%Annotation_name_catalog$name)
        {
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD")
          {
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity")
          {
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "disruptive_missense"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results))
  {
    if(!use_SPA)
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
    
    if(dim(results)[1]==1)
    {
      if(results[3]!="disruptive_missense")
      {
        if(!use_SPA)
        {
          results <- cbind(results,matrix(1,1,6))
          colnames(results)[(dim(results)[2]-5):dim(results)[2]] <- c("SKAT(1,25)-Disruptive","SKAT(1,1)-Disruptive","Burden(1,25)-Disruptive","Burden(1,1)-Disruptive","ACAT-V(1,25)-Disruptive","ACAT-V(1,1)-Disruptive")
        }else{
          results <- cbind(results,matrix(1,1,2))
          colnames(results)[(dim(results)[2]-5):dim(results)[2]] <- c("Burden(1,25)-Disruptive","Burden(1,1)-Disruptive")
        }
      }else
      {
        results <- c()
      }
    }
    
    if(!is.null(results))
    {
      if(dim(results)[1]==2)
      {
        if(!use_SPA)
        {
          results_m <- c(results[1,],rep(0,6))
          names(results_m)[(length(results_m)-5):length(results_m)] <- c("SKAT(1,25)-Disruptive","SKAT(1,1)-Disruptive","Burden(1,25)-Disruptive","Burden(1,1)-Disruptive","ACAT-V(1,25)-Disruptive","ACAT-V(1,1)-Disruptive")
          results_m[(length(results_m)-5):length(results_m)] <- results[2,c("SKAT(1,25)","SKAT(1,1)","Burden(1,25)","Burden(1,1)","ACAT-V(1,25)","ACAT-V(1,1)")]
          apc_num <- (length(results_m)-19)/6
          p_seq <- c(1:apc_num,1:apc_num+(apc_num+1),1:apc_num+2*(apc_num+1),1:apc_num+3*(apc_num+1),1:apc_num+4*(apc_num+1),1:apc_num+5*(apc_num+1),(6*apc_num+9):(6*apc_num+14))
          results_m["STAAR-O"] <- CCT(as.numeric(results_m[6:length(results_m)][p_seq]))
          results_m["STAAR-S(1,25)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num,6*apc_num+9)]))
          results_m["STAAR-S(1,1)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+(apc_num+1),6*apc_num+10)]))
          results_m["STAAR-B(1,25)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+2*(apc_num+1),6*apc_num+11)]))
          results_m["STAAR-B(1,1)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+3*(apc_num+1),6*apc_num+12)]))
          results_m["STAAR-A(1,25)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+4*(apc_num+1),6*apc_num+13)]))
          results_m["STAAR-A(1,1)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+5*(apc_num+1),6*apc_num+14)]))
          
          results <- c()
          results <- rbind(results,results_m)
        }else
        {
          results_m <- c(results[1,],rep(0,2))
          names(results_m)[(length(results_m)-1):length(results_m)] <- c("Burden(1,25)-Disruptive","Burden(1,1)-Disruptive")
          results_m[(length(results_m)-1):length(results_m)] <- results[2,c("Burden(1,25)","Burden(1,1)")]
          
          ## check whether the p-values is NA. If so, set NA equals 1.
          if(is.na(results_m[(length(results_m)-1)]))
          {
            results_m[(length(results_m)-1)] <- 1
          }
          
          if(is.na(results_m[length(results_m)]))
          {
            results_m[length(results_m)] <- 1
          }
          
          apc_num <- (length(results_m)-10)/2
          p_seq <- c(1:apc_num,1:apc_num+(apc_num+1),(length(results_m)-6):(length(results_m)-5))
          
          ## calculate STAAR-B
          pvalues_sub <- as.numeric(results_m[6:length(results_m)][p_seq])
          if(sum(is.na(pvalues_sub))>0)
          {
            if(sum(is.na(pvalues_sub))==length(pvalues_sub))
            {
              results_m["STAAR-B"] <- 1
            }else
            {
              ## not all NAs
              pvalues_sub <- pvalues_sub[!is.na(pvalues_sub)]
              if(sum(pvalues_sub[pvalues_sub<1])>0)
              {
                ## not all ones
                results_m["STAAR-B"] <- CCT(pvalues_sub[pvalues_sub<1])
                
              }else
              {
                results_m["STAAR-B"] <- 1
                
              }
            }
          }else
          {
            if(sum(pvalues_sub[pvalues_sub<1])>0)
            {
              results_m["STAAR-B"] <- CCT(pvalues_sub[pvalues_sub<1])
            }else
            {
              results_m["STAAR-B"] <- 1
            }
          }
          
          ## calculate STAAR-B(1,25)
          pvalues_sub <- as.numeric(results_m[6:length(results_m)][c(1:apc_num,(length(results_m)-6))])
          if(sum(is.na(pvalues_sub))>0)
          {
            if(sum(is.na(pvalues_sub))==length(pvalues_sub))
            {
              results_m["STAAR-B(1,25)"] <- 1
            }else
            {
              ## not all NAs
              pvalues_sub <- pvalues_sub[!is.na(pvalues_sub)]
              if(sum(pvalues_sub[pvalues_sub<1])>0)
              {
                ## not all ones
                results_m["STAAR-B(1,25)"] <- CCT(pvalues_sub[pvalues_sub<1])
                
              }else
              {
                results_m["STAAR-B(1,25)"] <- 1
                
              }
            }
          }else
          {
            if(sum(pvalues_sub[pvalues_sub<1])>0)
            {
              results_m["STAAR-B(1,25)"] <- CCT(pvalues_sub[pvalues_sub<1])
            }else
            {
              results_m["STAAR-B(1,25)"] <- 1
            }
          }
          
          ## calculate STAAR-B(1,1)
          pvalues_sub <- as.numeric(results_m[6:length(results_m)][c(1:apc_num+(apc_num+1),(length(results_m)-5))])
          if(sum(is.na(pvalues_sub))>0)
          {
            if(sum(is.na(pvalues_sub))==length(pvalues_sub))
            {
              results_m["STAAR-B(1,1)"] <- 1
            }else
            {
              ## not all NAs
              pvalues_sub <- pvalues_sub[!is.na(pvalues_sub)]
              if(sum(pvalues_sub[pvalues_sub<1])>0)
              {
                ## not all ones
                results_m["STAAR-B(1,1)"] <- CCT(pvalues_sub[pvalues_sub<1])
                
              }else
              {
                results_m["STAAR-B(1,1)"] <- 1
                
              }
            }
          }else
          {
            if(sum(pvalues_sub[pvalues_sub<1])>0)
            {
              results_m["STAAR-B(1,1)"] <- CCT(pvalues_sub[pvalues_sub<1])
            }else
            {
              results_m["STAAR-B(1,1)"] <- 1
            }
          }
          
          results <- c()
          results <- rbind(results,results_m)
          
        }
      }
    }
  }else
  {
    results <- c()
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

disruptive_missense <- function(chr,gene_name,genofile,obj_nullmodel,genes,
                                rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                                Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                                Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                                SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA))
  {
    use_SPA <- obj_nullmodel$use_SPA
  }else
  {
    use_SPA <- FALSE
  }
  
  ## get SNV id, position, REF, ALT (whole genome)
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant")
  {
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV")
  {
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel")
  {
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  position <- as.numeric(seqGetData(genofile, "position"))
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ### Gene
  kk <- which(genes[,1]==gene_name)
  
  sub_start_loc <- genes[kk,3]
  sub_end_loc <- genes[kk,4]
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## disruptive_missense
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Meta.SVM.Pred
  MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
  
  variant.id.gene <- seqGetData(genofile, "variant.id")
  lof.in.ds <- ((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
  variant.id.gene <- variant.id.gene[lof.in.ds]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV")
  {
    if(Use_annotation_weights)
    {
      for(k in 1:length(Annotation_name))
      {
        if(Annotation_name[k]%in%Annotation_name_catalog$name)
        {
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD")
          {
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity")
          {
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "disruptive_missense"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results))
  {
    if(!use_SPA)
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

synonymous <- function(chr,gene_name,genofile,obj_nullmodel,genes,
                       rare_maf_cutoff=0.01,rv_num_cutoff=2,
                       QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                       Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                       Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                       SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA))
  {
    use_SPA <- obj_nullmodel$use_SPA
  }else
  {
    use_SPA <- FALSE
  }
  
  ## get SNV id, position, REF, ALT (whole genome)
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant")
  {
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV")
  {
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel")
  {
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  position <- as.numeric(seqGetData(genofile, "position"))
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ### Gene
  kk <- which(genes[,1]==gene_name)
  
  sub_start_loc <- genes[kk,3]
  sub_end_loc <- genes[kk,4]
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ### synonymous
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  
  variant.id.gene <- seqGetData(genofile, "variant.id")
  lof.in.synonymous <- (GENCODE.EXONIC.Category=="synonymous SNV")
  variant.id.gene <- variant.id.gene[lof.in.synonymous]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV")
  {
    if(Use_annotation_weights)
    {
      for(k in 1:length(Annotation_name))
      {
        if(Annotation_name[k]%in%Annotation_name_catalog$name)
        {
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD")
          {
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity")
          {
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "synonymous"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results))
  {
    if(!use_SPA)
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

ptv <- function(chr,gene_name,genofile,obj_nullmodel,genes,
                rare_maf_cutoff=0.01,rv_num_cutoff=2,
                QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA))
  {
    use_SPA <- obj_nullmodel$use_SPA
  }else
  {
    use_SPA <- FALSE
  }
  
  ## get SNV id, position, REF, ALT (whole genome)
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant")
  {
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV")
  {
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel")
  {
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  position <- as.numeric(seqGetData(genofile, "position"))
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ### Gene
  kk <- which(genes[,1]==gene_name)
  
  sub_start_loc <- genes[kk,3]
  sub_end_loc <- genes[kk,4]
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## plof
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Gencode
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  
  variant.id.gene <- seqGetData(genofile, "variant.id")
  
  if(variant_type=="SNV")
  {
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")
  }
  
  if(variant_type=="Indel")
  {
    lof.in.plof <- (GENCODE.EXONIC.Category=="frameshift deletion")|(GENCODE.EXONIC.Category=="frameshift insertion")
  }
  
  if(variant_type=="variant")
  {
    lof.in.plof.snv <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")
    lof.in.plof.indel <- (GENCODE.EXONIC.Category=="frameshift deletion")|(GENCODE.EXONIC.Category=="frameshift insertion")
    lof.in.plof <- lof.in.plof.snv|lof.in.plof.indel
  }
  
  variant.id.gene <- variant.id.gene[lof.in.plof]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV")
  {
    if(Use_annotation_weights)
    {
      for(k in 1:length(Annotation_name))
      {
        if(Annotation_name[k]%in%Annotation_name_catalog$name)
        {
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD")
          {
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity")
          {
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "ptv"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results))
  {
    if(!use_SPA)
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

ptv_ds <- function(chr,gene_name,genofile,obj_nullmodel,genes,
                   rare_maf_cutoff=0.01,rv_num_cutoff=2,
                   QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                   Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                   Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                   SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA))
  {
    use_SPA <- obj_nullmodel$use_SPA
  }else
  {
    use_SPA <- FALSE
  }
  
  ## get SNV id, position, REF, ALT (whole genome)
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant")
  {
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV")
  {
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel")
  {
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  position <- as.numeric(seqGetData(genofile, "position"))
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ### Gene
  kk <- which(genes[,1]==gene_name)
  
  sub_start_loc <- genes[kk,3]
  sub_end_loc <- genes[kk,4]
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## plof_ds
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Gencode
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  ## Meta.SVM.Pred
  MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
  
  variant.id.gene <- seqGetData(genofile, "variant.id")
  
  if(variant_type=="SNV")
  {
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
  }
  
  if(variant_type=="Indel")
  {
    lof.in.plof <- (GENCODE.EXONIC.Category=="frameshift deletion")|(GENCODE.EXONIC.Category=="frameshift insertion")|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
  }
  
  if(variant_type=="variant")
  {
    lof.in.plof.snv <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")
    lof.in.plof.indel <- (GENCODE.EXONIC.Category=="frameshift deletion")|(GENCODE.EXONIC.Category=="frameshift insertion")
    lof.in.plof <- lof.in.plof.snv|lof.in.plof.indel|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
  }
  
  variant.id.gene <- variant.id.gene[lof.in.plof]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV")
  {
    if(Use_annotation_weights)
    {
      for(k in 1:length(Annotation_name))
      {
        if(Annotation_name[k]%in%Annotation_name_catalog$name)
        {
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD")
          {
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity")
          {
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else
    {
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "ptv_ds"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results))
  {
    if(!use_SPA)
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
  }
  
  seqResetFilter(genofile)
  
  return(results)
}

coding_incl_ptv <- function(chr,gene_name,genofile,obj_nullmodel,genes,
                            rare_maf_cutoff=0.01,rv_num_cutoff=2,
                            QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                            Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                            Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                            SPA_p_filter=FALSE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  phenotype.id <- obj_nullmodel$id_include
  n_pheno <- obj_nullmodel$n.pheno
  
  ## SPA status
  if(!is.null(obj_nullmodel$use_SPA))
  {
    use_SPA <- obj_nullmodel$use_SPA
  }else
  {
    use_SPA <- FALSE
  }
  
  ## get SNV id, position, REF, ALT (whole genome)
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant")
  {
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV")
  {
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel")
  {
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  position <- as.numeric(seqGetData(genofile, "position"))
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ### Gene
  kk <- which(genes[,1]==gene_name)
  
  sub_start_loc <- genes[kk,3]
  sub_end_loc <- genes[kk,4]
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  rm(position)
  gc()
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Gencode
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  ## Meta.SVM.Pred
  MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
  
  ################################################
  #           Coding
  ################################################
  variant.id.gene <- seqGetData(genofile, "variant.id")
  lof.in.coding <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")|(GENCODE.EXONIC.Category=="nonsynonymous SNV")|(GENCODE.EXONIC.Category=="synonymous SNV")|(GENCODE.EXONIC.Category=="frameshift deletion")|(GENCODE.EXONIC.Category=="frameshift insertion")
  variant.id.gene <- variant.id.gene[lof.in.coding]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## Gencode_Exonic
  GENCODE.EXONIC.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Gencode
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  ## Meta.SVM.Pred
  MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
  
  ## Annotation
  Anno.Int.PHRED.sub <- NULL
  Anno.Int.PHRED.sub.name <- NULL
  
  if(variant_type=="SNV")
  {
    if(Use_annotation_weights)
    {
      for(k in 1:length(Annotation_name))
      {
        if(Annotation_name[k]%in%Annotation_name_catalog$name)
        {
          Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,Annotation_name[k])
          Annotation.PHRED <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name==Annotation_name[k])]))
          
          if(Annotation_name[k]=="CADD")
          {
            Annotation.PHRED[is.na(Annotation.PHRED)] <- 0
          }
          
          if(Annotation_name[k]=="aPC.LocalDiversity")
          {
            Annotation.PHRED.2 <- -10*log10(1-10^(-Annotation.PHRED/10))
            Annotation.PHRED <- cbind(Annotation.PHRED,Annotation.PHRED.2)
            Anno.Int.PHRED.sub.name <- c(Anno.Int.PHRED.sub.name,paste0(Annotation_name[k],"(-)"))
          }
          Anno.Int.PHRED.sub <- cbind(Anno.Int.PHRED.sub,Annotation.PHRED)
        }
      }
      
      Anno.Int.PHRED.sub <- data.frame(Anno.Int.PHRED.sub)
      colnames(Anno.Int.PHRED.sub) <- Anno.Int.PHRED.sub.name
    }
  }
  
  ################################################
  #                  plof_ds
  ################################################
  variant.id.gene <- seqGetData(genofile, "variant.id")
  lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
  variant.id.gene.category <- variant.id.gene[lof.in.plof]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.plof,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_plof_ds <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "plof_ds"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_plof_ds <- rbind(results_plof_ds,results_temp)
  }
  
  if(!is.null(results_plof_ds))
  {
    if(!use_SPA)
    {
      colnames(results_plof_ds) <- colnames(results_plof_ds, do.NULL = FALSE, prefix = "col")
      colnames(results_plof_ds)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_plof_ds)[(dim(results_plof_ds)[2]-1):dim(results_plof_ds)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results_plof_ds) <- colnames(results_plof_ds, do.NULL = FALSE, prefix = "col")
      colnames(results_plof_ds)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_plof_ds)[dim(results_plof_ds)[2]] <- c("STAAR-B")
      
    }
  }
  
  ################################################
  #                  ptv_ds
  ################################################
  if(variant_type=="SNV")
  {
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
  }
  
  if(variant_type=="Indel")
  {
    lof.in.plof <- (GENCODE.EXONIC.Category=="frameshift deletion")|(GENCODE.EXONIC.Category=="frameshift insertion")|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
  }
  
  if(variant_type=="variant")
  {
    lof.in.plof.snv <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")
    lof.in.plof.indel <- (GENCODE.EXONIC.Category=="frameshift deletion")|(GENCODE.EXONIC.Category=="frameshift insertion")
    lof.in.plof <- lof.in.plof.snv|lof.in.plof.indel|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
  }
  
  variant.id.gene.category <- variant.id.gene[lof.in.plof]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.plof,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_ptv_ds <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "ptv_ds"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_ptv_ds <- rbind(results_ptv_ds,results_temp)
  }
  
  if(!is.null(results_ptv_ds))
  {
    if(!use_SPA)
    {
      colnames(results_ptv_ds) <- colnames(results_ptv_ds, do.NULL = FALSE, prefix = "col")
      colnames(results_ptv_ds)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_ptv_ds)[(dim(results_ptv_ds)[2]-1):dim(results_ptv_ds)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results_ptv_ds) <- colnames(results_ptv_ds, do.NULL = FALSE, prefix = "col")
      colnames(results_ptv_ds)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_ptv_ds)[dim(results_ptv_ds)[2]] <- c("STAAR-B")
      
    }
  }
  
  #####################################################
  #                      plof
  #####################################################
  lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")
  variant.id.gene.category <- variant.id.gene[lof.in.plof]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.plof,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_plof <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "plof"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_plof <- rbind(results_plof,results_temp)
  }
  
  if(!is.null(results_plof))
  {
    if(!use_SPA)
    {
      colnames(results_plof) <- colnames(results_plof, do.NULL = FALSE, prefix = "col")
      colnames(results_plof)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_plof)[(dim(results_plof)[2]-1):dim(results_plof)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results_plof) <- colnames(results_plof, do.NULL = FALSE, prefix = "col")
      colnames(results_plof)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_plof)[dim(results_plof)[2]] <- c("STAAR-B")
    }
  }
  
  #####################################################
  #                      ptv
  #####################################################
  if(variant_type=="SNV")
  {
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")
  }
  
  if(variant_type=="Indel")
  {
    lof.in.plof <- (GENCODE.EXONIC.Category=="frameshift deletion")|(GENCODE.EXONIC.Category=="frameshift insertion")
  }
  
  if(variant_type=="variant")
  {
    lof.in.plof.snv <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")
    lof.in.plof.indel <- (GENCODE.EXONIC.Category=="frameshift deletion")|(GENCODE.EXONIC.Category=="frameshift insertion")
    lof.in.plof <- lof.in.plof.snv|lof.in.plof.indel
  }
  
  variant.id.gene.category <- variant.id.gene[lof.in.plof]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.plof,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_ptv <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "ptv"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_ptv <- rbind(results_ptv,results_temp)
  }
  
  if(!is.null(results_ptv))
  {
    if(!use_SPA)
    {
      colnames(results_ptv) <- colnames(results_ptv, do.NULL = FALSE, prefix = "col")
      colnames(results_ptv)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_ptv)[(dim(results_ptv)[2]-1):dim(results_ptv)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results_ptv) <- colnames(results_ptv, do.NULL = FALSE, prefix = "col")
      colnames(results_ptv)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_ptv)[dim(results_ptv)[2]] <- c("STAAR-B")
    }
  }
  
  #############################################
  #             synonymous
  #############################################
  lof.in.synonymous <- (GENCODE.EXONIC.Category=="synonymous SNV")
  variant.id.gene.category <- variant.id.gene[lof.in.synonymous]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.synonymous,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results_synonymous <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "synonymous"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results_synonymous <- rbind(results_synonymous,results_temp)
  }
  
  if(!is.null(results_synonymous))
  {
    if(!use_SPA)
    {
      colnames(results_synonymous) <- colnames(results_synonymous, do.NULL = FALSE, prefix = "col")
      colnames(results_synonymous)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_synonymous)[(dim(results_synonymous)[2]-1):dim(results_synonymous)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results_synonymous) <- colnames(results_synonymous, do.NULL = FALSE, prefix = "col")
      colnames(results_synonymous)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results_synonymous)[dim(results_synonymous)[2]] <- c("STAAR-B")
    }
    
  }
  
  #################################################
  #        missense
  #################################################
  lof.in.missense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")
  variant.id.gene.category <- variant.id.gene[lof.in.missense]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.missense,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  results <- c()
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "missense"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  #################################################
  #         disruptive missense
  #################################################
  lof.in.dmissense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D")
  variant.id.gene.category <- variant.id.gene[lof.in.dmissense]
  
  seqSetFilter(genofile,variant.id=variant.id.gene.category,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  ## Annotation
  Anno.Int.PHRED.sub.category <- Anno.Int.PHRED.sub[lof.in.dmissense,]
  
  pvalues <- 0
  if(n_pheno == 1)
  {
    if(!use_SPA)
    {
      try(pvalues <- STAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
    }else{
      try(pvalues <- STAAR_Binary_SPA(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff),silent=silent)
    }
  }else
  {
    try(pvalues <- MultiSTAAR(Geno,obj_nullmodel,Anno.Int.PHRED.sub.category,rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff),silent=silent)
  }
  
  if(inherits(pvalues, "list"))
  {
    results_temp <- as.vector(genes[kk,])
    results_temp[3] <- "disruptive_missense"
    results_temp[2] <- chr
    results_temp[1] <- as.character(genes[kk,1])
    results_temp[4] <- pvalues$num_variant
    
    if(!use_SPA)
    {
      results_temp <- c(results_temp,pvalues$cMAC,pvalues$results_STAAR_S_1_25,pvalues$results_STAAR_S_1_1,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_A_1_25,
                        pvalues$results_STAAR_A_1_1,pvalues$results_ACAT_O,pvalues$results_STAAR_O)
    }else
    {
      results_temp <- c(results_temp,pvalues$cMAC,
                        pvalues$results_STAAR_B_1_25,pvalues$results_STAAR_B_1_1,pvalues$results_STAAR_B)
    }
    
    results <- rbind(results,results_temp)
  }
  
  if(!is.null(results))
  {
    if(!use_SPA)
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("ACAT-O","STAAR-O")
    }else
    {
      colnames(results) <- colnames(results, do.NULL = FALSE, prefix = "col")
      colnames(results)[1:5] <- c("Gene name","Chr","Category","#SNV","cMAC")
      colnames(results)[dim(results)[2]] <- c("STAAR-B")
    }
    
    if(dim(results)[1]==1)
    {
      if(results[3]!="disruptive_missense")
      {
        if(!use_SPA)
        {
          results <- cbind(results,matrix(1,1,6))
          colnames(results)[(dim(results)[2]-5):dim(results)[2]] <- c("SKAT(1,25)-Disruptive","SKAT(1,1)-Disruptive","Burden(1,25)-Disruptive","Burden(1,1)-Disruptive","ACAT-V(1,25)-Disruptive","ACAT-V(1,1)-Disruptive")
          results_missense <- results
          results_ds <- c()
        }else{
          results <- cbind(results,matrix(1,1,2))
          colnames(results)[(dim(results)[2]-1):dim(results)[2]] <- c("Burden(1,25)-Disruptive","Burden(1,1)-Disruptive")
          results_missense <- results
          results_ds <- c()
        }
      }else
      {
        results_missense <- c()
        results_ds <- results
        results <- c()
      }
    }
    
    if(!is.null(results))
    {
      if(dim(results)[1]==2)
      {
        if(!use_SPA)
        {
          results_m <- c(results[1,],rep(0,6))
          names(results_m)[(length(results_m)-5):length(results_m)] <- c("SKAT(1,25)-Disruptive","SKAT(1,1)-Disruptive","Burden(1,25)-Disruptive","Burden(1,1)-Disruptive","ACAT-V(1,25)-Disruptive","ACAT-V(1,1)-Disruptive")
          results_m[(length(results_m)-5):length(results_m)] <- results[2,c("SKAT(1,25)","SKAT(1,1)","Burden(1,25)","Burden(1,1)","ACAT-V(1,25)","ACAT-V(1,1)")]
          apc_num <- (length(results_m)-19)/6
          p_seq <- c(1:apc_num,1:apc_num+(apc_num+1),1:apc_num+2*(apc_num+1),1:apc_num+3*(apc_num+1),1:apc_num+4*(apc_num+1),1:apc_num+5*(apc_num+1),(6*apc_num+9):(6*apc_num+14))
          results_m["STAAR-O"] <- CCT(as.numeric(results_m[6:length(results_m)][p_seq]))
          results_m["STAAR-S(1,25)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num,6*apc_num+9)]))
          results_m["STAAR-S(1,1)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+(apc_num+1),6*apc_num+10)]))
          results_m["STAAR-B(1,25)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+2*(apc_num+1),6*apc_num+11)]))
          results_m["STAAR-B(1,1)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+3*(apc_num+1),6*apc_num+12)]))
          results_m["STAAR-A(1,25)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+4*(apc_num+1),6*apc_num+13)]))
          results_m["STAAR-A(1,1)"] <- CCT(as.numeric(results_m[6:length(results_m)][c(1:apc_num+5*(apc_num+1),6*apc_num+14)]))
          
          results_ds <- c()
          results_ds <- rbind(results_ds,results[2,])
          
          results <- c()
          results <- rbind(results,results_m)
        }else
        {
          results_m <- c(results[1,],rep(0,2))
          names(results_m)[(length(results_m)-1):length(results_m)] <- c("Burden(1,25)-Disruptive","Burden(1,1)-Disruptive")
          results_m[(length(results_m)-1):length(results_m)] <- results[2,c("Burden(1,25)","Burden(1,1)")]
          
          ## check whether the p-values is NA. If so, set NA equals 1.
          if(is.na(results_m[(length(results_m)-1)]))
          {
            results_m[(length(results_m)-1)] <- 1
          }
          
          if(is.na(results_m[length(results_m)]))
          {
            results_m[length(results_m)] <- 1
          }
          
          apc_num <- (length(results_m)-10)/2
          p_seq <- c(1:apc_num,1:apc_num+(apc_num+1),(length(results_m)-6):(length(results_m)-5))
          
          ## calculate STAAR-B
          pvalues_sub <- as.numeric(results_m[6:length(results_m)][p_seq])
          if(sum(is.na(pvalues_sub))>0)
          {
            if(sum(is.na(pvalues_sub))==length(pvalues_sub))
            {
              results_m["STAAR-B"] <- 1
            }else
            {
              ## not all NAs
              pvalues_sub <- pvalues_sub[!is.na(pvalues_sub)]
              if(sum(pvalues_sub[pvalues_sub<1])>0)
              {
                ## not all ones
                results_m["STAAR-B"] <- CCT(pvalues_sub[pvalues_sub<1])
                
              }else
              {
                results_m["STAAR-B"] <- 1
                
              }
            }
          }else
          {
            if(sum(pvalues_sub[pvalues_sub<1])>0)
            {
              results_m["STAAR-B"] <- CCT(pvalues_sub[pvalues_sub<1])
            }else
            {
              results_m["STAAR-B"] <- 1
            }
          }
          
          ## calculate STAAR-B(1,25)
          pvalues_sub <- as.numeric(results_m[6:length(results_m)][c(1:apc_num,(length(results_m)-6))])
          if(sum(is.na(pvalues_sub))>0)
          {
            if(sum(is.na(pvalues_sub))==length(pvalues_sub))
            {
              results_m["STAAR-B(1,25)"] <- 1
            }else
            {
              ## not all NAs
              pvalues_sub <- pvalues_sub[!is.na(pvalues_sub)]
              if(sum(pvalues_sub[pvalues_sub<1])>0)
              {
                ## not all ones
                results_m["STAAR-B(1,25)"] <- CCT(pvalues_sub[pvalues_sub<1])
                
              }else
              {
                results_m["STAAR-B(1,25)"] <- 1
                
              }
            }
          }else
          {
            if(sum(pvalues_sub[pvalues_sub<1])>0)
            {
              results_m["STAAR-B(1,25)"] <- CCT(pvalues_sub[pvalues_sub<1])
            }else
            {
              results_m["STAAR-B(1,25)"] <- 1
            }
          }
          
          ## calculate STAAR-B(1,1)
          pvalues_sub <- as.numeric(results_m[6:length(results_m)][c(1:apc_num+(apc_num+1),(length(results_m)-5))])
          if(sum(is.na(pvalues_sub))>0)
          {
            if(sum(is.na(pvalues_sub))==length(pvalues_sub))
            {
              results_m["STAAR-B(1,1)"] <- 1
            }else
            {
              ## not all NAs
              pvalues_sub <- pvalues_sub[!is.na(pvalues_sub)]
              if(sum(pvalues_sub[pvalues_sub<1])>0)
              {
                ## not all ones
                results_m["STAAR-B(1,1)"] <- CCT(pvalues_sub[pvalues_sub<1])
                
              }else
              {
                results_m["STAAR-B(1,1)"] <- 1
                
              }
            }
          }else
          {
            if(sum(pvalues_sub[pvalues_sub<1])>0)
            {
              results_m["STAAR-B(1,1)"] <- CCT(pvalues_sub[pvalues_sub<1])
            }else
            {
              results_m["STAAR-B(1,1)"] <- 1
            }
          }
          
          results_ds <- c()
          results_ds <- rbind(results_ds,results[2,])
          
          results <- c()
          results <- rbind(results,results_m)
        }
      }
    }
  }else
  {
    results <- c()
    results_ds <- c()
  }
  
  results_coding <- list(plof=results_plof,plof_ds=results_plof_ds,missense=results,disruptive_missense=results_ds,synonymous=results_synonymous,ptv=results_ptv,ptv_ds=results_ptv_ds)
  
  seqResetFilter(genofile)
  
  return(results_coding)
}

Gene_Centric_Coding <- function(chr,gene_name,category=c("all_categories","plof","plof_ds","missense","disruptive_missense","synonymous","ptv","ptv_ds","all_categories_incl_ptv"),
                                genofile,obj_nullmodel,rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                                Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,
                                Use_annotation_weights=c(TRUE,FALSE),Annotation_name=NULL,
                                SPA_p_filter=TRUE,p_filter_cutoff=0.05,silent=FALSE){
  
  ## evaluate choices
  category <- match.arg(category)
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  genes <- genes_info[genes_info[,2]==chr,]
  
  if(category=="all_categories")
  {
    results <- coding(chr,gene_name,genofile,obj_nullmodel,genes,
                      rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                      QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                      Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                      Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                      SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="plof")
  {
    results <- plof(chr,gene_name,genofile,obj_nullmodel,genes,
                    rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                    QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                    Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                    Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                    SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="plof_ds")
  {
    results <- plof_ds(chr,gene_name,genofile,obj_nullmodel,genes,
                       rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                       QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                       Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                       Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                       SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="missense")
  {
    results <- missense(chr,gene_name,genofile,obj_nullmodel,genes,
                        rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                        QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                        Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                        Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                        SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="disruptive_missense")
  {
    results <- disruptive_missense(chr,gene_name,genofile,obj_nullmodel,genes,
                                   rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                                   QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                   Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                   Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                                   SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  if(category=="synonymous")
  {
    results <- synonymous(chr,gene_name,genofile,obj_nullmodel,genes,
                          rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                          QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                          Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                          Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                          SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  
  if(category=="ptv")
  {
    results <- ptv(chr,gene_name,genofile,obj_nullmodel,genes,
                   rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                   QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                   Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                   Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                   SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  
  if(category=="ptv_ds")
  {
    results <- ptv_ds(chr,gene_name,genofile,obj_nullmodel,genes,
                      rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                      QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                      Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                      Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                      SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  
  if(category=="all_categories_incl_ptv")
  {
    results <- coding_incl_ptv(chr,gene_name,genofile,obj_nullmodel,genes,
                               rare_maf_cutoff=rare_maf_cutoff,rv_num_cutoff=rv_num_cutoff,
                               QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                               Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                               Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,
                               SPA_p_filter=SPA_p_filter,p_filter_cutoff=p_filter_cutoff,silent=silent)
  }
  
  return(results)
}

obj_nullmodel <- get(load(Train_Null_Model))

## Parameter
QC_label <- "annotation/filter"
geno_missing_imputation <- "mean"
variant_type <- "SNV"

## Annotation_dir
Annotation_dir <- "annotation/info/FunctionalAnnotation"
## Annotation channel
Annotation_name_catalog <- read.csv(Annotation_name_catalog)
## Use_annotation_weights
Use_annotation_weights <- TRUE
## Annotation name
Annotation_name <- c("CADD","LINSIGHT","FATHMM.XF","aPC.EpigeneticActive","aPC.EpigeneticRepressed","aPC.EpigeneticTranscription",
                     "aPC.Conservation","aPC.LocalDiversity","aPC.Mappability","aPC.TF","aPC.Protein")

## output file name
output_file_name <- paste0(OUTPUT_PATH,"/",trait,"_AoU_WGS_Coding_Train")

## Chr
gene_num_in_array <- 50 
group.num.allchr <- ceiling(table(genes_info[,2])/gene_num_in_array)
sum(group.num.allchr)

chr <- which.max(arrayid <= cumsum(group.num.allchr))
group.num <- group.num.allchr[chr]

if (chr == 1){
  groupid <- arrayid
}else{
  groupid <- arrayid - cumsum(group.num.allchr)[chr-1]
}

coding_longmasks <- c("TTN","PCDHA2","PCDHA3","PCDHGA1","PCDHGA2","PCDHGA3","PCDHGB1","PCDHGA4","PCDHGB2","PCDHGB3")
genes_info_chr <- genes_info[genes_info[,2]==chr,]
genes_info_chr <- genes_info_chr[!genes_info_chr[,1] %in% coding_longmasks,]
sub_seq_num <- dim(genes_info_chr)[1]

if(groupid < group.num){ 
  sub_seq_id <- ((groupid - 1)*gene_num_in_array + 1):(groupid*gene_num_in_array)
}else{
  sub_seq_id <- ((groupid - 1)*gene_num_in_array + 1):sub_seq_num
}

agds.path <- GDS_File

genofile <- seqOpen(agds.path)

gene_centric_coding_dnanexus <- function(genes_info_chr,gene_name,chr,genofile,obj_nullmodel,rare_maf_cutoff,
                                         QC_label,variant_type,geno_missing_imputation,
                                         Annotation_dir,Annotation_name_catalog,
                                         Use_annotation_weights,Annotation_name,silent){
  results <- try(Gene_Centric_Coding(chr=chr,gene_name=gene_name,genofile=genofile,obj_nullmodel=obj_nullmodel,rare_maf_cutoff=rare_maf_cutoff,
                                     QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                     Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                     Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,silent=silent),silent=TRUE)
  return(results)
}

results_coding <- list()
count <- 1
for(kk in sub_seq_id){
  print(kk)
  gene_name <- genes_info_chr[kk,1]
  results <- gene_centric_coding_dnanexus(genes_info_chr=genes_info_chr,gene_name = gene_name,chr=chr,genofile=genofile,obj_nullmodel=obj_nullmodel,rare_maf_cutoff=0.01,
                                          QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                          Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,
                                          Use_annotation_weights=Use_annotation_weights,Annotation_name=Annotation_name,silent=TRUE)
  
  results_coding[[count]] <- results
  count <- count + 1
}

seqClose(genofile)

rm(list=setdiff(ls(), c("results_coding", "output_file_name","arrayid","agds.path","trait"))); gc()
save(results_coding, file = paste0(output_file_name,"_",arrayid,".Rdata"))


Overwriting Coding_Analysis.R


In [8]:
%%writefile Coding_Analysis.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${arrayid} ${trait} ${Train_Null_Model} ${Annotation_name_catalog} ${GDS_File} ${OUTPUT_PATH}

Overwriting Coding_Analysis.sh


In [9]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(trait in c("BMI","Height")){
    for(arrayid in c(86,112,113,332)){
        if(arrayid < 41){
            chr <- 1
        }else if(arrayid < 66){
            chr <- 2
        }else if(arrayid < 87){
            chr <- 3
        }else if(arrayid < 102){
            chr <- 4
        }else if(arrayid < 120){
            chr <- 5
        }else if(arrayid < 141){
            chr <- 6
        }else if(arrayid < 159){
            chr <- 7
        }else if(arrayid < 173){
            chr <- 8
        }else if(arrayid < 189){
            chr <- 9
        }else if(arrayid < 204){
            chr <- 10
        }else if(arrayid < 230){
            chr <- 11
        }else if(arrayid < 250){
            chr <- 12
        }else if(arrayid < 257){
            chr <- 13
        }else if(arrayid < 269){
            chr <- 14
        }else if(arrayid < 281){
            chr <- 15
        }else if(arrayid < 298){
            chr <- 16
        }else if(arrayid < 321){
            chr <- 17
        }else if(arrayid < 327){
            chr <- 18
        }else if(arrayid < 355){
            chr <- 19
        }else if(arrayid < 366){
            chr <- 20
        }else if(arrayid < 371){
            chr <- 21
        }else{
            chr <- 22
        }
        
        tasks <- rbind(tasks, data.frame(
            '--env arrayid'=arrayid,
            '--env trait'=trait,
            '--input Train_Null_Model'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels/",trait,"_Train_Null_Model.RData"),
            '--input Annotation_name_catalog'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataGDS/acaf_threshold_v7/Annotation_name_catalog.csv",
            '--input GDS_File'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataAGDS/exome_v7.1/exome.chr",chr,".gds"),
            '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/Coding_Analysis.R",
            '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GeneCentricCoding",
            check.names = FALSE
        )) 
    }
}  
   
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [10]:
!Rscript score_task.R

In [11]:
!gsutil -m cp Coding_Analysis.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://Coding_Analysis.R [Content-Type=application/octet-stream]...
/ [1/1 files][129.1 KiB/129.1 KiB] 100% Done                                    
Operation completed over 1 objects/129.1 KiB.                                    


In [12]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 64 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Coding_Analysis.sh \
  --tasks score_task.txt

Job properties:
  job-id: coding-ana--williamsjacr--240805-152344-37
  job-name: coding-analysis
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/17061318534041850195
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/840692259032739568
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/9318378643191275524
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/15924022795533132610
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3860031643969366395
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/18348738718336721892
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10678717588606309768
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/9692262135783927507
Launched job

In [13]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'coding-ana--williamsjacr--240805-152344-37' --users 'williamsjacr' --status '*'

Job Name           Task  Status                 Last Update
---------------  ------  ---------------------  --------------
coding-analysis       3  Success                08-05 18:23:33
coding-analysis       3  worker was terminated  08-05 17:02:36
coding-analysis       2  Success                08-05 17:36:46
coding-analysis       3  worker was terminated  08-05 16:37:48
coding-analysis       4  Success                08-05 16:21:02
coding-analysis       6  Success                08-05 16:44:42
coding-analysis       8  Success                08-05 16:04:54
coding-analysis       7  Success                08-05 16:47:22
coding-analysis       6  worker was terminated  08-05 15:39:07
coding-analysis       5  Success                08-05 15:52:56
coding-analysis       4  worker was terminated  08-05 15:41:16
coding-analysis       3  worker was terminated  08-05 16:08:13
coding-analysis       2  worker was terminated  08-05 16:28:00
coding-analysis       1  Success                08-05 15:5

In [46]:
%%writefile Coding_Analysis_Summary.R
rm(list=ls())
gc()

library(data.table)

INPUT_PATH <- commandArgs(TRUE)[1]
print(INPUT_PATH)

OUTPUT_PATH <- commandArgs(TRUE)[2]
print(OUTPUT_PATH)

gene_centric_coding_jobs_num <- 379

trait <- "BMI"
i <- 1

for(trait in c("BMI","TC","HDL","LDL","logTG","Height")){
  gene_centric_results_name <- paste0(trait,"_AoU_WGS_Coding_Train")
  
  results_coding_genome <- NULL
  for (i in 1:gene_centric_coding_jobs_num){
    results <- get(load(paste0(INPUT_PATH,"/",gene_centric_results_name,"_",i,".Rdata")))
    results_coding_genome <- c(results_coding_genome, results)
  }
  
  combine_jake <- function(x){
    a <- data.frame(Gene = c(unlist(x$plof[,c("Gene name")]),unlist(x$plof_ds[,c("Gene name")]),unlist(x$missense[,c("Gene name")]),unlist(x$disruptive_missense[,c("Gene name")]),unlist(x$synonymous[,c("Gene name")])),
                    Chr = c(unlist(x$plof[,c("Chr")]),unlist(x$plof_ds[,c("Chr")]),unlist(x$missense[,c("Chr")]),unlist(x$disruptive_missense[,c("Chr")]),unlist(x$synonymous[,c("Chr")])),
                    Category = c(unlist(x$plof[,c("Category")]),unlist(x$plof_ds[,c("Category")]),unlist(x$missense[,c("Category")]),unlist(x$disruptive_missense[,c("Category")]),unlist(x$synonymous[,c("Category")])),
                    Number_SNV = c(unlist(x$plof[,c("#SNV")]),unlist(x$plof_ds[,c("#SNV")]),unlist(x$missense[,c("#SNV")]),unlist(x$disruptive_missense[,c("#SNV")]),unlist(x$synonymous[,c("#SNV")])),
                    Burden_1_1 = c(unlist(x$plof[,c("Burden(1,1)")]),unlist(x$plof_ds[,c("Burden(1,1)")]),unlist(x$missense[,c("Burden(1,1)")]),unlist(x$disruptive_missense[,c("Burden(1,1)")]),unlist(x$synonymous[,c("Burden(1,1)")])),
                    STAARB = c(unlist(x$plof[,c("STAAR-B(1,1)")]),unlist(x$plof_ds[,c("STAAR-B(1,1)")]),unlist(x$missense[,c("STAAR-B(1,1)")]),unlist(x$disruptive_missense[,c("STAAR-B(1,1)")]),unlist(x$synonymous[,c("STAAR-B(1,1)")])))
    return(a)
  }
  
  results_coding_genome <- lapply(results_coding_genome, combine_jake)
  results_coding_genome <- rbindlist(results_coding_genome)
  results_coding_genome$Number_SNV <- as.numeric(results_coding_genome$Number_SNV)
  results_coding_genome <- results_coding_genome[results_coding_genome$Number_SNV < 2000,]
  
  write.csv(results_coding_genome,file = paste0(OUTPUT_PATH,"/",trait,"_coding_sig.csv"),row.names = FALSE)
}

Overwriting Coding_Analysis_Summary.R


In [47]:
%%writefile Coding_Analysis_Summary.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${INPUT_PATH} ${OUTPUT_PATH}

Overwriting Coding_Analysis_Summary.sh


In [48]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)


tasks <- rbind(tasks, data.frame(
            '--input-recursive INPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GeneCentricCoding",
            '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/Coding_Analysis_Summary.R",
            '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GeneCentricCoding",
            check.names = FALSE
        ))

   
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [49]:
!Rscript score_task.R

In [50]:
!gsutil -m cp Coding_Analysis_Summary.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://Coding_Analysis_Summary.R [Content-Type=application/octet-stream]...
/ [1/1 files][  2.3 KiB/  2.3 KiB] 100% Done                                    
Operation completed over 1 objects/2.3 KiB.                                      


In [51]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Coding_Analysis_Summary.sh \
  --tasks score_task.txt

Job properties:
  job-id: coding-ana--williamsjacr--240805-203638-20
  job-name: coding-analysis-summary
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/18328105492588917735
Launched job-id: coding-ana--williamsjacr--240805-203638-20
1 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'coding-ana--williamsjacr--240805-203638-20' --users 'williamsjacr' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'coding-ana--williamsjacr--240805-203638-20' --users 'williamsjacr'
Waiting for job to complete...
Monitoring for failed tasks to retry...
*** This dsub process must continue running to retry failed tasks.


In [ ]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'coding-ana--williamsjacr--240805-194340-74' --users 'williamsjacr' --status '*'

In [71]:
%%writefile Single_RareVariant_PRS.R
rm(list = ls())
library(gdsfmt)
library(SeqArray)
library(SeqVarTools)
library(STAAR)
library(STAARpipeline)
library(STAARpipelineSummary)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(readr)
library(dplyr)
library(caret)
library(ranger)
library(SuperLearner)
library(dplyr)
library(boot)
library(stringr)
library(glmnet)

Gene_Centric_Coding_G_Star <- function(chr,gene_name,category=c("plof","plof_ds","missense","disruptive_missense","synonymous"),
                                       genofile,obj_nullmodel,rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                       QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                                       Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,silent=FALSE){
  ## evaluate choices
  category <- match.arg(category)
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  genes <- genes_info[genes_info[,2]==chr,]  

  phenotype.id <- obj_nullmodel$id_include
  ## get SNV id, position, REF, ALT (whole genome)
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant"){
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV"){
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel"){
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  position <- as.numeric(seqGetData(genofile, "position"))
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ### Gene
  kk <- which(genes[,1]==gene_name)
  
  sub_start_loc <- genes[kk,3]
  sub_end_loc <- genes[kk,4]
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## plof
  ## Gencode_Exonic
  GENCODE.EXONIC.Category  <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Gencode
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  ## Meta.SVM.Pred
  MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
  
  variant.id.gene <- seqGetData(genofile, "variant.id")  
  
  if(category == "plof"){
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")
    variant.id.gene <- variant.id.gene[lof.in.plof]
  }else if(category == "plof_ds"){
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
    variant.id.gene <- variant.id.gene[lof.in.plof]
  }else if(category == "missense"){
    lof.in.missense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")
    variant.id.gene <- variant.id.gene[lof.in.missense]
  }else if(category == "disruptive_missense"){
    lof.in.ds <- ((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
    variant.id.gene <- variant.id.gene[lof.in.ds]
  }else{
    lof.in.synonymous <- (GENCODE.EXONIC.Category=="synonymous SNV")
    variant.id.gene <- variant.id.gene[lof.in.synonymous]
  }
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  genotype <- Geno
  
  if(dim(genotype)[2] == 1){
    return(matrix(0,nrow = dim(genotype)[1],ncol = 1))
  }
  
  if(!is.null(attr(class(genotype), "package")) && attr(class(genotype), "package") == "Matrix"){
    genotype <- as.matrix(genotype)
  }
  genotype <- matrix_flip(genotype)
  MAF <- genotype$MAF
  RV_label <- as.vector((MAF<rare_maf_cutoff)&(MAF>0))
  Geno_rare <- genotype$Geno[,RV_label]
  G <- Geno_rare
  rm(Geno_rare)
  gc()
  
  if(is.null(dim(G))){
    G <- matrix(G,ncol = 1)
  }
  
  C <- G%*%matrix(1,nrow=ncol(G),ncol = 1)
  
  seqResetFilter(genofile)
  
  return(C)
}

trait <- commandArgs(TRUE)[1]
print(trait)

Train_Pvals <- commandArgs(TRUE)[2]
print(Train_Pvals)

Train_NullModel <- commandArgs(TRUE)[3]
print(Train_NullModel)

Tune_NullModel <- commandArgs(TRUE)[4]
print(Tune_NullModel)

Validation_NullModel <- commandArgs(TRUE)[5]
print(Validation_NullModel)

Annotation_name_catalog <- commandArgs(TRUE)[6]
print(Annotation_name_catalog)

INPUT_PATH <- commandArgs(TRUE)[7]
print(INPUT_PATH)

all_phenotypes_file <- commandArgs(TRUE)[8]
print(all_phenotypes_file)

all_train_file <- commandArgs(TRUE)[9]
print(all_train_file)

all_tune_file <- commandArgs(TRUE)[10]
print(all_tune_file)

all_validation_file <- commandArgs(TRUE)[11]
print(all_validation_file)

OUTPUT_PATH <- commandArgs(TRUE)[12]
print(OUTPUT_PATH)

Train_PVals_All <- read.csv(Train_Pvals)
Train_PVals_All <- Train_PVals_All[Train_PVals_All$STAARB <= 1e-03,]

## Null Model
obj_nullmodel_train <- get(load(Train_NullModel))
obj_nullmodel_tune <- get(load(Tune_NullModel))
obj_nullmodel_validation <- get(load(Validation_NullModel))

obj_nullmodel <- obj_nullmodel_train
obj_nullmodel$id_include <- c(obj_nullmodel_train$id_include,obj_nullmodel_tune$id_include,obj_nullmodel_validation$id_include)

## Parameter
QC_label <- "annotation/filter"
geno_missing_imputation <- "mean"
variant_type <- "SNV"

## Annotation_dir
Annotation_dir <- "annotation/info/FunctionalAnnotation"
## Annotation channel
Annotation_name_catalog <- read.csv(Annotation_name_catalog)

G_star_gene_centric_coding <- list()

for(i in 1:nrow(Train_PVals_All)){
  ## Chr
  chr <- Train_PVals_All$Chr[i]
  ## Gene name
  gene_name <- Train_PVals_All$Gene[i]
  ## Coding mask
  category <- Train_PVals_All$Category[i]
  
  ### gds file
  gds.path <- paste0(INPUT_PATH,"/exome.chr",chr,".gds")
  genofile <- seqOpen(gds.path)
  
  G_star_gene_centric_coding[[i]] <- Gene_Centric_Coding_G_Star(chr=chr,gene_name=gene_name,category=category ,
                                                                genofile,obj_nullmodel,rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                                                QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                                                Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,silent=FALSE) 
  seqClose(genofile) 
} 

G_star_gene_centric_coding <- do.call(cbind,G_star_gene_centric_coding)

col_remove <- apply(G_star_gene_centric_coding,2,function(x){sum(x != 0)}) > 10 & colSums(G_star_gene_centric_coding) > 10 
G_star_gene_centric_coding <- G_star_gene_centric_coding[,col_remove,drop = FALSE]

Train_PVals_All <- Train_PVals_All[col_remove,]

print(str(G_star_gene_centric_coding))

ids_gstar <- obj_nullmodel$id_include

G_star_gene_centric_coding_train <- G_star_gene_centric_coding[ids_gstar %in% obj_nullmodel_train$id_include,]
G_star_gene_centric_coding_tune <- G_star_gene_centric_coding[ids_gstar %in% obj_nullmodel_tune$id_include,]
G_star_gene_centric_coding_vad <- G_star_gene_centric_coding[ids_gstar %in% obj_nullmodel_validation$id_include,]

rm(G_star_gene_centric_coding)

X_train <- data.frame(IID = ids_gstar[ids_gstar %in% obj_nullmodel_train$id_include],G_star_gene_centric_coding_train)
pheno_train <- read.delim(all_train_file)
pheno_train <- inner_join(pheno_train,X_train)
print(str(pheno_train))
X_train <- as.matrix(pheno_train[,22:ncol(pheno_train),drop = FALSE])
model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_train)
y_train <- model.null$residual

X_tune <- data.frame(IID = ids_gstar[ids_gstar %in% obj_nullmodel_tune$id_include],G_star_gene_centric_coding_tune)
pheno_tune <- read.delim(all_tune_file)
pheno_tune <- inner_join(pheno_tune,X_tune)
X_tune <- as.matrix(pheno_tune[,22:ncol(pheno_tune),drop = FALSE])
model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_tune)
y_tune <- model.null$residual

X_valid <- data.frame(IID = ids_gstar[ids_gstar %in% obj_nullmodel_validation$id_include],G_star_gene_centric_coding_vad)
pheno_valid <- read.delim(all_validation_file)
pheno_valid <- inner_join(pheno_valid,X_valid)
X_valid <- as.matrix(pheno_valid[,22:ncol(pheno_valid),drop = FALSE])
model.null <- lm(as.formula(paste0(trait,"~age+age2+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")),data=pheno_valid)
y_valid <- model.null$residual

lasso_train <- glmnet(X_train,y_train,family = "gaussian",alpha = 1)
ridge_train <- glmnet(X_train,y_train,family = "gaussian",alpha = 0)
lm_train <- lm.fit(cbind(1,X_train),y_train)
lm_train$coefficients[is.na(lm_train$coefficients)] <- 0

beta_matrix <- cbind(as.matrix(lasso_train$beta),as.matrix(ridge_train$beta),lm_train$coefficients[-1])
beta_matrix <- as.data.frame(beta_matrix)
colnames(beta_matrix) <- c(paste0("lasso_prs",1:ncol(as.matrix(lasso_train$beta))),paste0("ridge_prs",1:ncol(as.matrix(ridge_train$beta))),paste0("lm_prs",1))

beta_matrix <- cbind(Train_PVals_All[,c(1:4)],beta_matrix)


lasso_prs_tune <- predict(lasso_train,X_tune)
ridge_prs_tune <- predict(ridge_train,X_tune)
lm_prs_tune <- as.numeric(cbind(1,X_tune)%*%matrix(lm_train$coefficients,ncol = 1))

lasso_prs_vad <- predict(lasso_train,X_valid)
ridge_prs_vad <- predict(ridge_train,X_valid)
lm_prs_vad <- as.numeric(cbind(1,X_valid)%*%matrix(lm_train$coefficients,ncol = 1))






lasso_tune_dat <- data.frame(y = y_tune,lasso_prs_tune)
colnames(lasso_tune_dat) <- c("y",paste0("lasso_prs",1:(ncol(lasso_tune_dat) - 1)))
lasso_valid_dat <- data.frame(y = y_valid,lasso_prs_vad)
colnames(lasso_valid_dat) <- c("y",paste0("lasso_prs",1:(ncol(lasso_valid_dat) - 1)))


ridge_tune_dat <- data.frame(y = y_tune,ridge_prs_tune)
colnames(ridge_tune_dat) <- c("y",paste0("ridge_prs",1:(ncol(ridge_tune_dat) - 1)))
ridge_valid_dat <- data.frame(y = y_valid,ridge_prs_vad)
colnames(ridge_valid_dat) <- c("y",paste0("ridge_prs",1:(ncol(ridge_valid_dat) - 1)))

lm_tune_dat <- data.frame(y = y_tune,lm_prs_tune)
colnames(lm_tune_dat) <- c("y",paste0("lm_prs",1:(ncol(lm_tune_dat) - 1)))
lm_valid_dat <- data.frame(y = y_valid,lm_prs_vad)
colnames(lm_valid_dat) <- c("y",paste0("lm_prs",1:(ncol(lm_valid_dat) - 1)))





all_prs_tune <- cbind(lasso_prs_tune,ridge_prs_tune,lm_prs_tune)
colnames(all_prs_tune) <- c(paste0("lasso_prs",1:ncol(lasso_prs_tune)),paste0("ridge_prs",1:ncol(ridge_prs_tune)),"lm_prs1")
all_prs_valid <- cbind(lasso_prs_vad,ridge_prs_vad,lm_prs_vad)
colnames(all_prs_valid) <- c(paste0("lasso_prs",1:ncol(lasso_prs_vad)),paste0("ridge_prs",1:ncol(ridge_prs_vad)),"lm_prs1")

all_tune <- data.frame(y = y_tune,all_prs_tune)
all_valid <- data.frame(y = y_valid,all_prs_valid)

best_r2 <- vector()
count <- 1
for(i in colnames(all_prs_tune)){
  tmp <- data.frame(y = all_tune$y,x = all_prs_tune[,i])
  best_r2[count] <- summary(lm(y~x,data = tmp))$r.squared
  count <- count + 1
}

best_thresh <- colnames(all_prs_tune)[which.max(best_r2)]
r2_bestoverall_tune <- best_r2[which.max(best_r2)]


all_prs_tune <- as.data.frame(all_prs_tune)
all_prs_valid <- as.data.frame(all_prs_valid)

mtx <- cor(all_prs_tune)
drop <- names(all_prs_tune)[apply(mtx,2,function(x){sum(is.na(x))}) == (nrow(mtx) - 1)]

all_prs_tune <- dplyr::select(all_prs_tune, -c(drop))
all_prs_valid <- dplyr::select(all_prs_valid, -c(drop))

mtx <- cor(all_prs_tune)
drop <- findCorrelation(mtx,cutoff=0.98)
drop <- names(all_prs_tune)[drop]

all_prs_tune <- dplyr::select(all_prs_tune, -c(drop))
all_prs_valid <- dplyr::select(all_prs_valid, -c(drop))

drop <- findLinearCombos(all_prs_tune)$remove
drop <- names(data.frame(all_prs_tune))[drop]

all_prs_tune <- dplyr::select(all_prs_tune, -c(drop))
all_prs_valid <- dplyr::select(all_prs_valid, -c(drop))



SL.library <- c(
  "SL.glmnet",
  "SL.ridge",
  "SL.glm",
  "SL.mean"
)

full_superlearner <- SuperLearner(Y = y_tune, X = all_prs_tune, family = gaussian(),
                                  # For a real analysis we would use V = 10.
                                  # V = 3,
                                  SL.library = SL.library,cvControl = list(V = 20))
cvsl <- CV.SuperLearner(Y = y_tune, X = all_prs_tune, family = gaussian(),
                        # For a real analysis we would use V = 10.
                        # V = 3,
                        SL.library = SL.library,cvControl = list(V = 20))

best_algorithm <- summary(cvsl)$Table$Algorithm[which.min(summary(cvsl)$Table$Ave)]

a_tune <- predict(full_superlearner, all_prs_tune, onlySL = FALSE)
a_vad <- predict(full_superlearner, all_prs_valid, onlySL = FALSE)

prs_best_tune_sl <- a_tune$pred
prs_best_tune_glmnet <- a_tune$library.predict[,1]
prs_best_tune_ridge <- a_tune$library.predict[,2]
prs_best_tune_glm <- a_tune$library.predict[,3]
prs_best_tune_mean <- a_tune$library.predict[,4]

prs_best_vad_sl <- a_vad$pred
prs_best_vad_glmnet <- a_vad$library.predict[,1]
prs_best_vad_ridge <- a_vad$library.predict[,2]
prs_best_vad_glm <- a_vad$library.predict[,3]
prs_best_vad_mean <- a_vad$library.predict[,4]

if(best_algorithm == "SL.glmnet_All"){
  #final
  prs_best_tune <- prs_best_tune_glmnet
  prs_best_vad <- prs_best_vad_glmnet
}else if(best_algorithm == "SL.ridge_All"){
  #final
  prs_best_tune <- prs_best_tune_ridge
  prs_best_vad <- prs_best_vad_ridge
}else if(best_algorithm == "SL.glm_All"){
  #final
  prs_best_tune <- prs_best_tune_glm
  prs_best_vad <- prs_best_vad_glm
}else if(best_algorithm == "SL.mean_All"){
  #final
  prs_best_tune <- prs_best_tune_mean
  prs_best_vad <- prs_best_vad_mean
} else {
  prs_best_tune <- prs_best_tune_sl
  prs_best_vad <- prs_best_vad_sl
}

tune_dat_sl_R2 <- data.frame(y = y_tune,x = prs_best_tune)
valid_dat_sl_R2 <- data.frame(y = y_valid,x = prs_best_vad)

r2_sl_tune <- summary(lm(y~x,data = tune_dat_sl_R2))$r.squared

if(r2_sl_tune < r2_bestoverall_tune){
  prs_best_tune_sl <- all_tune[,best_thresh]
  prs_best_vad_sl <- all_valid[,best_thresh] 
  
  tune_dat_sl_R2 <- data.frame(y = y_tune,x = prs_best_tune_sl)
  valid_dat_sl_R2 <- data.frame(y = y_valid,x = prs_best_vad_sl)
}





if(r2_sl_tune < r2_bestoverall_tune){
  beta_matrix$Beta_Final <- beta_matrix[,best_thresh]
  write.csv(beta_matrix[,c("Gene","Chr","Category","Number_SNV","Beta_Final")],file = paste0(OUTPUT_PATH,"/",trait,"_final_coef.csv"),row.names = FALSE)
}else{
  if(best_algorithm == "SL.glmnet_All"){
    beta_full_superlearner <- data.frame(Coef = row.names(coef(full_superlearner$fitLibrary$SL.glmnet_All$object)),Beta = as.numeric(coef(full_superlearner$fitLibrary$SL.glmnet_All$object)))
  }else if(best_algorithm == "SL.ridge_All"){
    beta_full_superlearner <- data.frame(Coef = row.names(full_superlearner$fitLibrary$SL.ridge_All$bestCoef),Beta = as.numeric(full_superlearner$fitLibrary$SL.ridge_All$bestCoef[,1]))
  }else if(best_algorithm == "SL.glm_All"){
    beta_full_superlearner <- data.frame(Coef = names(coef(full_superlearner$fitLibrary$SL.glm_All$object)),Beta = as.numeric(coef(full_superlearner$fitLibrary$SL.glm_All$object)))
  }else{
    beta_lasso <- data.frame(Coef = row.names(coef(full_superlearner$fitLibrary$SL.glmnet_All$object)),Beta = as.numeric(coef(full_superlearner$fitLibrary$SL.glmnet_All$object)))
    beta_ridge <- data.frame(Coef = row.names(full_superlearner$fitLibrary$SL.ridge_All$bestCoef),Beta = as.numeric(full_superlearner$fitLibrary$SL.ridge_All$bestCoef[,1]))
    beta_glm <- data.frame(Coef = names(coef(full_superlearner$fitLibrary$SL.glm_All$object)),Beta = as.numeric(coef(full_superlearner$fitLibrary$SL.glm_All$object)))
    beta_mean <- data.frame(Coef = names(coef(full_superlearner$fitLibrary$SL.glm_All$object)), Beta = 0)
    beta_mean$Beta[1] <- full_superlearner$fitLibrary$SL.mean_All$object
    
    beta_full_superlearner <- data.frame(Coef = names(coef(full_superlearner$fitLibrary$SL.glm_All$object)), Beta = 0)
    beta_full_superlearner$Beta <- as.numeric(full_superlearner$coef[names(full_superlearner$coef) == "SL.glmnet_All"]) * beta_lasso$Beta + 
      as.numeric(full_superlearner$coef[names(full_superlearner$coef) == "SL.ridge_All"]) * beta_ridge$Beta + 
      as.numeric(full_superlearner$coef[names(full_superlearner$coef) == "SL.glm_All"]) * beta_glm$Beta + 
      as.numeric(full_superlearner$coef[names(full_superlearner$coef) == "SL.mean_All"]) * beta_mean$Beta 
  }
  
  beta_full_superlearner <- beta_full_superlearner[-1,]
  
  beta_matrix$Beta_Final <- 0
  for(i in 1:nrow(beta_full_superlearner)){
    beta_matrix$Beta_Final <- beta_matrix[,beta_full_superlearner$Coef[i]]*beta_full_superlearner$Beta[i]
  }
  write.csv(beta_matrix[,c("Gene","Chr","Category","Number_SNV","Beta_Final")],file = paste0(OUTPUT_PATH,"/",trait,"_final_coef.csv"),row.names = FALSE)
}





all_phenotypes <- read.csv(all_phenotypes_file)

print(str(all_phenotypes))

RV_PRS <- data.frame(IID = pheno_valid$IID,Y = valid_dat_sl_R2[,1],RV_PRS = valid_dat_sl_R2[,2],pheno_valid[,c("PC1","PC2","PC3","PC4","PC5")])

print(str(RV_PRS))

write.csv(RV_PRS,file = paste0(OUTPUT_PATH,"/",trait,"_BestPRS.csv"),row.names = FALSE)

RV_PRS_raw <- RV_PRS
RV_PRS_adjusted <- RV_PRS


tmp <- data.frame(y = RV_PRS_adjusted[,"RV_PRS"],RV_PRS_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
mod <- lm(y~.,data = tmp)
print(summary(mod))
R <- mod$residuals
tmp <- data.frame(y = R^2,RV_PRS_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
mod <- lm(y~.,data = tmp)
print(summary(mod))

y_hat <- predict(mod,tmp)
if(sum(y_hat < 0) > 0){
  mod <- lm(y~1,data = tmp)
  y_hat <- predict(mod,tmp)
}
if(sum(sqrt(y_hat)) == 0){
  RV_PRS_adjusted[,"RV_PRS"] <- 0
}else{
  RV_PRS_adjusted[,"RV_PRS"] <- R/sqrt(y_hat)
}


RV_PRS_raw_EUR <- RV_PRS_raw[RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
RV_PRS_raw_SAS <- RV_PRS_raw[RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
RV_PRS_raw_AMR <- RV_PRS_raw[RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
RV_PRS_raw_AFR <- RV_PRS_raw[RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
RV_PRS_raw_EAS <- RV_PRS_raw[RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
RV_PRS_raw_MID <- RV_PRS_raw[RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]

str(RV_PRS_raw_EUR)
str(RV_PRS_raw_SAS)
str(RV_PRS_raw_AMR)
str(RV_PRS_raw_AFR)
str(RV_PRS_raw_EAS)
str(RV_PRS_raw_MID)

RV_PRS_adjusted_EUR <- RV_PRS_adjusted[RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
RV_PRS_adjusted_SAS <- RV_PRS_adjusted[RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
RV_PRS_adjusted_AMR <- RV_PRS_adjusted[RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
RV_PRS_adjusted_AFR <- RV_PRS_adjusted[RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
RV_PRS_adjusted_EAS <- RV_PRS_adjusted[RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
RV_PRS_adjusted_MID <- RV_PRS_adjusted[RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]

str(RV_PRS_adjusted_EUR)
str(RV_PRS_adjusted_SAS)
str(RV_PRS_adjusted_AMR)
str(RV_PRS_adjusted_AFR)
str(RV_PRS_adjusted_EAS)
str(RV_PRS_adjusted_MID)

RV_PRS_raw_EUR$Y <- scale(RV_PRS_raw_EUR$Y)
RV_PRS_raw_SAS$Y <- scale(RV_PRS_raw_SAS$Y)
RV_PRS_raw_AMR$Y <- scale(RV_PRS_raw_AMR$Y)
RV_PRS_raw_AFR$Y <- scale(RV_PRS_raw_AFR$Y)
RV_PRS_raw_EAS$Y <- scale(RV_PRS_raw_EAS$Y)
RV_PRS_raw_MID$Y <- scale(RV_PRS_raw_MID$Y)

RV_PRS_adjusted_EUR$Y <- scale(RV_PRS_adjusted_EUR$Y)
RV_PRS_adjusted_SAS$Y <- scale(RV_PRS_adjusted_SAS$Y)
RV_PRS_adjusted_AMR$Y <- scale(RV_PRS_adjusted_AMR$Y)
RV_PRS_adjusted_AFR$Y <- scale(RV_PRS_adjusted_AFR$Y)
RV_PRS_adjusted_EAS$Y <- scale(RV_PRS_adjusted_EAS$Y)
RV_PRS_adjusted_MID$Y <- scale(RV_PRS_adjusted_MID$Y)

RV_PRS_raw_EUR$RV_PRS <- scale(RV_PRS_raw_EUR$RV_PRS)
RV_PRS_raw_SAS$RV_PRS <- scale(RV_PRS_raw_SAS$RV_PRS)
RV_PRS_raw_AMR$RV_PRS <- scale(RV_PRS_raw_AMR$RV_PRS)
RV_PRS_raw_AFR$RV_PRS <- scale(RV_PRS_raw_AFR$RV_PRS)
RV_PRS_raw_EAS$RV_PRS <- scale(RV_PRS_raw_EAS$RV_PRS)
RV_PRS_raw_MID$RV_PRS <- scale(RV_PRS_raw_MID$RV_PRS)

best_beta_raw_RV_EUR <- coef(lm(Y~RV_PRS,data = RV_PRS_raw_EUR))[2]
se_beta_raw_RV_EUR <- summary(lm(Y~RV_PRS,data = RV_PRS_raw_EUR))$coefficients[2,2]
best_beta_raw_RV_SAS <- coef(lm(Y~RV_PRS,data = RV_PRS_raw_SAS))[2]
se_beta_raw_RV_SAS <- summary(lm(Y~RV_PRS,data = RV_PRS_raw_SAS))$coefficients[2,2]
best_beta_raw_RV_AMR <- coef(lm(Y~RV_PRS,data = RV_PRS_raw_AMR))[2]
se_beta_raw_RV_AMR <- summary(lm(Y~RV_PRS,data = RV_PRS_raw_AMR))$coefficients[2,2]
best_beta_raw_RV_AFR <- coef(lm(Y~RV_PRS,data = RV_PRS_raw_AFR))[2]
se_beta_raw_RV_AFR <- summary(lm(Y~RV_PRS,data = RV_PRS_raw_AFR))$coefficients[2,2]
best_beta_raw_RV_EAS <- coef(lm(Y~RV_PRS,data = RV_PRS_raw_EAS))[2]
se_beta_raw_RV_EAS <- summary(lm(Y~RV_PRS,data = RV_PRS_raw_EAS))$coefficients[2,2]
best_beta_raw_RV_MID <- coef(lm(Y~RV_PRS,data = RV_PRS_raw_MID))[2]
se_beta_raw_RV_MID <- summary(lm(Y~RV_PRS,data = RV_PRS_raw_MID))$coefficients[2,2]

best_beta_adjusted_RV_EUR <- coef(lm(Y~RV_PRS,data = RV_PRS_adjusted_EUR))[2]
se_beta_adjusted_RV_EUR <- summary(lm(Y~RV_PRS,data = RV_PRS_adjusted_EUR))$coefficients[2,2]
best_beta_adjusted_RV_SAS <- coef(lm(Y~RV_PRS,data = RV_PRS_adjusted_SAS))[2]
se_beta_adjusted_RV_SAS <- summary(lm(Y~RV_PRS,data = RV_PRS_adjusted_SAS))$coefficients[2,2]
best_beta_adjusted_RV_AMR <- coef(lm(Y~RV_PRS,data = RV_PRS_adjusted_AMR))[2]
se_beta_adjusted_RV_AMR <- summary(lm(Y~RV_PRS,data = RV_PRS_adjusted_AMR))$coefficients[2,2]
best_beta_adjusted_RV_AFR <- coef(lm(Y~RV_PRS,data = RV_PRS_adjusted_AFR))[2]
se_beta_adjusted_RV_AFR <- summary(lm(Y~RV_PRS,data = RV_PRS_adjusted_AFR))$coefficients[2,2]
best_beta_adjusted_RV_EAS <- coef(lm(Y~RV_PRS,data = RV_PRS_adjusted_EAS))[2]
se_beta_adjusted_RV_EAS <- summary(lm(Y~RV_PRS,data = RV_PRS_adjusted_EAS))$coefficients[2,2]
best_beta_adjusted_RV_MID <- coef(lm(Y~RV_PRS,data = RV_PRS_adjusted_MID))[2]
se_beta_adjusted_RV_MID <- summary(lm(Y~RV_PRS,data = RV_PRS_adjusted_MID))$coefficients[2,2]

RV_PRS_Results <- data.frame(trait = trait,ancestry = c("EUR","SAS","AMR","AFR","EAS","MID"), 
                             beta_raw = c(best_beta_raw_RV_EUR,best_beta_raw_RV_SAS,best_beta_raw_RV_AMR,best_beta_raw_RV_AFR,best_beta_raw_RV_EAS,best_beta_raw_RV_MID), 
                             se_raw = c(se_beta_raw_RV_EUR,se_beta_raw_RV_SAS,se_beta_raw_RV_AMR,se_beta_raw_RV_AFR,se_beta_raw_RV_EAS,se_beta_raw_RV_MID), 
                             beta_adjusted = c(best_beta_adjusted_RV_EUR,best_beta_adjusted_RV_SAS,best_beta_adjusted_RV_AMR,best_beta_adjusted_RV_AFR,best_beta_adjusted_RV_EAS,best_beta_adjusted_RV_MID), 
                             se_adjusted = c(se_beta_adjusted_RV_EUR,se_beta_adjusted_RV_SAS,se_beta_adjusted_RV_AMR,se_beta_adjusted_RV_AFR,se_beta_adjusted_RV_EAS,se_beta_adjusted_RV_MID))

write.csv(RV_PRS_Results,file = paste0(OUTPUT_PATH,"/",trait,"Best_Betas.csv"),row.names = FALSE)


Overwriting Single_RareVariant_PRS.R


In [72]:
%%writefile Single_RareVariant_PRS.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${trait} ${Train_Pvals} ${Train_NullModel} ${Tune_NullModel} ${Validation_NullModel} ${Annotation_name_catalog} ${INPUT_PATH} ${all_phenotypes_file} ${all_train_file} ${all_tune_file} ${all_validation_file} ${OUTPUT_PATH}

Overwriting Single_RareVariant_PRS.sh


In [73]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(trait in c("BMI","LDL","HDL","logTG","TC","Height")){
 tasks <- rbind(tasks, data.frame(
            '--env trait'=trait,
            '--input-recursive INPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataAGDS/exome_v7.1",
            '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/Single_RareVariant_PRS.R",
            '--input Train_Pvals'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GeneCentricCoding/",trait,"_coding_sig.csv"),
            '--input Train_NullModel'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels/",trait,"_Train_Null_Model.RData"),
            '--input Tune_NullModel'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels/",trait,"_Tune_Null_Model.RData"),
            '--input Validation_NullModel'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels/",trait,"_Validation_Null_Model.RData"),
            '--input Annotation_name_catalog'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataGDS/acaf_threshold_v7/Annotation_name_catalog.csv",
            '--input all_phenotypes_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
            '--input all_train_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Train.txt",
            '--input all_tune_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Tune.txt",
            '--input all_validation_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/All_Validation.txt",
            '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/RareVariant_PRS",
            check.names = FALSE
        ))   
}

   
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [74]:
!Rscript score_task.R

In [75]:
!gsutil -m cp Single_RareVariant_PRS.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://Single_RareVariant_PRS.R [Content-Type=application/octet-stream]...
/ [1/1 files][ 23.6 KiB/ 23.6 KiB] 100% Done                                    
Operation completed over 1 objects/23.6 KiB.                                     


In [76]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Single_RareVariant_PRS.sh \
  --tasks score_task.txt

Job properties:
  job-id: single-rar--williamsjacr--240807-002443-67
  job-name: single-rarevariant-prs
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/5211521007745152674
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3582161128721624158
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/12226037870621958711
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3508336469533073098
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/10313404690612761132
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/14000302688532583800
Launched job-id: single-rar--williamsjacr--240807-002443-67
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'single-rar--williamsjacr--240

In [77]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'single-rar--williamsjacr--240807-002443-67' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
single-rarev...       6  Pending   08-07 00:24:44
single-rarev...       5  Pending   08-07 00:24:44
single-rarev...       4  Pending   08-07 00:24:44
single-rarev...       3  Pending   08-07 00:24:43
single-rarev...       2  Pending   08-07 00:24:43
single-rarev...       1  Pending   08-07 00:24:43



In [7]:
%%writefile RareVariantCoefficients.R
rm(list = ls())
library(gdsfmt)
library(SeqArray)
library(SeqVarTools)
library(STAAR)
library(STAARpipeline)
library(STAARpipelineSummary)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(readr)
library(dplyr)
library(caret)
library(ranger)
library(SuperLearner)
library(dplyr)
library(boot)
library(stringr)
library(glmnet)

Gene_Centric_Coding_G_Star <- function(chr,gene_name,category=c("plof","plof_ds","missense","disruptive_missense","synonymous"),
                                       genofile,obj_nullmodel,rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                       QC_label="annotation/filter",variant_type=c("SNV","Indel","variant"),geno_missing_imputation=c("mean","minor"),
                                       Annotation_dir="annotation/info/FunctionalAnnotation",Annotation_name_catalog,silent=FALSE){
  ## evaluate choices
  category <- match.arg(category)
  variant_type <- match.arg(variant_type)
  geno_missing_imputation <- match.arg(geno_missing_imputation)
  
  genes <- genes_info[genes_info[,2]==chr,]  
  
  phenotype.id <- obj_nullmodel$id_include
  ## get SNV id, position, REF, ALT (whole genome)
  filter <- seqGetData(genofile, QC_label)
  if(variant_type=="variant"){
    SNVlist <- filter == "PASS"
  }
  
  if(variant_type=="SNV"){
    SNVlist <- (filter == "PASS") & isSNV(genofile)
  }
  
  if(variant_type=="Indel"){
    SNVlist <- (filter == "PASS") & (!isSNV(genofile))
  }
  
  position <- as.numeric(seqGetData(genofile, "position"))
  variant.id <- seqGetData(genofile, "variant.id")
  
  rm(filter)
  gc()
  
  ### Gene
  kk <- which(genes[,1]==gene_name)
  
  sub_start_loc <- genes[kk,3]
  sub_end_loc <- genes[kk,4]
  
  is.in <- (SNVlist)&(position>=sub_start_loc)&(position<=sub_end_loc)
  variant.id.gene <- variant.id[is.in]
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## plof
  ## Gencode_Exonic
  GENCODE.EXONIC.Category  <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.EXONIC.Category")]))
  ## Gencode
  GENCODE.Category <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="GENCODE.Category")]))
  ## Meta.SVM.Pred
  MetaSVM_pred <- seqGetData(genofile, paste0(Annotation_dir,Annotation_name_catalog$dir[which(Annotation_name_catalog$name=="MetaSVM")]))
  
  variant.id.gene <- seqGetData(genofile, "variant.id")  
  
  if(category == "plof"){
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")
    variant.id.gene <- variant.id.gene[lof.in.plof]
  }else if(category == "plof_ds"){
    lof.in.plof <- (GENCODE.EXONIC.Category=="stopgain")|(GENCODE.EXONIC.Category=="stoploss")|(GENCODE.Category=="splicing")|(GENCODE.Category=="exonic;splicing")|(GENCODE.Category=="ncRNA_splicing")|(GENCODE.Category=="ncRNA_exonic;splicing")|((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
    variant.id.gene <- variant.id.gene[lof.in.plof]
  }else if(category == "missense"){
    lof.in.missense <- (GENCODE.EXONIC.Category=="nonsynonymous SNV")
    variant.id.gene <- variant.id.gene[lof.in.missense]
  }else if(category == "disruptive_missense"){
    lof.in.ds <- ((GENCODE.EXONIC.Category=="nonsynonymous SNV")&(MetaSVM_pred=="D"))
    variant.id.gene <- variant.id.gene[lof.in.ds]
  }else{
    lof.in.synonymous <- (GENCODE.EXONIC.Category=="synonymous SNV")
    variant.id.gene <- variant.id.gene[lof.in.synonymous]
  }
  
  seqSetFilter(genofile,variant.id=variant.id.gene,sample.id=phenotype.id)
  
  ## genotype id
  id.genotype <- seqGetData(genofile,"sample.id")
  # id.genotype.match <- rep(0,length(id.genotype))
  
  id.genotype.merge <- data.frame(id.genotype,index=seq(1,length(id.genotype)))
  phenotype.id.merge <- data.frame(phenotype.id)
  phenotype.id.merge <- dplyr::left_join(phenotype.id.merge,id.genotype.merge,by=c("phenotype.id"="id.genotype"))
  id.genotype.match <- phenotype.id.merge$index
  
  ## Genotype
  Geno <- seqGetData(genofile, "$dosage")
  Geno <- Geno[id.genotype.match,,drop=FALSE]
  
  ## impute missing
  if(!is.null(dim(Geno)))
  {
    if(dim(Geno)[2]>0)
    {
      if(geno_missing_imputation=="mean")
      {
        Geno <- matrix_flip_mean(Geno)$Geno
      }
      if(geno_missing_imputation=="minor")
      {
        Geno <- matrix_flip_minor(Geno)$Geno
      }
    }
  }
  
  genotype <- Geno
  
  if(dim(genotype)[2] == 1){
    return(matrix(0,nrow = dim(genotype)[1],ncol = 1))
  }
  
  if(!is.null(attr(class(genotype), "package")) && attr(class(genotype), "package") == "Matrix"){
    genotype <- as.matrix(genotype)
  }
  genotype <- matrix_flip(genotype)
  MAF <- genotype$MAF
  RV_label <- as.vector((MAF<rare_maf_cutoff)&(MAF>0))
  Geno_rare <- genotype$Geno[,RV_label]
  G <- Geno_rare
  rm(Geno_rare)
  gc()
  
  if(is.null(dim(G))){
    G <- matrix(G,ncol = 1)
  }
  
  C <- G%*%matrix(1,nrow=ncol(G),ncol = 1)
  
  seqResetFilter(genofile)
  
  return(C)
}

trait <- commandArgs(TRUE)[1]
print(trait)

Train_Pvals <- commandArgs(TRUE)[2]
print(Train_Pvals)

Train_NullModel <- commandArgs(TRUE)[3]
print(Train_NullModel)

Tune_NullModel <- commandArgs(TRUE)[4]
print(Tune_NullModel)

Validation_NullModel <- commandArgs(TRUE)[5]
print(Validation_NullModel)

Annotation_name_catalog <- commandArgs(TRUE)[6]
print(Annotation_name_catalog)

INPUT_PATH <- commandArgs(TRUE)[7]
print(INPUT_PATH)

RV_PRS <- commandArgs(TRUE)[8]
print(RV_PRS)

OUTPUT_PATH <- commandArgs(TRUE)[9]
print(OUTPUT_PATH)

Train_PVals_All <- read.csv(Train_Pvals)
Train_PVals_All <- Train_PVals_All[Train_PVals_All$STAARB <= 1e-03,]

## Null Model
obj_nullmodel_train <- get(load(Train_NullModel))
obj_nullmodel_tune <- get(load(Tune_NullModel))
obj_nullmodel_validation <- get(load(Validation_NullModel))

obj_nullmodel <- obj_nullmodel_train
obj_nullmodel$id_include <- c(obj_nullmodel_train$id_include,obj_nullmodel_tune$id_include,obj_nullmodel_validation$id_include)

## Parameter
QC_label <- "annotation/filter"
geno_missing_imputation <- "mean"
variant_type <- "SNV"

## Annotation_dir
Annotation_dir <- "annotation/info/FunctionalAnnotation"
## Annotation channel
Annotation_name_catalog <- read.csv(Annotation_name_catalog)

G_star_gene_centric_coding <- list()

for(i in 1:nrow(Train_PVals_All)){
  ## Chr
  chr <- Train_PVals_All$Chr[i]
  ## Gene name
  gene_name <- Train_PVals_All$Gene[i]
  ## Coding mask
  category <- Train_PVals_All$Category[i]
  
  ### gds file
  gds.path <- paste0(INPUT_PATH,"/exome.chr",chr,".gds")
  genofile <- seqOpen(gds.path)
  
  G_star_gene_centric_coding[[i]] <- Gene_Centric_Coding_G_Star(chr=chr,gene_name=gene_name,category=category ,
                                                                genofile,obj_nullmodel,rare_maf_cutoff=0.01,rv_num_cutoff=2,
                                                                QC_label=QC_label,variant_type=variant_type,geno_missing_imputation=geno_missing_imputation,
                                                                Annotation_dir=Annotation_dir,Annotation_name_catalog=Annotation_name_catalog,silent=FALSE) 
  seqClose(genofile) 
} 

G_star_gene_centric_coding <- do.call(cbind,G_star_gene_centric_coding)

col_remove <- apply(G_star_gene_centric_coding,2,function(x){sum(x != 0)}) > 10 & colSums(G_star_gene_centric_coding) > 10 
G_star_gene_centric_coding <- G_star_gene_centric_coding[,col_remove,drop = FALSE]

Train_PVals_All <- Train_PVals_All[col_remove,]

print(str(G_star_gene_centric_coding))

ids_gstar <- obj_nullmodel$id_include

G_star_gene_centric_coding_train <- G_star_gene_centric_coding[ids_gstar %in% obj_nullmodel_train$id_include,]
G_star_gene_centric_coding_tune <- G_star_gene_centric_coding[ids_gstar %in% obj_nullmodel_tune$id_include,]
G_star_gene_centric_coding_vad <- G_star_gene_centric_coding[ids_gstar %in% obj_nullmodel_validation$id_include,]

rm(G_star_gene_centric_coding)



X_valid <- data.frame(IID = ids_gstar[ids_gstar %in% obj_nullmodel_validation$id_include],G_star_gene_centric_coding_vad)
RV_PRS <- read.csv(RV_PRS)
tmp <- inner_join(RV_PRS[,c("IID","RV_PRS")],X_valid)
tmp <- subset(tmp,select = -c(IID))
Train_PVals_All$Beta <- coef(lm(RV_PRS~.,tmp))[-1]
Train_PVals_All$Beta[is.na(Train_PVals_All$Beta)] <- 0
Train_PVals_All$Beta[abs(Train_PVals_All$Beta) < 1e-10] <- 0

write.csv(Train_PVals_All,file = paste0(OUTPUT_PATH,"/",trait,"_final_coef.csv"),row.names = FALSE)

Writing RareVariantCoefficients.R


In [8]:
%%writefile RareVariantCoefficients.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${trait} ${Train_Pvals} ${Train_NullModel} ${Tune_NullModel} ${Validation_NullModel} ${Annotation_name_catalog} ${INPUT_PATH} ${RV_PRS} ${OUTPUT_PATH}

Writing RareVariantCoefficients.sh


In [19]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(trait in c("BMI","LDL","HDL","logTG","TC","Height")){
 tasks <- rbind(tasks, data.frame(
            '--env trait'=trait,
            '--input-recursive INPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataAGDS/exome_v7.1",
            '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/RareVariantCoefficients.R",
            '--input Train_Pvals'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GeneCentricCoding/",trait,"_coding_sig.csv"),
            '--input Train_NullModel'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels/",trait,"_Train_Null_Model.RData"),
            '--input Tune_NullModel'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels/",trait,"_Tune_Null_Model.RData"),
            '--input Validation_NullModel'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels/",trait,"_Validation_Null_Model.RData"),
            '--input Annotation_name_catalog'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/dataGDS/acaf_threshold_v7/Annotation_name_catalog.csv",
            '--input RV_PRS'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/RareVariant_PRS/",trait,"_BestPRS.csv"),
            '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/RareVariant_PRS",
            check.names = FALSE
        ))   
}

   
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [20]:
!Rscript score_task.R

In [21]:
!gsutil -m cp RareVariantCoefficients.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://RareVariantCoefficients.R [Content-Type=application/octet-stream]...
/ [1/1 files][  8.4 KiB/  8.4 KiB] 100% Done                                    
Operation completed over 1 objects/8.4 KiB.                                      


In [22]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script RareVariantCoefficients.sh \
  --tasks score_task.txt

Job properties:
  job-id: rarevarian--williamsjacr--240902-144749-64
  job-name: rarevariantcoefficients
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/313460965501063702
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/1070285458177013171
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/8852094039577346100
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/3294359260856517488
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/1907302209952564823
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/12671696551866663672
Launched job-id: rarevarian--williamsjacr--240902-144749-64
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'rarevarian--williamsjacr--24090

In [ ]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'rarevarian--williamsjacr--240902-144749-64' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
rarevariantc...       6  Success   09-02 16:33:40
rarevariantc...       5  Success   09-02 15:24:48
rarevariantc...       4  Success   09-02 15:32:14
rarevariantc...       3  Success   09-02 15:28:02
rarevariantc...       2  Success   09-02 15:27:59
rarevariantc...       1  Success   09-02 15:41:49



In [18]:
!gsutil cat gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/rarevarian--williamsjacr--240902-142432-20.1.log

2024-09-02 14:27:36 INFO: gsutil -h Content-Type:text/plain -u terra-vpc-sc-804f445b -mq cp /tmp/continuous_logging_action/output gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/data/logging/rarevarian--williamsjacr--240902-142432-20.1.log
2024-09-02 14:27:37 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Continuous/RareVariant_PRS
2024-09-02 14:27:37 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/GeneCentricCoding
2024-09-02 14:27:37 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels
2024-09-02 14:27:37 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/NullModels
2024-09-02 14:27:37 INFO: mkdir -m 777 -p /mnt/data/input/gs/fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Re

In [8]:
%%writefile Common_Plus_Rare.R
rm(list = ls())
library(readr)
library(dplyr)
library(caret)
library(ranger)
library(SuperLearner)
library(dplyr)
library(boot)
library(stringr)
library(glmnet)

trait <- commandArgs(TRUE)[1]
print(trait)

RareVariantPRS <- commandArgs(TRUE)[2]
print(RareVariantPRS)

CommonVariantPRS <- commandArgs(TRUE)[3]
print(CommonVariantPRS)

all_phenotypes_file <- commandArgs(TRUE)[4]
print(all_phenotypes_file)

OUTPUT_PATH <- commandArgs(TRUE)[5]
print(OUTPUT_PATH)

RV_PRS <- read.csv(RareVariantPRS)
CV_PRS <- read.delim(CommonVariantPRS)

CV_RV_PRS <- inner_join(RV_PRS,CV_PRS)

CV_RV_PRS_raw <- CV_RV_PRS
CV_RV_PRS_adjusted <- CV_RV_PRS

for(i in c("RV_PRS","prs")){
  tmp <- data.frame(y = CV_RV_PRS_adjusted[,i],CV_RV_PRS_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
  mod <- lm(y~.,data = tmp)
  R <- mod$residuals
  tmp <- data.frame(y = R^2,CV_RV_PRS_adjusted[,c("PC1","PC2","PC3","PC4","PC5")])
  mod <- lm(y~.,data = tmp)
  y_hat <- predict(mod,tmp)
  if(sum(y_hat < 0) > 0){
    mod <- lm(y~1,data = tmp)
    y_hat <- predict(mod,tmp)
  }
  if(sum(sqrt(y_hat)) == 0){
    CV_RV_PRS_adjusted[,i] <- 0
  }else{
    CV_RV_PRS_adjusted[,i] <- R/sqrt(y_hat)
  }
}

all_phenotypes <- read.csv(all_phenotypes_file)

CV_RV_PRS_raw_EUR <- CV_RV_PRS_raw[CV_RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
CV_RV_PRS_raw_SAS <- CV_RV_PRS_raw[CV_RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
CV_RV_PRS_raw_AMR <- CV_RV_PRS_raw[CV_RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
CV_RV_PRS_raw_AFR <- CV_RV_PRS_raw[CV_RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
CV_RV_PRS_raw_EAS <- CV_RV_PRS_raw[CV_RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
CV_RV_PRS_raw_MID <- CV_RV_PRS_raw[CV_RV_PRS_raw$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]

CV_RV_PRS_adjusted_EUR <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EUR"],]
CV_RV_PRS_adjusted_SAS <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "SAS"],]
CV_RV_PRS_adjusted_AMR <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AMR"],]
CV_RV_PRS_adjusted_AFR <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "AFR"],]
CV_RV_PRS_adjusted_EAS <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "EAS"],]
CV_RV_PRS_adjusted_MID <- CV_RV_PRS_adjusted[CV_RV_PRS_adjusted$IID %in% all_phenotypes$IID[all_phenotypes$ancestry == "MID"],]

CV_RV_PRS_raw_EUR$Y <- scale(CV_RV_PRS_raw_EUR$Y)
CV_RV_PRS_raw_SAS$Y <- scale(CV_RV_PRS_raw_SAS$Y)
CV_RV_PRS_raw_AMR$Y <- scale(CV_RV_PRS_raw_AMR$Y)
CV_RV_PRS_raw_AFR$Y <- scale(CV_RV_PRS_raw_AFR$Y)
CV_RV_PRS_raw_EAS$Y <- scale(CV_RV_PRS_raw_EAS$Y)
CV_RV_PRS_raw_MID$Y <- scale(CV_RV_PRS_raw_MID$Y)

CV_RV_PRS_adjusted_EUR$Y <- scale(CV_RV_PRS_adjusted_EUR$Y)
CV_RV_PRS_adjusted_SAS$Y <- scale(CV_RV_PRS_adjusted_SAS$Y)
CV_RV_PRS_adjusted_AMR$Y <- scale(CV_RV_PRS_adjusted_AMR$Y)
CV_RV_PRS_adjusted_AFR$Y <- scale(CV_RV_PRS_adjusted_AFR$Y)
CV_RV_PRS_adjusted_EAS$Y <- scale(CV_RV_PRS_adjusted_EAS$Y)
CV_RV_PRS_adjusted_MID$Y <- scale(CV_RV_PRS_adjusted_MID$Y)

CV_RV_PRS_raw_EUR$RV_PRS <- scale(CV_RV_PRS_raw_EUR$RV_PRS)
CV_RV_PRS_raw_SAS$RV_PRS <- scale(CV_RV_PRS_raw_SAS$RV_PRS)
CV_RV_PRS_raw_AMR$RV_PRS <- scale(CV_RV_PRS_raw_AMR$RV_PRS)
CV_RV_PRS_raw_AFR$RV_PRS <- scale(CV_RV_PRS_raw_AFR$RV_PRS)
CV_RV_PRS_raw_EAS$RV_PRS <- scale(CV_RV_PRS_raw_EAS$RV_PRS)
CV_RV_PRS_raw_MID$RV_PRS <- scale(CV_RV_PRS_raw_MID$RV_PRS)

CV_RV_PRS_raw_EUR$prs <- scale(CV_RV_PRS_raw_EUR$prs)
CV_RV_PRS_raw_SAS$prs <- scale(CV_RV_PRS_raw_SAS$prs)
CV_RV_PRS_raw_AMR$prs <- scale(CV_RV_PRS_raw_AMR$prs)
CV_RV_PRS_raw_AFR$prs <- scale(CV_RV_PRS_raw_AFR$prs)
CV_RV_PRS_raw_EAS$prs <- scale(CV_RV_PRS_raw_EAS$prs)
CV_RV_PRS_raw_MID$prs <- scale(CV_RV_PRS_raw_MID$prs)


best_beta_raw_CV_EUR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_EUR))[2]
se_beta_raw_CV_EUR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_EUR))$coefficients[2,2]
best_beta_raw_RV_EUR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_EUR))[3]
se_beta_raw_RV_EUR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_EUR))$coefficients[3,2]

best_beta_raw_CV_SAS <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_SAS))[2]
se_beta_raw_CV_SAS <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_SAS))$coefficients[2,2]
best_beta_raw_RV_SAS <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_SAS))[3]
se_beta_raw_RV_SAS <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_SAS))$coefficients[3,2]

best_beta_raw_CV_AMR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_AMR))[2]
se_beta_raw_CV_AMR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_AMR))$coefficients[2,2]
best_beta_raw_RV_AMR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_AMR))[3]
se_beta_raw_RV_AMR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_AMR))$coefficients[3,2]

best_beta_raw_CV_AFR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_AFR))[2]
se_beta_raw_CV_AFR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_AFR))$coefficients[2,2]
best_beta_raw_RV_AFR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_AFR))[3]
se_beta_raw_RV_AFR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_AFR))$coefficients[3,2]

best_beta_raw_CV_EAS <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_EAS))[2]
se_beta_raw_CV_EAS <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_EAS))$coefficients[2,2]
best_beta_raw_RV_EAS <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_EAS))[3]
se_beta_raw_RV_EAS <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_EAS))$coefficients[3,2]

best_beta_raw_CV_MID <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_MID))[2]
se_beta_raw_CV_MID <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_MID))$coefficients[2,2]
best_beta_raw_RV_MID <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_MID))[3]
se_beta_raw_RV_MID <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_raw_MID))$coefficients[3,2]



best_beta_adjusted_CV_EUR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_EUR))[2]
se_beta_adjusted_CV_EUR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_EUR))$coefficients[2,2]
best_beta_adjusted_RV_EUR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_EUR))[3]
se_beta_adjusted_RV_EUR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_EUR))$coefficients[3,2]

best_beta_adjusted_CV_SAS <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_SAS))[2]
se_beta_adjusted_CV_SAS <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_SAS))$coefficients[2,2]
best_beta_adjusted_RV_SAS <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_SAS))[3]
se_beta_adjusted_RV_SAS <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_SAS))$coefficients[3,2]

best_beta_adjusted_CV_AMR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_AMR))[2]
se_beta_adjusted_CV_AMR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_AMR))$coefficients[2,2]
best_beta_adjusted_RV_AMR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_AMR))[3]
se_beta_adjusted_RV_AMR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_AMR))$coefficients[3,2]

best_beta_adjusted_CV_AFR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_AFR))[2]
se_beta_adjusted_CV_AFR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_AFR))$coefficients[2,2]
best_beta_adjusted_RV_AFR <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_AFR))[3]
se_beta_adjusted_RV_AFR <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_AFR))$coefficients[3,2]

best_beta_adjusted_CV_EAS <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_EAS))[2]
se_beta_adjusted_CV_EAS <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_EAS))$coefficients[2,2]
best_beta_adjusted_RV_EAS <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_EAS))[3]
se_beta_adjusted_RV_EAS <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_EAS))$coefficients[3,2]

best_beta_adjusted_CV_MID <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_MID))[2]
se_beta_adjusted_CV_MID <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_MID))$coefficients[2,2]
best_beta_adjusted_RV_MID <- coef(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_MID))[3]
se_beta_adjusted_RV_MID <- summary(lm(Y~prs + RV_PRS,data = CV_RV_PRS_adjusted_MID))$coefficients[3,2]


CV_PRS_Results <- data.frame(trait = trait,ancestry = c("EUR","SAS","AMR","AFR","EAS","MID"), 
                             beta_raw = c(best_beta_raw_CV_EUR,best_beta_raw_CV_SAS,best_beta_raw_CV_AMR,best_beta_raw_CV_AFR,best_beta_raw_CV_EAS,best_beta_raw_CV_MID), 
                             se_raw = c(se_beta_raw_CV_EUR,se_beta_raw_CV_SAS,se_beta_raw_CV_AMR,se_beta_raw_CV_AFR,se_beta_raw_CV_EAS,se_beta_raw_CV_MID), 
                             beta_adjusted = c(best_beta_adjusted_CV_EUR,best_beta_adjusted_CV_SAS,best_beta_adjusted_CV_AMR,best_beta_adjusted_CV_AFR,best_beta_adjusted_CV_EAS,best_beta_adjusted_CV_MID), 
                             se_adjusted = c(se_beta_adjusted_CV_EUR,se_beta_adjusted_CV_SAS,se_beta_adjusted_CV_AMR,se_beta_adjusted_CV_AFR,se_beta_adjusted_CV_EAS,se_beta_adjusted_CV_MID),
                             Method = "CV")

RV_PRS_Results <- data.frame(trait = trait,ancestry = c("EUR","SAS","AMR","AFR","EAS","MID"), 
                             beta_raw = c(best_beta_raw_RV_EUR,best_beta_raw_RV_SAS,best_beta_raw_RV_AMR,best_beta_raw_RV_AFR,best_beta_raw_RV_EAS,best_beta_raw_RV_MID), 
                             se_raw = c(se_beta_raw_RV_EUR,se_beta_raw_RV_SAS,se_beta_raw_RV_AMR,se_beta_raw_RV_AFR,se_beta_raw_RV_EAS,se_beta_raw_RV_MID), 
                             beta_adjusted = c(best_beta_adjusted_RV_EUR,best_beta_adjusted_RV_SAS,best_beta_adjusted_RV_AMR,best_beta_adjusted_RV_AFR,best_beta_adjusted_RV_EAS,best_beta_adjusted_RV_MID), 
                             se_adjusted = c(se_beta_adjusted_RV_EUR,se_beta_adjusted_RV_SAS,se_beta_adjusted_RV_AMR,se_beta_adjusted_RV_AFR,se_beta_adjusted_RV_EAS,se_beta_adjusted_RV_MID),
                             Method = "RV")


write.csv(rbind(CV_PRS_Results,RV_PRS_Results),file = paste0(OUTPUT_PATH,"/",trait,"Best_Betas.csv"),row.names = FALSE)


Overwriting Common_Plus_Rare.R


In [9]:
%%writefile Common_Plus_Rare.sh
#!/bin/bash

set -o errexit
set -o nounset

Rscript ${R_Script} ${trait} ${RareVariantPRS} ${CommonVariantPRS} ${all_phenotypes_file} ${OUTPUT_PATH}

Writing Common_Plus_Rare.sh


In [10]:
%%writefile score_task.R

tasks <- data.frame(check.names = FALSE)

for(trait in c("BMI","LDL","HDL","logTG","TC","Height")){
 tasks <- rbind(tasks, data.frame(
            '--env trait'=trait,
            '--input R_Script'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/Common_Plus_Rare.R",
            '--input RareVariantPRS'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/RareVariant_PRS/",trait,"_BestPRS.csv"),
            '--input CommonVariantPRS'=paste0("gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/OneCommonPRS/",trait,"_Best_Validation_All.txt"),
            '--input all_phenotypes_file'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/all_phenotypes.csv",
            '--output-recursive OUTPUT_PATH'="gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Results/Continuous/Common_Plus_Rare/",
            check.names = FALSE
        ))   
}

   
write.table(tasks, 
            file="score_task.txt", 
            row.names=F, col.names=T, 
            sep='\t', quote=F)

Overwriting score_task.R


In [11]:
!Rscript score_task.R

In [12]:
!gsutil -m cp Common_Plus_Rare.R gs://fc-secure-797107a7-4402-4122-941c-9a486e0d633e/JW/AoU_Phenotypes/Scripts/

Copying file://Common_Plus_Rare.R [Content-Type=application/octet-stream]...
/ [1/1 files][ 10.0 KiB/ 10.0 KiB] 100% Done                                    
Operation completed over 1 objects/10.0 KiB.                                     


In [13]:
%%bash --out score_batch

source ~/aou_dsub.bash # This file was created via notebook 01_dsub_setup.ipynb.

aou_dsub \
  --image willja16/r_with_plink \
  --disk-size 100 \
  --boot-disk-size 25 \
  --min-ram 16 \
  --timeout "96h" \
  --logging "${WORKSPACE_BUCKET}/data/logging" \
  --script Common_Plus_Rare.sh \
  --tasks score_task.txt

Job properties:
  job-id: common-plu--williamsjacr--240807-165037-00
  job-name: common-plus-rare
  user-id: williamsjacr
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/11963387640502094454
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/7050916832642191274
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/16627059054397606154
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/17531345581312317542
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/6893272649865612905
Provider internal-id (operation): projects/52933917155/locations/us-central1/operations/15003385858120078740
Launched job-id: common-plu--williamsjacr--240807-165037-00
6 task(s)
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'common-plu--williamsjacr--240807-1

In [18]:
!dstat --provider google-cls-v2 --project terra-vpc-sc-804f445b --location us-central1 --jobs 'common-plu--williamsjacr--240807-165037-00' --users 'williamsjacr' --status '*'

Job Name           Task  Status    Last Update
---------------  ------  --------  --------------
common-plus-...       6  Success   08-07 16:54:03
common-plus-...       5  Success   08-07 16:54:02
common-plus-...       4  Success   08-07 16:54:08
common-plus-...       3  Success   08-07 16:54:07
common-plus-...       2  Success   08-07 16:54:14
common-plus-...       1  Success   08-07 16:53:58

